## Introduction: Maritme Data System PyData Competition

The code below:
1. Loads a dataset of vessel positions, speed and heading
2. Builds a graph <-- <your code here>
3. Plots a graph
4. Evaluates the graph

To win this competition, you need to submit the graph with the least number of nodes, and the least number of land intersections. The graph should allow you to route from any node to any other node.

The winners of this competition will be posted on our Github and will be rewared with 2 tickets to the ElbPhilharmonie.

## 1- Load data

In [74]:
import pandas as pd

def load_dataframe():
    DEHAM_FILE = 'data/hamburg-parsedais.csv'
    # HEADERS = ['time','cog','heading','lat','lon','mmsi','sog']
    deham_df = pd.read_csv(DEHAM_FILE)
    return deham_df

# Retrieve data
deham_df = load_dataframe()
# deham_df.head(2)

/Users/pierregarreau/Virtualenvs/pydata-hamburg-manifoldlearning/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


## 2- Build your graph

In [107]:
import networkx as nx

# --- --- --- --- --- --- --- --- ----
# --- --- --- --- --- --- --- --- ----
# --- BUILD GRAPH, YOUR MAGIC HERE ---
# --- --- --- --- --- --- --- --- ----
# --- --- --- --- --- --- --- --- ----

def build(G: nx.Graph, df: pd.DataFrame) -> None:
    # -- Build nodes
    # -- For instance
    MAX_NODES = 10000
    node_count = 0
    print('-- build nodes --')
    for index, row in df.iterrows():
        node_count += 1
        lon_lat = (row['lon'], row['lat'])
        G.add_node(index, position=lon_lat, source='ais')
        if node_count > MAX_NODES:
            break
    print('-- done --')
    
    # -- Build edges
    # -- For instance
    MAX_EDGES = 50
    edge_count = 0
    print('-- build edges --')
    for node_id in G.nodes():
        edge_count += 1
        G.add_edge(node_id, node_id + 1)
        if edge_count > MAX_EDGES:
            break
    print('-- done --')
    
# Build your graph
G = nx.Graph()
build(G, deham_df)

-- build nodes --
-- done --
-- build edges --
-- done --


## 3- Visualize

In [110]:
import numpy as np

from pyproj import Proj, transform
from bokeh.plotting import figure, show
from bokeh.tile_providers import STAMEN_TONER
from bokeh.io import output_notebook
output_notebook()

# VISUALIZE DATA
def plot(G: nx.Graph) -> None:
    # Modes
    from_proj = Proj(init="epsg:4326")
    to_proj = Proj(init="epsg:3857")

    # Nodes
    all_nodes = G.nodes
    lonlat_arr = np.array([node.get('position') for node in all_nodes.values()])
    x, y = transform(from_proj, to_proj, lonlat_arr[:, 0], lonlat_arr[:, 1])

    fig = figure(tools='pan, wheel_zoom')
    fig.axis.visible = False
    fig.add_tile(STAMEN_TONER)
    fig.circle(x, y, color='firebrick')

    # Edges
    edge_x = []
    edge_y = []
    positions = {node_id: G.node[node_id]['position'] for node_id in G.nodes()}
    count = 0
    for edge in G.edges:
        x, y = transform(from_proj, to_proj, 
                         [positions[edge[0]][0], positions[edge[1]][0]],
                         [positions[edge[0]][1], positions[edge[1]][1]])
        edge_x.append([x[0], x[1]])
        edge_y.append([y[0], y[1]])
        count += 1
        if count > 10:
            break

    fig.multi_line(edge_x, edge_y, color="blue", alpha=0.5, line_width=2)
    
    show(fig)
    
# Plot Dataset
plot(G)

Loading BokehJS ...

## 4- Results

In [118]:
from shapely.wkt import loads
from shapely.geometry import LineString
from shapely.geometry.base import BaseGeometry

# Report here, grading
DEHAM_POLYGON = 'POLYGON((10.0415182 53.528001,10.0415503 53.5279539,10.0425017 53.5276032,10.0433386 53.5272604,10.0442847 53.5269108,10.0451999 53.5265158,10.0459878 53.5261779,10.0473418 53.5255657,10.0486518 53.5249559,10.0512926 53.5234993,10.0516286 53.5236623,10.052026 53.5234071,10.0517347 53.5232492,10.0542724 53.52168,10.0556475 53.520761,10.0565817 53.5200958,10.0578635 53.5191008,10.0587354 53.5182899,10.0595354 53.5174587,10.0601447 53.5166254,10.0606016 53.5158254,10.0608735 53.5151463,10.0613519 53.513794,10.0614668 53.5116076,10.0609901 53.508895,10.0609423 53.5085737,10.0614058 53.5085226,10.06132 53.5080224,10.0610367 53.5081092,10.0607792 53.5081449,10.0602223 53.5070546,10.0596552 53.5061622,10.0590969 53.5053374,10.058951 53.5052863,10.0583416 53.5045819,10.0572687 53.5032137,10.056653 53.502306,10.0562725 53.5017526,10.0559635 53.5013339,10.0564591 53.5012059,10.0566158 53.5011655,10.0577316 53.5011859,10.0588388 53.5014462,10.0593109 53.5014871,10.0598516 53.5013441,10.0604353 53.5009919,10.0603409 53.5008336,10.0597315 53.500803,10.0589075 53.5008234,10.0580749 53.5009255,10.0576458 53.5010072,10.0572681 53.5010685,10.0567445 53.5010634,10.0563669 53.5011399,10.0558948 53.5012625,10.0550022 53.4999912,10.0543675 53.499061,10.053532 53.497722,10.053077 53.496701,10.052616 53.495424,10.0524791 53.4944,10.052546 53.492607,10.0526177 53.4914879,10.0527941 53.490148,10.0530211 53.4890164,10.053167 53.4882401,10.0533352 53.4869966,10.0536906 53.4857326,10.0542656 53.4849103,10.0547806 53.4843281,10.0556647 53.4833935,10.0568573 53.482199,10.05891 53.480766,10.0602652 53.4797364,10.0610708 53.4789535,10.0615128 53.4783244,10.0617336 53.4783763,10.0618953 53.4781202,10.061696 53.4780716,10.0619302 53.4776829,10.06259 53.4764023,10.0630538 53.4753578,10.0635419 53.4740701,10.0638144 53.4727632,10.063914 53.4710038,10.0639686 53.4699166,10.0640781 53.4691902,10.0643637 53.467982,10.0644724 53.4677093,10.0648387 53.4677644,10.0649138 53.4675453,10.0645426 53.4674898,10.065003 53.4663773,10.0653977 53.4653737,10.0657069 53.4642977,10.066239 53.4630509,10.0669257 53.4620238,10.0679814 53.4610478,10.0690694 53.4602279,10.0702816 53.4593563,10.0716292 53.458508,10.0724875 53.4580583,10.0738522 53.4574655,10.0750881 53.457026,10.0757293 53.456845,10.0780665 53.4561112,10.0792649 53.4557218,10.0802673 53.4554064,10.0816404 53.4548014,10.0830272 53.454122,10.0844352 53.4532387,10.085904 53.4521081,10.0862976 53.4518303,10.0871919 53.4510932,10.0881129 53.4503481,10.0885499 53.4500398,10.0898564 53.4489129,10.0938083 53.4463636,10.0939812 53.4463026,10.0941931 53.446283,10.0943881 53.4463307,10.094495 53.4464282,10.094508 53.4465902,10.0942556 53.4472177,10.0959487 53.4477104,10.0959512 53.4480353,10.0970816 53.4483935,10.0972153 53.4481786,10.0979179 53.4472972,10.098091 53.447256,10.0987242 53.4464153,10.0994654 53.4455274,10.1003658 53.444403,10.1009297 53.4436138,10.1007491 53.443249,10.100608 53.4431914,10.100467 53.4431811,10.1003735 53.4429917,10.1002112 53.4427641,10.09999 53.4426552,10.0997861 53.4426447,10.0995249 53.4427,10.0984878 53.4435658,10.0964421 53.4452584,10.0963509 53.4453861,10.0960911 53.445496,10.0958147 53.4455355,10.0956164 53.4455145,10.0949603 53.4452474,10.0949205 53.4451914,10.0949342 53.4451507,10.0950154 53.4450954,10.0960462 53.4445538,10.0977847 53.4430894,10.099399 53.4417074,10.1000663 53.4411327,10.1004225 53.4405304,10.101001 53.439394,10.100982 53.438626,10.101793 53.436825,10.1020567 53.4366261,10.1024604 53.4363313,10.102631 53.436049,10.1033876 53.4354222,10.1035035 53.4353195,10.1035846 53.4351421,10.103878 53.4345,10.1040257 53.4336554,10.1045795 53.4331386,10.1044863 53.4330405,10.104539 53.4329799,10.1046354 53.4329805,10.1048074 53.4328756,10.1048606 53.4327805,10.1047474 53.4327178,10.1048227 53.4326384,10.1049425 53.4326585,10.1052373 53.4324707,10.1054297 53.4322243,10.105442 53.4320536,10.1053806 53.4319511,10.1053871 53.4318915,10.1069308 53.4302126,10.1069568 53.4301942,10.1070799 53.4301924,10.1071363 53.4301234,10.107064 53.4300838,10.1070626 53.4300581,10.1073954 53.4296941,10.1074773 53.4296648,10.1076656 53.4296014,10.107768 53.4294648,10.1077106 53.4294233,10.1077802 53.429316,10.1078539 53.4293038,10.1078539 53.4292355,10.1079358 53.4291403,10.1079932 53.4291135,10.1081037 53.4291233,10.1082156 53.4291745,10.1085991 53.4291086,10.1094344 53.4286354,10.1098602 53.4282743,10.109935 53.4279328,10.109983 53.4278791,10.1102123 53.4279035,10.1113423 53.4275132,10.1120957 53.4271131,10.1122365 53.4269254,10.1123839 53.4269317,10.1124881 53.4269662,10.1144974 53.4264038,10.1147348 53.4261244,10.1148208 53.4260909,10.1149462 53.42618,10.1152164 53.4261849,10.1157569 53.426058,10.1167149 53.4257336,10.1173618 53.4257311,10.1185901 53.4254481,10.1187293 53.4252773,10.1188253 53.4252611,10.1190111 53.4253917,10.1192247 53.4254265,10.1210015 53.4250986,10.1229239 53.4249744,10.1246269 53.4250091,10.1261448 53.4251006,10.1266729 53.4251711,10.1286707 53.4251144,10.1314177 53.4249443,10.1337148 53.424598,10.1359485 53.4240428,10.1394063 53.4227157,10.1406769 53.4220244,10.142345 53.421073,10.1438957 53.4196958,10.1449156 53.4188724,10.1449493 53.4187872,10.1448358 53.4187246,10.1448197 53.4186675,10.1448857 53.4186251,10.1451085 53.4186601,10.1451674 53.4186576,10.1460587 53.4178809,10.1460839 53.4178358,10.145962 53.4177255,10.1460377 53.4176729,10.1462142 53.4176955,10.1462773 53.4176904,10.1471308 53.4169688,10.1471686 53.4169011,10.1470635 53.4168134,10.1470841 53.4167516,10.1471981 53.4167257,10.1473116 53.4167633,10.1474083 53.4167558,10.1480431 53.4161344,10.1480473 53.4160041,10.1479014 53.4159209,10.1479519 53.4158766,10.1480292 53.4158695,10.1482581 53.4159316,10.1483324 53.4158288,10.1481095 53.4157597,10.1480949 53.4156945,10.148817 53.414606,10.1494093 53.4138463,10.1502264 53.4126858,10.1509424 53.4117157,10.1526044 53.4097329,10.1528811 53.4093937,10.1531226 53.4092033,10.1538591 53.4088826,10.1542094 53.40857,10.154417 53.4082424,10.1544353 53.4080313,10.1544597 53.4079258,10.1548077 53.4076455,10.1547803 53.4075691,10.1547299 53.4075304,10.1547752 53.4074853,10.154881 53.4075035,10.1550031 53.4074594,10.1563068 53.4064418,10.1563343 53.4063645,10.1561939 53.4062762,10.1563145 53.4062198,10.156461 53.4062899,10.1565551 53.4062617,10.1574196 53.405668,10.1575936 53.4056589,10.1579661 53.4058973,10.158018 53.4058209,10.1580607 53.4058145,10.1581462 53.4058755,10.1585156 53.4055879,10.1585308 53.4055097,10.1583233 53.4054241,10.1582011 53.4054369,10.1580546 53.4055515,10.157905 53.4055515,10.1577982 53.4054733,10.1578653 53.4054005,10.15897 53.4046599,10.1592326 53.4044743,10.1595002 53.4041078,10.1598126 53.4040521,10.1601118 53.4039392,10.1599164 53.4035716,10.160069 53.4035789,10.1602095 53.4036735,10.1608628 53.4035133,10.1621755 53.4025997,10.162261 53.4024541,10.1621877 53.4023303,10.1623465 53.4022939,10.1626334 53.4023121,10.163031 53.4022705,10.1635815 53.4019539,10.163901 53.40165,10.1646091 53.4011431,10.1646004 53.4010478,10.1647358 53.4010196,10.1649976 53.4012049,10.1663004 53.400595,10.1679662 53.39991,10.1687501 53.3994963,10.1691073 53.3991827,10.1692147 53.3991742,10.1693937 53.3991656,10.1708113 53.3987191,10.1710975 53.3985638,10.1712407 53.3985424,10.171391 53.3986107,10.171842 53.3985937,10.1727431 53.3984161,10.1732733 53.3981269,10.1736487 53.3982155,10.1741069 53.3981984,10.1746366 53.3981301,10.1749373 53.3980448,10.1749838 53.3979167,10.175152 53.3979039,10.1751784 53.3980209,10.1773792 53.3977587,10.1799286 53.3976413,10.1824682 53.3978534,10.1832915 53.3979929,10.1833215 53.3979388,10.1835116 53.3979047,10.1839256 53.3979881,10.1840049 53.3982883,10.1843855 53.3982725,10.1845769 53.3981039,10.1847417 53.3981167,10.1849748 53.398158,10.1851999 53.3984212,10.1853285 53.3987058,10.1858009 53.3988822,10.1860665 53.3986984,10.1861801 53.3987599,10.1858493 53.3989938,10.1857981 53.3993727,10.1858763 53.3994283,10.1860562 53.3994901,10.1871218 53.3998453,10.1877677 53.4005111,10.1881783 53.4006328,10.1882808 53.4005807,10.1877422 53.4001211,10.187574 53.399893,10.1875822 53.3998325,10.1880948 53.3993869,10.18873 53.3988626,10.1887367 53.3988392,10.1887091 53.3988239,10.1866214 53.3984667,10.1865402 53.3984757,10.1864862 53.398533,10.1864486 53.3985452,10.1864029 53.3985463,10.1863507 53.3985321,10.1863225 53.3984992,10.1862965 53.3984559,10.1863158 53.3984026,10.1863894 53.3983489,10.1864754 53.3983312,10.1888278 53.3987347,10.1889021 53.3987582,10.1889489 53.3987981,10.1889379 53.3988378,10.187986 53.3997093,10.1879672 53.3998455,10.1879833 53.3999412,10.1885006 53.4004033,10.1886836 53.4003819,10.1900542 53.4007043,10.1917781 53.4011334,10.1922197 53.4011622,10.1923644 53.4010893,10.192225 53.4009928,10.1923466 53.4009252,10.1926621 53.4008827,10.192911 53.4011232,10.1950053 53.4013074,10.1960696 53.4011027,10.1972712 53.4011897,10.1980351 53.4013228,10.1981467 53.4011488,10.1987046 53.4012972,10.1994771 53.4013381,10.2003017 53.4015372,10.2015683 53.4016018,10.2031883 53.4014227,10.2045431 53.4011266,10.204996 53.4010413,10.2060861 53.4006422,10.206211 53.4005134,10.206211 53.4001909,10.2059573 53.4000997,10.205393 53.4000865,10.2049427 53.399916,10.2047493 53.3999448,10.2048038 53.4000532,10.2045325 53.4001765,10.2038115 53.4002891,10.2036855 53.4004554,10.2035707 53.4011072,10.2036056 53.4004272,10.2034973 53.400246,10.2031877 53.4001565,10.2029275 53.4002328,10.2026463 53.4003528,10.2023009 53.4003761,10.2019795 53.4003906,10.2017087 53.4006166,10.2015607 53.4003259,10.2035885 53.399719,10.2037343 53.3995624,10.2037173 53.3993968,10.2063178 53.3998541,10.2063865 53.3999974,10.2063125 53.4003388,10.2064122 53.4004272,10.206564 53.4004725,10.2074404 53.4001792,10.2074068 53.3999393,10.2085837 53.3997722,10.2085116 53.3994767,10.206335 53.3995624,10.2045819 53.399279,10.2088241 53.3984365,10.2087726 53.3994652,10.2090051 53.3996582,10.2106093 53.3995573,10.2108146 53.3994036,10.2123734 53.39917,10.2129182 53.3989943,10.2126779 53.3986566,10.2124289 53.398457,10.2114767 53.3987323,10.2113235 53.398681,10.2111329 53.398938,10.210565 53.3987192,10.209854 53.3988152,10.2093305 53.3989432,10.2091073 53.3990967,10.2091502 53.3989227,10.2108027 53.3981938,10.2114161 53.3981755,10.2126152 53.3977931,10.214075 53.3975831,10.2140852 53.3977821,10.2143172 53.3977405,10.2147088 53.3977155,10.2150436 53.3974492,10.217596 53.3970906,10.2201343 53.3968679,10.2201709 53.3970138,10.2205829 53.3969984,10.2205829 53.3967937,10.2231234 53.3966095,10.2269257 53.3964252,10.2270802 53.3967937,10.2273635 53.3967937,10.2273892 53.3964099,10.2306937 53.3963485,10.2311145 53.3929898,10.229461 53.393178,10.227917 53.3934459,10.2257378 53.3936241,10.2237221 53.3937863,10.2215466 53.3938732,10.2178004 53.3938926,10.2175966 53.3944876,10.217179 53.394126,10.2157834 53.3940973,10.2153331 53.3944644,10.2155332 53.394854,10.2149683 53.394324,10.2137474 53.3944532,10.2133477 53.3948083,10.2135752 53.3951627,10.2129296 53.3946987,10.212277 53.3949071,10.2119538 53.3951344,10.2121772 53.3954695,10.211483 53.3950101,10.2107283 53.3952133,10.2105054 53.3954478,10.2107329 53.3956967,10.2101615 53.3952425,10.2095822 53.3952489,10.2090248 53.3954158,10.2088271 53.3957433,10.2090449 53.3960459,10.2084342 53.3954216,10.2082188 53.3954864,10.2072473 53.3960692,10.2073396 53.3963776,10.2067819 53.3957927,10.2056876 53.3960869,10.2056018 53.3960166,10.2063855 53.3957869,10.2064126 53.3955059,10.2062031 53.3953135,10.2059529 53.3952948,10.2056101 53.3953403,10.205613 53.3957485,10.2054288 53.3959458,10.2059271 53.39666,10.2051487 53.3959995,10.2046627 53.3952541,10.2038262 53.3954389,10.2036086 53.3956089,10.2037136 53.3958013,10.2040888 53.3959086,10.204614 53.395752,10.2047041 53.3958192,10.2042013 53.3962084,10.2041338 53.3964768,10.204509 53.3968929,10.2043664 53.3969555,10.2039162 53.3966379,10.203173 53.3956598,10.2027773 53.3956071,10.2019222 53.3958109,10.2009567 53.3960364,10.200918 53.3962049,10.2018745 53.3973319,10.2016786 53.39738,10.2006682 53.3961576,10.1998001 53.3964798,10.2002265 53.3970685,10.2000344 53.3971235,10.1995431 53.3965386,10.1987031 53.3967779,10.1992085 53.3975449,10.1989857 53.3975976,10.1983893 53.3967756,10.196239 53.3967226,10.1959458 53.3968546,10.1961929 53.3975289,10.1959662 53.3975518,10.1957127 53.3968211,10.1945064 53.3965233,10.1932733 53.3966447,10.1931888 53.3968234,10.1932349 53.3971464,10.193062 53.3971693,10.1930389 53.3967822,10.19316 53.3964366,10.192695 53.3961701,10.1920083 53.3961701,10.1917827 53.3965096,10.191656 53.3964936,10.191752 53.3962279,10.1915561 53.3961316,10.1917723 53.3959654,10.1912872 53.3958155,10.191195 53.3959667,10.1905381 53.39604,10.190419 53.3963034,10.1904267 53.3967318,10.190273 53.396741,10.1902538 53.3962691,10.1904771 53.3957228,10.1897987 53.3954095,10.1896663 53.3956962,10.1890708 53.3957741,10.1890171 53.3960856,10.1888826 53.3960833,10.188971 53.3953595,10.188825 53.3952312,10.1880451 53.3951556,10.1877275 53.3952489,10.1876571 53.3962208,10.1874804 53.3962185,10.1875015 53.3954744,10.18702 53.3952367,10.1862148 53.3952745,10.1855067 53.3951849,10.18482 53.3953001,10.1848251 53.3957225,10.1846484 53.3957316,10.1846591 53.3951529,10.1840153 53.3949418,10.1832 53.3949674,10.1831502 53.3951498,10.1832462 53.3955575,10.1830695 53.3955758,10.1829504 53.3950925,10.1829926 53.3949336,10.1824549 53.394781,10.1816657 53.394801,10.1813523 53.3949482,10.1815022 53.3954315,10.1813024 53.3954613,10.1811351 53.39492,10.1803783 53.394833,10.1797882 53.3950378,10.1798964 53.3953445,10.1797508 53.3953738,10.1795513 53.3948813,10.1791506 53.3948486,10.1787362 53.3949528,10.1786248 53.3950582,10.1787785 53.395317,10.1786095 53.3953422,10.178494 53.3950891,10.1778666 53.3949921,10.1774386 53.3951529,10.1775838 53.3953491,10.1774186 53.395372,10.1772419 53.3950834,10.1770416 53.3949674,10.176355 53.3950186,10.1762107 53.3952399,10.1763676 53.3954609,10.1762178 53.3954962,10.1761033 53.3952383,10.1759365 53.3950378,10.1756248 53.3949625,10.1744289 53.3955844,10.1743903 53.3955697,10.1755197 53.394909,10.1757494 53.3948074,10.1751132 53.3946146,10.1731842 53.3952615,10.1727451 53.3952613,10.1723482 53.3954647,10.172238 53.3954238,10.1721215 53.3954287,10.1719309 53.3955027,10.1719131 53.3955767,10.1718378 53.3956322,10.1715053 53.3956877,10.1706407 53.3959257,10.170602 53.3960003,10.1705565 53.3960552,10.1704324 53.3960526,10.1702816 53.395968,10.1692885 53.3962825,10.1691644 53.3963592,10.1692728 53.396421,10.1692265 53.3964993,10.1691112 53.3965231,10.168956 53.3964332,10.1688984 53.3964174,10.1678432 53.3967954,10.1677989 53.3968826,10.1678739 53.3969486,10.167759 53.3970069,10.1675329 53.3969355,10.1667216 53.3971999,10.1667113 53.3973383,10.1666418 53.3973585,10.1664423 53.3973267,10.1658393 53.3975356,10.1658689 53.3976265,10.1657728 53.3976889,10.1656531 53.3976678,10.1655201 53.3976545,10.1647044 53.3979876,10.1647088 53.3980722,10.1646423 53.3981198,10.1644384 53.3981039,10.1635206 53.3984714,10.1635251 53.3985771,10.1634231 53.3986485,10.1632458 53.3985982,10.162328 53.3989868,10.1624367 53.399081,10.162352 53.3991587,10.1620986 53.3991168,10.1612896 53.3994463,10.1614208 53.3995373,10.161372 53.3996046,10.1612896 53.3996247,10.1611485 53.3995527,10.1610206 53.3995572,10.1599326 53.4000721,10.1587736 53.400657,10.1587403 53.4007321,10.1589353 53.4008259,10.1589152 53.4009097,10.1587857 53.4009161,10.158613 53.4008466,10.1584964 53.4008647,10.1564931 53.4020848,10.1565017 53.402162,10.1566615 53.4022366,10.156601 53.402301,10.1564197 53.4022804,10.1562168 53.402283,10.1555174 53.4027102,10.1546841 53.4031761,10.1546712 53.4032379,10.1548482 53.4032997,10.1548957 53.4033614,10.1548482 53.4034026,10.1547014 53.4033949,10.1545201 53.4033151,10.1543517 53.4033563,10.1533628 53.4041353,10.1533377 53.4041728,10.1534644 53.404211,10.1534812 53.4042674,10.1533927 53.4042902,10.1532217 53.4042465,10.1531551 53.4042521,10.1520618 53.4051002,10.1520608 53.4051458,10.1521817 53.4051934,10.1522033 53.4052642,10.1521126 53.4053015,10.151914 53.4052668,10.1518115 53.4052967,10.1509541 53.4059431,10.1509323 53.4060393,10.1513719 53.4061303,10.1513413 53.406214,10.1504681 53.4061076,10.1497905 53.4063414,10.1496745 53.4063486,10.1495611 53.4062918,10.1494985 53.4063128,10.1494241 53.4063377,10.1494669 53.4064506,10.1493287 53.4065549,10.1489906 53.4070293,10.1485445 53.407466,10.1485755 53.4076699,10.1496256 53.4078555,10.149524 53.4079405,10.1483129 53.4077099,10.147728 53.407556,10.1474183 53.4078216,10.1469308 53.4082397,10.1469755 53.4083755,10.147111 53.408475,10.1472779 53.408533,10.148734 53.4087174,10.1487746 53.408775,10.1487201 53.4088272,10.146766 53.4085825,10.1465371 53.4086682,10.1461176 53.409277,10.1469026 53.4095331,10.1479831 53.4096233,10.1480315 53.409681,10.1478991 53.4097299,10.1463283 53.4095925,10.1457616 53.4096121,10.1453349 53.4100499,10.1454706 53.4103524,10.1457501 53.4104806,10.1467899 53.41048,10.1472724 53.4105057,10.1473165 53.4105596,10.1472556 53.4106047,10.1469444 53.4105921,10.1467468 53.4106134,10.1466185 53.4107466,10.1461855 53.4113214,10.1462094 53.4114026,10.1466023 53.4114201,10.1466417 53.4114743,10.1465557 53.4115254,10.1458188 53.4114663,10.1457399 53.411344,10.1461021 53.4107609,10.14603 53.4106874,10.1455564 53.410597,10.1449927 53.4106021,10.1445502 53.4113168,10.1444632 53.4115457,10.1443966 53.412165,10.1445016 53.4122203,10.1459497 53.4122734,10.1459874 53.4123174,10.1459399 53.4123642,10.1445674 53.4123407,10.1444163 53.4124052,10.1446077 53.4130421,10.1446653 53.413098,10.1452911 53.4131376,10.1453444 53.4131729,10.1452616 53.4132196,10.1447366 53.4131903,10.144679 53.4132311,10.1447015 53.4134484,10.1445864 53.4136862,10.1435524 53.4145664,10.1425461 53.41518,10.1416513 53.4157245,10.1417254 53.4160104,10.1407302 53.4163821,10.1406551 53.4168297,10.1399556 53.4167926,10.1388055 53.4176523,10.138417 53.418548,10.1376591 53.4184551,10.1373174 53.419318,10.1359423 53.4200587,10.1354227 53.4201872,10.133759 53.42077,10.1328689 53.4213739,10.1320817 53.4215488,10.1314755 53.4217978,10.130224 53.4219346,10.1299724 53.4219829,10.1290082 53.421943,10.1282904 53.4220115,10.1273566 53.4219132,10.1267648 53.4220438,10.1265727 53.4219921,10.1252126 53.4220423,10.1247574 53.4219343,10.1234914 53.4217617,10.1225536 53.4218614,10.1228155 53.4223115,10.1226546 53.4223499,10.1222903 53.4218894,10.1211278 53.422013,10.1212491 53.4223435,10.1211525 53.4223627,10.1209058 53.4220366,10.1195893 53.4220965,10.1198543 53.4224586,10.1197201 53.4225058,10.1192861 53.4221103,10.118787 53.4218817,10.117071 53.422208,10.1166494 53.4224667,10.1168541 53.4228229,10.1167641 53.4228278,10.1164693 53.4224862,10.1161663 53.4224716,10.1154228 53.4226816,10.1152655 53.4229303,10.1153638 53.4230572,10.1152155 53.423106,10.11511 53.4230572,10.113767 53.4233158,10.1136769 53.4234085,10.1135641 53.4234169,10.1131945 53.4230454,10.1128721 53.4230436,10.1126288 53.423389,10.1120965 53.4235744,10.1120415 53.4238056,10.1119164 53.4238428,10.1116216 53.4235988,10.111163 53.4236232,10.11077 53.4237842,10.1106717 53.4239892,10.1109665 53.4241599,10.1108355 53.4242185,10.1102623 53.4238672,10.1099593 53.4238867,10.1094844 53.4240721,10.1091486 53.4244088,10.1094434 53.4246479,10.1093151 53.424708,10.1086655 53.4242136,10.1080104 53.4239989,10.1072638 53.4238068,10.1066511 53.4239208,10.1061107 53.4242039,10.1062335 53.4244283,10.106471 53.4243698,10.1067248 53.4246284,10.1079122 53.4251749,10.1078466 53.4252285,10.1057094 53.4242722,10.1055293 53.4242711,10.1052869 53.4243728,10.1045752 53.4238011,10.1043262 53.4236868,10.1040522 53.4237517,10.1048619 53.4247933,10.1049842 53.4251104,10.1054603 53.4255517,10.1058965 53.4262734,10.10579 53.4262929,10.1052875 53.4256158,10.1048077 53.4252809,10.1032806 53.4251464,10.1031733 53.4254401,10.1022404 53.4266132,10.1044901 53.4268945,10.1047911 53.4269808,10.1047788 53.4270649,10.1021681 53.4267363,10.1015614 53.427385,10.1013581 53.4280784,10.1017548 53.4282429,10.101872 53.4285194,10.1022633 53.4285753,10.1027344 53.4286668,10.1027533 53.4287896,10.1011425 53.4285901,10.1008076 53.4287136,10.1005524 53.4292458,10.1004487 53.4298682,10.1006082 53.4300202,10.101587 53.4301646,10.1015731 53.430234,10.1005604 53.430101,10.1003052 53.4301437,10.1001776 53.4316213,10.1004751 53.4317605,10.1004408 53.4318256,10.1001218 53.4318113,10.1000022 53.4318779,10.0997789 53.4324955,10.0998427 53.4325525,10.0999266 53.4325994,10.0998666 53.4326713,10.0996752 53.4326903,10.0990054 53.4332984,10.0983834 53.4341772,10.0987256 53.4342398,10.0987008 53.4343223,10.0982741 53.4343152,10.0980363 53.4346684,10.0979374 53.4349983,10.0979049 53.4350511,10.0975898 53.4353545,10.0976117 53.4355619,10.0977842 53.4355878,10.0977778 53.4356784,10.0976992 53.4356934,10.0973757 53.4356506,10.0971123 53.4359606,10.0968894 53.4363675,10.0969691 53.4365208,10.0968956 53.4365791,10.0967709 53.4366093,10.09675 53.4368271,10.096902 53.4368757,10.0969187 53.4369506,10.096824 53.4369707,10.0966091 53.436952,10.0964369 53.4369667,10.0956173 53.4377024,10.0951178 53.4382488,10.0951978 53.4385632,10.0950864 53.4389617,10.0952983 53.4389922,10.0953595 53.4390379,10.0953751 53.4390854,10.0952624 53.4391151,10.0950033 53.439055,10.0946361 53.4390793,10.0946717 53.4393036,10.0948536 53.4393325,10.0948021 53.4395556,10.0939437 53.4401122,10.0935052 53.4406252,10.0935369 53.4410186,10.0937569 53.4410739,10.0937979 53.4411414,10.093729 53.4411881,10.0936043 53.4411697,10.0933312 53.4412109,10.0921856 53.4418039,10.0913853 53.4425845,10.0912356 53.4431696,10.091433 53.4432348,10.0914435 53.4433016,10.0913485 53.4433231,10.0911798 53.4433169,10.0908341 53.4434853,10.0907169 53.4434677,10.0904063 53.4436366,10.0902459 53.4436824,10.0900916 53.4436768,10.0886967 53.4444169,10.0883748 53.4448124,10.0883651 53.4451381,10.0888433 53.4452766,10.0888664 53.4453233,10.0888205 53.4453638,10.0887189 53.4453701,10.0870052 53.444983,10.0864585 53.4451596,10.085573 53.4457495,10.0854844 53.4458626,10.0854821 53.4459565,10.0856596 53.4461544,10.0858393 53.4462252,10.08596 53.4462303,10.0860353 53.4462758,10.0860718 53.4463968,10.0859573 53.4465408,10.0857982 53.4466784,10.0857349 53.4469269,10.085788 53.4470594,10.0863518 53.4472121,10.0863733 53.4472693,10.0863202 53.4473311,10.0862167 53.4473466,10.0860136 53.4472908,10.0857035 53.4472798,10.0852323 53.4473845,10.0847092 53.4475774,10.0835504 53.4484276,10.0835075 53.4490665,10.0837402 53.4491984,10.0837619 53.4492663,10.0837268 53.4493075,10.0836112 53.4493272,10.0834524 53.4492805,10.0832319 53.449166,10.0829582 53.449179,10.0824597 53.4495769,10.0824098 53.4497451,10.081309 53.4502576,10.0817084 53.4504886,10.081662 53.4505134,10.0812443 53.4502961,10.0803639 53.4508189,10.0808713 53.4510695,10.0808844 53.4511211,10.0808278 53.451166,10.0807248 53.4511544,10.0801704 53.4508729,10.0791603 53.4506871,10.078495 53.4506102,10.0778862 53.450581,10.077465 53.4506664,10.0769936 53.4509133,10.0755596 53.4515762,10.0752004 53.4518046,10.0753895 53.4521316,10.0754143 53.4523342,10.0753622 53.4524553,10.0754222 53.4525422,10.0754836 53.4526291,10.0756566 53.4527761,10.0756255 53.4528331,10.0753353 53.452773,10.0744616 53.4527738,10.0737142 53.4528438,10.0700156 53.4537806,10.0695514 53.453912,10.0691995 53.4541675,10.0690879 53.454188,10.0690622 53.4541215,10.0693712 53.4538149,10.0689678 53.4535644,10.0679353 53.4541261,10.0674091 53.4544124,10.0676333 53.4545639,10.0685987 53.4542493,10.0686931 53.4542595,10.0687017 53.454326,10.0659122 53.4553072,10.0642986 53.4560023,10.0642128 53.4559461,10.064084 53.4558899,10.0638866 53.4558796,10.0636978 53.4560278,10.0638351 53.4561096,10.0639896 53.456125,10.0639467 53.4562118,10.0625468 53.4569642,10.0610157 53.4577287,10.0601709 53.4581762,10.0597168 53.4578573,10.0594732 53.4579531,10.0599147 53.4582812,10.0598237 53.4583262,10.0595553 53.4582171,10.0594632 53.4582683,10.0596273 53.458401,10.0594544 53.4584698,10.0578791 53.4592316,10.0572898 53.4595477,10.057028 53.4597809,10.0559728 53.4602588,10.0552713 53.4606768,10.0545759 53.4610411,10.0532037 53.4618052,10.0522363 53.4623012,10.0520405 53.4623038,10.0517273 53.4624098,10.051359 53.4625451,10.0510581 53.4627022,10.0506836 53.4629215,10.0506345 53.4630751,10.0503828 53.4631884,10.0504257 53.4633382,10.050322 53.463421,10.049996 53.4636672,10.0496706 53.4636014,10.0495846 53.4637074,10.0496951 53.4638499,10.0493389 53.4641444,10.0481974 53.4651101,10.0469736 53.4661569,10.0463429 53.4666752,10.0459628 53.4669338,10.0456532 53.4671445,10.0453374 53.4673598,10.044937 53.4676328,10.0441462 53.4680692,10.0439301 53.4679836,10.043795 53.4680019,10.0437213 53.4680421,10.0438994 53.4681883,10.0438134 53.4683491,10.0433652 53.4685465,10.042766 53.4688063,10.0426899 53.4688571,10.0424688 53.4688352,10.0423154 53.4688644,10.0421373 53.4689412,10.0421741 53.4690289,10.0421066 53.4691714,10.0416313 53.4694071,10.0407933 53.4697171,10.0393857 53.4702378,10.038773 53.4696822,10.0379081 53.4699757,10.0367681 53.4703839,10.0375372 53.4710828,10.0360796 53.4716289,10.0356769 53.4717822,10.0345839 53.4709437,10.0342079 53.4711311,10.0349787 53.4717063,10.0349946 53.4719529,10.0348336 53.4720141,10.0333167 53.4724739,10.033065 53.4721706,10.0328501 53.471765,10.0325554 53.4718673,10.0327396 53.4721852,10.0330097 53.4725251,10.0326905 53.4726018,10.0314625 53.4729125,10.0302103 53.4731313,10.028848 53.4733481,10.0273855 53.473506,10.0270749 53.4729525,10.0269705 53.4729014,10.0267408 53.4729225,10.0267376 53.4731204,10.0268477 53.4735916,10.0267335 53.4736008,10.026631 53.4733833,10.0256223 53.47314,10.0250918 53.4730548,10.024842 53.4730454,10.024625 53.4730372,10.024533 53.4729981,10.0244901 53.4729342,10.024134 53.4730219,10.0239012 53.4730449,10.0236539 53.4731182,10.0233021 53.4731368,10.0229669 53.4730839,10.0228733 53.4730522,10.0224367 53.4729705,10.0221957 53.4729229,10.021924 53.4729484,10.0217382 53.4729881,10.0216238 53.4729711,10.0215427 53.4730222,10.0213812 53.4730001,10.0212133 53.4730226,10.0209136 53.47313,10.0205894 53.4731442,10.0204941 53.4731356,10.0203559 53.473164,10.0202939 53.4731158,10.0199031 53.4730732,10.0197744 53.4731016,10.0197029 53.4730846,10.0195837 53.4730363,10.0193406 53.4730193,10.0191452 53.4729711,10.0189689 53.4729995,10.0187115 53.4729711,10.0186257 53.4729995,10.0188831 53.4730449,10.0193454 53.4730817,10.02017 53.4732208,10.0208325 53.4732009,10.0209899 53.4732131,10.0210137 53.473286,10.0209612 53.4733711,10.0208478 53.4733826,10.0206419 53.4733654,10.0197499 53.4732255,10.0185802 53.473048,10.0183973 53.4730015,10.0183613 53.4728961,10.0184207 53.4728122,10.0186686 53.47273,10.0187769 53.4726062,10.0189837 53.4724963,10.0195258 53.4723827,10.0195144 53.4723242,10.0194052 53.4723001,10.0186495 53.4723065,10.0182922 53.4722307,10.0176914 53.4721223,10.0171399 53.4719539,10.016395 53.4716972,10.0161337 53.4716142,10.0160169 53.4716436,10.0159029 53.4717222,10.0158761 53.4718265,10.0161014 53.4720235,10.0163647 53.4721635,10.0168684 53.4723761,10.0171333 53.472451,10.0174235 53.4725072,10.0177664 53.4725392,10.0177797 53.4725676,10.0176841 53.4726498,10.0175751 53.4727616,10.0175534 53.4728403,10.0176074 53.4728868,10.0176374 53.4729225,10.0167492 53.4728011,10.0156328 53.4726332,10.0147866 53.4725188,10.0147885 53.4724646,10.0148706 53.4724438,10.0150386 53.4724831,10.0152067 53.4725224,10.0152727 53.4725117,10.0154168 53.4725474,10.0155068 53.4725581,10.0154948 53.472526,10.0154228 53.4725153,10.0153447 53.4724867,10.0152247 53.4724688,10.0151707 53.4724438,10.0151107 53.472426,10.0150206 53.4724295,10.0149786 53.4723938,10.0149786 53.4723509,10.0150206 53.4722831,10.0149699 53.472233,10.0150039 53.4721547,10.0149614 53.4720587,10.0149669 53.4719833,10.0149996 53.4719451,10.0149657 53.4719021,10.0147832 53.4718895,10.0146569 53.4719082,10.014571 53.4720486,10.0144479 53.4721951,10.0143766 53.4722773,10.0143545 53.4723214,10.0143822 53.4723641,10.0144722 53.4723517,10.0145161 53.4723831,10.0144854 53.4724616,10.0142544 53.4724409,10.0113542 53.4720114,10.0084356 53.4715886,10.0066154 53.4713307,10.0061505 53.4713067,10.0058615 53.4712252,10.0056928 53.4711189,10.0056187 53.4709827,10.0056587 53.4708661,10.0057145 53.4707997,10.0057905 53.4708148,10.0058419 53.4707117,10.0057771 53.4705688,10.0056872 53.4704493,10.0056659 53.4703491,10.0057094 53.4702413,10.0058271 53.4701779,10.0059984 53.470181,10.0061776 53.4702282,10.0064041 53.4703198,10.0065556 53.4703383,10.0066677 53.4703651,10.0067438 53.4704254,10.006687 53.4705088,10.0066658 53.4705956,10.0067502 53.47054,10.0068315 53.4705265,10.0068503 53.4704737,10.0068697 53.4704256,10.0069618 53.4704254,10.007119 53.4704405,10.0074486 53.4705552,10.0075398 53.4706156,10.007692 53.4706699,10.0078542 53.4707333,10.0079404 53.4707635,10.0080215 53.4707695,10.0081027 53.4707906,10.0081635 53.4708389,10.0082045 53.4708758,10.0082801 53.4708902,10.0083765 53.4708782,10.0084931 53.4708993,10.0086503 53.4709053,10.0088734 53.4709053,10.0091269 53.4708993,10.0092657 53.4708723,10.0094007 53.470848,10.0094864 53.4708146,10.009568 53.4707695,10.0095984 53.4707303,10.0096137 53.4706971,10.0096492 53.4706759,10.0096948 53.470691,10.009796 53.4707119,10.0098961 53.4706886,10.0099802 53.4706511,10.0099802 53.4705385,10.0099749 53.4704697,10.0100312 53.4704121,10.0100958 53.470404,10.0101956 53.4704666,10.010411 53.4704822,10.0108051 53.4705354,10.0117074 53.4705075,10.0117673 53.470514,10.0118054 53.4705594,10.0119415 53.4705886,10.0119742 53.4706372,10.0119742 53.4706858,10.0120341 53.4707182,10.0122464 53.4706955,10.0123226 53.4706631,10.0123988 53.4706663,10.0124805 53.4706793,10.012633 53.4706663,10.0128399 53.4706274,10.0128889 53.470595,10.0129651 53.4705886,10.0130032 53.4706015,10.0131012 53.470595,10.013221 53.4706339,10.0133625 53.4706566,10.0134605 53.4706696,10.013613 53.4706856,10.0135749 53.4706696,10.0132863 53.4706242,10.0132046 53.4705886,10.0131556 53.4705756,10.0130685 53.4705691,10.0130195 53.4705756,10.012927 53.4705464,10.0128616 53.4705497,10.0128181 53.4705918,10.0124969 53.4706437,10.0124043 53.470621,10.0123063 53.4706274,10.0122355 53.4706631,10.0121375 53.4706631,10.0120667 53.4706663,10.0120667 53.4706404,10.0120667 53.4706048,10.0119905 53.4705464,10.0118489 53.4704686,10.0117088 53.4704603,10.0108734 53.4704791,10.0108051 53.4704634,10.0105791 53.4704259,10.0104741 53.4704134,10.0101536 53.4703227,10.010038 53.4703258,10.0100184 53.4702494,10.0099568 53.470254,10.0099414 53.4703319,10.0097797 53.4703136,10.0098105 53.4702448,10.0097104 53.4702082,10.0096257 53.4701257,10.0094563 53.4701348,10.009387 53.4700982,10.0094024 53.4700523,10.0092715 53.4700477,10.0091175 53.4700111,10.0089481 53.4700386,10.0084973 53.4699121,10.008463 53.4698094,10.0083219 53.4697505,10.0081082 53.4696389,10.0080284 53.4696009,10.0079652 53.4696338,10.0079572 53.4697098,10.0079133 53.4697668,10.0078615 53.4697715,10.0078455 53.4697074,10.0078176 53.4696504,10.0077897 53.4696029,10.0077133 53.469627,10.0077179 53.4696576,10.0076899 53.4697146,10.0075862 53.4697336,10.0075104 53.4697074,10.0074346 53.4696647,10.0074585 53.4696148,10.0073708 53.4695602,10.0072112 53.4695317,10.0071873 53.4695839,10.0072072 53.4696481,10.0071593 53.4696932,10.0072032 53.4697692,10.0071873 53.469838,10.0070915 53.4698499,10.0070038 53.4698214,10.0069878 53.4697644,10.0070516 53.4696979,10.0070835 53.4696409,10.0070756 53.4695934,10.0070038 53.4695649,10.0069918 53.4695222,10.006924 53.4695151,10.0068761 53.469527,10.0068362 53.4694676,10.0068003 53.4694462,10.0067684 53.4694605,10.0067484 53.4695032,10.0067125 53.4695056,10.0066886 53.4694676,10.0066435 53.4694874,10.0065955 53.4695058,10.0065307 53.4694941,10.0065011 53.469451,10.0064572 53.4694225,10.0064793 53.4693969,10.006446 53.4693883,10.0063981 53.4694118,10.0063896 53.4694487,10.0063699 53.4694722,10.0063332 53.4694655,10.0062824 53.4694,10.0062542 53.46939,10.0062091 53.4693916,10.006195 53.4694269,10.0061865 53.4694454,10.0061386 53.4694403,10.0060821 53.4693883,10.0060413 53.4693335,10.0060647 53.4692897,10.0060443 53.469274,10.0059845 53.4692669,10.0058971 53.4692692,10.0058967 53.4692919,10.0058927 53.4693204,10.0058588 53.4693299,10.005771 53.4693358,10.0057411 53.4693489,10.0057471 53.469375,10.005781 53.4694094,10.005789 53.469445,10.0057571 53.4694759,10.0056793 53.4694985,10.0055975 53.4694985,10.0055357 53.4694973,10.0055137 53.4695198,10.0055097 53.4695531,10.0055157 53.4695733,10.0055077 53.4695911,10.0054678 53.4696018,10.0054359 53.4696303,10.0053821 53.4696469,10.0053054 53.4696516,10.0052803 53.4696789,10.0052983 53.4697205,10.0052684 53.4697929,10.0052105 53.4698309,10.0051208 53.4698463,10.005019 53.4698606,10.0049536 53.4699102,10.0049512 53.4699912,10.0049073 53.4700719,10.004898 53.4701073,10.0048954 53.4702168,10.0048395 53.4702975,10.0047379 53.4703588,10.004644 53.4704151,10.0046252 53.4704709,10.0045142 53.4705677,10.0044382 53.4707091,10.0044869 53.4707777,10.0047102 53.471037,10.0046971 53.4711448,10.0046068 53.4711957,10.0044413 53.4712026,10.0038929 53.4711833,10.0038864 53.4711485,10.0030603 53.4711446,10.0022407 53.4711678,10.0015468 53.4712198,10.001458 53.4712035,10.0006323 53.4706958,10.00064 53.4706278,10.0007197 53.4705822,10.0012181 53.4705162,10.0016242 53.4704416,10.0021749 53.4703877,10.0024189 53.4703639,10.003027 53.4703044,10.0030052 53.4700906,10.0030994 53.4700667,10.0032661 53.4700243,10.0040459 53.4698573,10.0041363 53.4696657,10.0041814 53.46957,10.0042095 53.4695168,10.0045423 53.4695353,10.0051665 53.4694677,10.0053242 53.4693975,10.0055197 53.4692586,10.0055676 53.4692147,10.0055755 53.4691743,10.0055048 53.469105,10.005414 53.4690532,10.0053073 53.4690576,10.0026556 53.4694122,10.0009365 53.4697843,10.0006381 53.4698489,10.0002876 53.4699216,9.9999298 53.4699959,9.9997436 53.4698704,9.9995511 53.4698952,9.9993766 53.4699176,9.9992507 53.4700872,9.9979764 53.4702703,9.997723 53.4703493,9.9976482 53.4704789,9.996877 53.4706215,9.9967805 53.4706393,9.9966138 53.4705346,9.9965117 53.4705908,9.9954764 53.4711603,9.9954719 53.4720289,9.9953551 53.4720481,9.9942823 53.4722248,9.9939588 53.4716137,9.9932995 53.4717214,9.9929739 53.4716879,9.992622 53.4715375,9.9925074 53.4715566,9.9919783 53.4716448,9.9915924 53.4718226,9.9913842 53.4719185,9.9913972 53.4725139,9.9920444 53.4726427,9.9916676 53.4727804,9.9910792 53.4726847,9.9907255 53.4718312,9.9906358 53.4716122,9.990457 53.4716085,9.9904553 53.4715232,9.9905089 53.4704022,9.990241 53.4703984,9.9901915 53.4715238,9.99019 53.471549,9.9901872 53.4715964,9.9901279 53.4715959,9.989938 53.4716615,9.989777 53.4728168,9.9895824 53.4730442,9.9893418 53.4731791,9.9892222 53.4732461,9.9887957 53.4733875,9.9881524 53.4733463,9.9870461 53.4735416,9.9867734 53.4737233,9.986222 53.4738089,9.9859652 53.4736848,9.9859986 53.4734942,9.9862859 53.4731424,9.9860064 53.473069,9.9857268 53.4729956,9.9856789 53.4730587,9.9853666 53.4734888,9.985254 53.473618,9.9851016 53.4737905,9.9837548 53.4740334,9.9818947 53.4744395,9.9802263 53.4748037,9.9790781 53.4750175,9.9784772 53.4741705,9.9783267 53.4740762,9.9772151 53.4742505,9.9770877 53.4740106,9.9770165 53.4739088,9.9768903 53.4739321,9.9766964 53.4735998,9.9768594 53.4735655,9.9764851 53.4729146,9.9761659 53.4729256,9.9755363 53.4718091,9.974715 53.4710187,9.9737069 53.4712006,9.9743271 53.4721866,9.9752478 53.4741843,9.9750113 53.4742457,9.9749874 53.4756042,9.9748401 53.4759515,9.9747219 53.4761676,9.9743046 53.4764835,9.9730816 53.476945,9.9728061 53.4769736,9.9726006 53.4769538,9.972489 53.4768571,9.972329 53.4767847,9.9721412 53.4767346,9.9719588 53.4766635,9.9702093 53.4733027,9.970601 53.473219,9.9698122 53.4718025,9.9696802 53.4717694,9.9687084 53.4719426,9.9688046 53.4721037,9.96805 53.4722562,9.9694514 53.4748991,9.9704006 53.4766966,9.9700796 53.477233,9.9700099 53.4775641,9.9700313 53.4777093,9.9700304 53.4778253,9.9699774 53.4779752,9.9698352 53.478149,9.9695614 53.4783001,9.9673179 53.4793172,9.9670241 53.4793318,9.9668979 53.4792619,9.96681 53.479192,9.9662536 53.4780384,9.9661216 53.4777647,9.9659201 53.4773781,9.9658794 53.477301,9.964385 53.4743156,9.9641726 53.473907,9.9637646 53.4730371,9.9634235 53.4729191,9.9618104 53.47322,9.9617488 53.4732827,9.9621844 53.4741079,9.9621907 53.4743596,9.963705 53.477472,9.9644624 53.4788735,9.964622 53.480027,9.9646234 53.4804469,9.9644768 53.4806457,9.964245 53.4807785,9.9626915 53.4814834,9.9618504 53.4818971,9.9615929 53.4819635,9.9613869 53.4819175,9.9612667 53.4818409,9.9606158 53.4809623,9.959899 53.480065,9.959011 53.478791,9.9583927 53.4777903,9.9558436 53.4742809,9.9555088 53.4743559,9.9538625 53.4747247,9.9553419 53.4769854,9.9555254 53.4770623,9.9566055 53.4785897,9.9565755 53.4787966,9.9571657 53.4796003,9.9572004 53.4796468,9.9573883 53.4799028,9.9574159 53.4799405,9.9585279 53.4814539,9.958946 53.4821898,9.9590852 53.4829131,9.9589499 53.4834195,9.9588134 53.4836578,9.9580149 53.4844407,9.9574844 53.4847309,9.9571925 53.4846389,9.9567634 53.4843771,9.9565832 53.4841269,9.9562742 53.4839787,9.9558708 53.4840707,9.9556819 53.4840349,9.9555017 53.4839736,9.9555017 53.4839021,9.9557077 53.4838204,9.9558622 53.4837591,9.9557763 53.4836927,9.9553901 53.4838408,9.9552614 53.4837438,9.955433 53.4836621,9.9556219 53.483611,9.955536 53.4835599,9.9553472 53.4835395,9.9550039 53.4835548,9.9545747 53.4838102,9.9552099 53.4840809,9.9564716 53.4844537,9.9569436 53.4848228,9.9571927 53.485038,9.9569828 53.4852769,9.9564708 53.4854743,9.9563739 53.4854352,9.9561951 53.4855041,9.9557842 53.4854136,9.9555995 53.4854645,9.9554743 53.4855336,9.955121 53.4857346,9.9549887 53.4857093,9.9547874 53.4855306,9.9544683 53.4850206,9.9544855 53.4848521,9.9543739 53.484658,9.9542623 53.4844588,9.9539723 53.4843272,9.9539768 53.4846112,9.9544998 53.4854733,9.9544639 53.4858434,9.9543641 53.4859682,9.9502518 53.4878522,9.9499806 53.4881492,9.9510403 53.488426,9.9514883 53.4883195,9.9542815 53.487015,9.9544779 53.4869496,9.9547589 53.4869255,9.9552969 53.4867071,9.9554115 53.4866606,9.9554689 53.4866931,9.9549609 53.4873776,9.952162 53.4909229,9.950776 53.4926773,9.9500215 53.4935864,9.9491427 53.4946873,9.9490102 53.4947189,9.9490037 53.4949174,9.9486725 53.4953163,9.9483388 53.4953544,9.9482642 53.495547,9.9479145 53.4957174,9.9474872 53.4958551,9.9470157 53.4959593,9.9465522 53.4960418,9.9460675 53.4960941,9.9455362 53.4961363,9.9435203 53.4961657,9.9427595 53.4962025,9.9422204 53.4962845,9.9416215 53.4964072,9.9402501 53.4968784,9.9380692 53.4976369,9.9379219 53.4976726,9.9368197 53.5111692,9.9367861 53.5115013,9.9367176 53.5122701,9.9366186 53.5133384,9.9365403 53.5140932,9.9353645 53.5148323,9.9344933 53.514288,9.9342764 53.5142652,9.9340489 53.5143265,9.9334165 53.5138233,9.9317808 53.5124011,9.9307628 53.5128045,9.9298131 53.5131932,9.9359664 53.5186908,9.9361755 53.5188811,9.9362893 53.5193289,9.9363781 53.5196782,9.9363797 53.5218506,9.936425 53.5219794,9.9364249 53.5221212,9.9364248 53.5221995,9.9363555 53.5222901,9.9362426 53.5223427,9.9349745 53.5225457,9.9349944 53.5227248,9.9349986 53.5227621,9.9350025 53.5228999,9.9347296 53.5229064,9.9347363 53.5230525,9.9348532 53.5230515,9.9348769 53.5233913,9.9345523 53.5233964,9.9345557 53.523474,9.9345575 53.5235514,9.9348935 53.5235471,9.9348997 53.5236781,9.934937 53.5238309,9.9350459 53.5238507,9.9351266 53.5238796,9.9365791 53.5241117,9.9366674 53.5241668,9.9367134 53.5256168,9.9367163 53.5256557,9.9367596 53.5261968,9.9367073 53.5263013,9.9364909 53.5263566,9.9364624 53.5265189,9.9364598 53.5268193,9.9364534 53.5275497,9.9365367 53.5298221,9.9365531 53.5303505,9.9364973 53.5311872,9.9359824 53.5326411,9.9349867 53.5342684,9.9339444 53.5354335,9.9325469 53.5364988,9.9321649 53.5367057,9.9314468 53.5371527,9.929816 53.5379638,9.9277913 53.5387354,9.9276874 53.5387136,9.9275558 53.538701,9.9275387 53.5386994,9.9275024 53.538718,9.9274759 53.5387534,9.927456 53.5387751,9.9274329 53.5387996,9.927418 53.5388272,9.9273849 53.5388508,9.9273452 53.5388685,9.9272526 53.538895,9.9270325 53.5389501,9.9268638 53.5389805,9.9267066 53.5389992,9.9265727 53.5390218,9.9259655 53.5391418,9.9253613 53.5392745,9.9247248 53.5393902,9.9244184 53.5394106,9.9241434 53.5393619,9.9239457 53.5392827,9.9238141 53.5391765,9.9237374 53.5390893,9.9237427 53.5389944,9.9232965 53.5389592,9.9235299 53.5387519,9.9240447 53.5385878,9.9248159 53.53825,9.9250332 53.5381941,9.9253519 53.5381335,9.9253252 53.5379852,9.9251684 53.537953,9.9250105 53.5380719,9.9247572 53.5381767,9.9239878 53.5385191,9.9234435 53.5386961,9.923142 53.5389402,9.9229489 53.539004,9.9226847 53.5392542,9.9224968 53.539314,9.9217406 53.5393266,9.9211197 53.5395415,9.9054568 53.5396117,9.9053507 53.5395945,9.9053006 53.5395312,9.9051689 53.5391307,9.9052324 53.5390822,9.9054544 53.5390784,9.9098305 53.5390848,9.9097986 53.5357369,9.9097066 53.5357343,9.9096945 53.5354755,9.9092102 53.5354829,9.9092074 53.5349361,9.909005 53.5347658,9.9164703 53.5316857,9.9271268 53.5272637,9.927307 53.5274173,9.9274255 53.5273651,9.9274743 53.5273959,9.9281078 53.5271404,9.9283515 53.5263706,9.9286584 53.5254012,9.9286619 53.52539,9.9281805 53.5249488,9.9280643 53.5248522,9.9278589 53.5246811,9.9275984 53.5244642,9.9272309 53.5242043,9.9270734 53.52424,9.9270501 53.5242162,9.926467 53.5244597,9.926821 53.524734,9.917254 53.5287435,9.9151668 53.5296299,9.9148291 53.5293543,9.9058025 53.533189,9.900743 53.5352133,9.8862725 53.5354745,9.8863527 53.5368255,9.8956304 53.5366442,9.8956286 53.5365707,9.8957549 53.5365675,9.8958525 53.5365644,9.8964689 53.5365444,9.8964698 53.5366214,9.8984254 53.5365836,9.9006553 53.5365265,9.9011827 53.5366105,9.9011827 53.5365461,9.9013684 53.5365455,9.9013742 53.5366661,9.9013879 53.5369501,9.9008978 53.5369537,9.9009285 53.5377758,9.9009306 53.5378407,9.9008972 53.5378849,9.9007954 53.537929,9.8964631 53.5396323,9.8907463 53.5397166,9.8877077 53.5397701,9.8851333 53.539871,9.8832172 53.5399507,9.8831272 53.5400153,9.8811539 53.5401022,9.8809438 53.5401669,9.8808238 53.5401758,9.8806721 53.5401433,9.8789631 53.5402382,9.8787348 53.5402145,9.8785784 53.5401523,9.8784912 53.5400801,9.878455 53.5400149,9.8784443 53.5399494,9.8784453 53.5398928,9.8784772 53.5398253,9.8785655 53.5397612,9.8787601 53.539706,9.8788867 53.5396863,9.8792765 53.539664,9.8799543 53.5396481,9.8801609 53.5397515,9.880218 53.5397777,9.8802495 53.5397949,9.8826294 53.5395349,9.8830671 53.539458,9.8833374 53.5393648,9.8835248 53.5392239,9.8836132 53.539096,9.8837896 53.5387919,9.8838297 53.5386374,9.883821 53.5381251,9.883615 53.5374209,9.8827463 53.5372821,9.8824131 53.5372349,9.8817442 53.5372657,9.8815832 53.5371874,9.8815832 53.5370406,9.8816593 53.5368547,9.8853968 53.5346398,9.8883858 53.5332423,9.8899306 53.5325222,9.8905457 53.5322205,9.8914232 53.531868,9.8929365 53.5312341,9.8934477 53.5290271,9.8913225 53.5292822,9.8889142 53.529526,9.8885629 53.5305358,9.8848102 53.5327292,9.8847505 53.5327299,9.8845903 53.5326321,9.884357 53.5326112,9.884273 53.5326292,9.8840969 53.5325969,9.8839555 53.5324733,9.8836216 53.5322916,9.8821772 53.5329001,9.882147 53.5329592,9.8821741 53.5330053,9.8822842 53.5330177,9.8824187 53.53323,9.8824574 53.5333469,9.8824139 53.5333792,9.8828864 53.5336912,9.8825862 53.5338506,9.8823477 53.5339678,9.8820712 53.534103,9.8818513 53.5341893,9.8816218 53.5340444,9.8813365 53.5342534,9.8813848 53.5342771,9.88128 53.5343395,9.8813693 53.534399,9.8811236 53.5345303,9.8810489 53.5344828,9.8807878 53.5346144,9.8807327 53.5345739,9.8806591 53.5346128,9.8807386 53.5346757,9.8806054 53.5347452,9.8806828 53.5347959,9.8804886 53.5349032,9.880457 53.5348871,9.8801932 53.5350332,9.8800658 53.5351442,9.8799335 53.5352287,9.879902 53.5354109,9.8787373 53.5361746,9.8774341 53.536881,9.877544 53.5369555,9.8774654 53.5369829,9.8774335 53.5369712,9.8768858 53.5366545,9.8766003 53.5368201,9.876142 53.5370894,9.8761305 53.5371276,9.8758419 53.5373202,9.8757046 53.5379109,9.8756276 53.5379717,9.8753261 53.53797,9.8753124 53.5380401,9.8738785 53.5389738,9.8737653 53.5390406,9.8736942 53.5390591,9.8713591 53.5401337,9.8697174 53.5408615,9.8694875 53.5409174,9.8675762 53.5411407,9.8674402 53.541134,9.8673302 53.5410861,9.8671964 53.5407918,9.8672109 53.540705,9.8695454 53.5383469,9.8697538 53.5382355,9.8698957 53.5380936,9.872594 53.535433,9.8725117 53.5353073,9.8723802 53.535199,9.8720374 53.5351654,9.8716882 53.5352033,9.8713012 53.5353914,9.8708801 53.5357255,9.8706845 53.5358994,9.870763 53.5359391,9.8705174 53.536189,9.8703922 53.5361487,9.8697262 53.5366185,9.8691787 53.536778,9.8657753 53.5400944,9.8648529 53.5409893,9.8645388 53.5412738,9.86425 53.5413155,9.8641918 53.5412616,9.8631905 53.541398,9.8612381 53.5416575,9.8612531 53.5417047,9.8610133 53.5417354,9.8609153 53.5417823,9.8608418 53.541873,9.8607252 53.541938,9.8604848 53.5420056,9.8601906 53.5420518,9.8602027 53.5420957,9.8600048 53.5421248,9.8599878 53.5420791,9.8592384 53.5421726,9.8592575 53.5422203,9.859049 53.5422487,9.8590271 53.5421838,9.8579136 53.5423327,9.8577534 53.5423541,9.8574413 53.5424644,9.8565589 53.5425828,9.856505 53.5425993,9.8565275 53.5427195,9.8564171 53.5427345,9.8563391 53.5426163,9.8561768 53.5425798,9.8561011 53.5424971,9.8560752 53.5424535,9.8560874 53.5423929,9.8561492 53.5423305,9.8562761 53.5422898,9.8567888 53.5422156,9.8569839 53.5421715,9.8570495 53.5420996,9.8570413 53.541959,9.8567836 53.5413505,9.8566938 53.5411966,9.856566 53.5411104,9.8560444 53.5408599,9.8560591 53.5408283,9.8560709 53.5408005,9.855861 53.5407534,9.8558638 53.5407403,9.8551495 53.5404874,9.8549976 53.5404513,9.8547977 53.5404549,9.8546166 53.540478,9.854291 53.5405234,9.8541711 53.5405537,9.8541832 53.5405767,9.8535838 53.5406585,9.8534027 53.5407126,9.8533732 53.5407033,9.8537088 53.5405119,9.8536695 53.5404819,9.853603 53.5404469,9.8546947 53.5395182,9.8546468 53.5395018,9.8546106 53.5394724,9.8558917 53.5383056,9.8560037 53.5381473,9.8559974 53.5379718,9.8560369 53.5378077,9.8560082 53.5372286,9.8560767 53.5369773,9.8568655 53.5362571,9.8566935 53.5362573,9.8565695 53.5362271,9.8564734 53.5362368,9.856418 53.5362408,9.8563024 53.5362549,9.8562407 53.5362966,9.8561235 53.5363093,9.8559384 53.5363143,9.8557313 53.5363385,9.8556768 53.5363737,9.8556316 53.5363978,9.8555586 53.5364187,9.8554177 53.5364233,9.8553239 53.536488,9.8545839 53.5372315,9.8543983 53.5374253,9.8541604 53.5374918,9.853901 53.537516,9.8537901 53.5373773,9.8531564 53.5374244,9.8530427 53.5374328,9.8530628 53.5376464,9.8524569 53.5377354,9.8504243 53.5396245,9.8507015 53.540427,9.8522372 53.5411479,9.8524417 53.5411167,9.8524915 53.5410694,9.8526055 53.5411133,9.8525561 53.541127,9.8525661 53.5411767,9.8529383 53.5412294,9.8528879 53.5410838,9.8530004 53.5410672,9.853056 53.5412216,9.8538561 53.5411147,9.8537951 53.5409525,9.8539084 53.5409374,9.8540325 53.5412673,9.85434 53.5412281,9.8542421 53.54094,9.8542834 53.5409344,9.8542661 53.5408925,9.8543726 53.5408775,9.8544848 53.5411817,9.8546479 53.5411536,9.8546007 53.5410083,9.8546715 53.5409832,9.8545507 53.5409083,9.8546284 53.5408868,9.8547753 53.5409789,9.854767 53.5409444,9.8548356 53.5409348,9.8548269 53.5409105,9.8547553 53.5409175,9.8547509 53.5409062,9.8548225 53.5408949,9.8548181 53.5408732,9.8547436 53.5408819,9.8547279 53.540836,9.8548668 53.5408185,9.8552283 53.5417584,9.8550682 53.5417779,9.8550377 53.5417006,9.8549841 53.5417201,9.8549496 53.5417642,9.8551201 53.5422562,9.8552516 53.5426027,9.8552518 53.542694,9.8551931 53.542774,9.855081 53.5428239,9.8543261 53.5430305,9.854055 53.5431465,9.8529221 53.5439726,9.8528006 53.5440083,9.850763 53.5443599,9.8483721 53.5436312,9.8482427 53.5436725,9.8479443 53.5435868,9.8477985 53.5435449,9.8459988 53.5438031,9.8459265 53.5439838,9.8420652 53.544523,9.8413187 53.5444067,9.8399277 53.5446255,9.839556 53.5448488,9.8394314 53.5449237,9.8363513 53.5453991,9.8348464 53.5456611,9.8339905 53.5459799,9.8335296 53.5459924,9.8315332 53.5462703,9.8289467 53.5464504,9.8289785 53.5466764,9.8279698 53.5467486,9.8279895 53.5468833,9.8285527 53.54689,9.8285508 53.5470447,9.8279218 53.5470509,9.8279278 53.5469793,9.8278105 53.5468467,9.8264189 53.5470935,9.8264061 53.5472416,9.8263484 53.5473335,9.8261998 53.54738,9.8260327 53.5473655,9.8259047 53.5473228,9.825749 53.547315,9.8239516 53.5475979,9.8235599 53.5475594,9.8233718 53.5475035,9.8228808 53.5474153,9.8219655 53.5472544,9.82108 53.5470724,9.8207331 53.5469408,9.820433 53.5467771,9.8201463 53.5465382,9.8199088 53.5462474,9.8196855 53.546091,9.8190873 53.5457056,9.8187432 53.5455481,9.8185443 53.5454429,9.8183886 53.5452926,9.8165396 53.5441569,9.8157988 53.5437337,9.8140194 53.5428762,9.8121894 53.5421357,9.8110031 53.5416434,9.8105877 53.5413866,9.8103598 53.5410602,9.8102055 53.5406702,9.8102925 53.5402555,9.8103709 53.5400277,9.8105979 53.5397733,9.8109686 53.5395405,9.8114807 53.5393087,9.81203 53.5391026,9.8129866 53.5387617,9.8139703 53.5383366,9.8138925 53.5382731,9.8139877 53.5382354,9.814054 53.5382886,9.8146454 53.5380554,9.815156 53.5377879,9.8179128 53.5366617,9.8184569 53.5364919,9.8199328 53.5359129,9.8240424 53.5342624,9.8241231 53.534062,9.8244386 53.5339414,9.8245404 53.5338199,9.824197 53.5336669,9.8244921 53.5335059,9.8249064 53.5333751,9.8248688 53.533189,9.8232013 53.5323899,9.8170426 53.5295555,9.8163012 53.5293681,9.8069631 53.5308835,9.8056425 53.531111,9.7983079 53.5335124,9.7961197 53.5341729,9.795227 53.5343872,9.7946605 53.5346983,9.7936243 53.5353293,9.7925608 53.5356976,9.7924058 53.5357253,9.7922533 53.5357176,9.7921829 53.5355928,9.7921427 53.5354668,9.7921272 53.5353357,9.7921778 53.5351624,9.7922359 53.5350349,9.7921744 53.5350073,9.7921339 53.5349891,9.7919184 53.5351762,9.7918496 53.5351977,9.7918461 53.5352075,9.7916546 53.5352622,9.7916154 53.5354455,9.7916606 53.5356508,9.7916434 53.5356785,9.7915453 53.5357013,9.7916331 53.5358794,9.7914979 53.5359011,9.7909629 53.5354888,9.7906746 53.5355727,9.7903723 53.5356583,9.7904553 53.5357985,9.790689 53.5363337,9.7907073 53.5363973,9.790703 53.5364444,9.7906748 53.536486,9.7906192 53.5365299,9.7905071 53.5365666,9.7903849 53.5365676,9.7901888 53.5365296,9.7900067 53.536474,9.7888049 53.5367982,9.7844789 53.5383003,9.7830933 53.5388009,9.7805162 53.5393965,9.7786102 53.5401053,9.777081 53.5409317,9.7749682 53.5419918,9.7746487 53.5419807,9.7738908 53.542239,9.7732932 53.5425771,9.7729222 53.5425965,9.7710579 53.5437328,9.7707438 53.543753,9.7672033 53.545945,9.7663278 53.5464958,9.7657134 53.5467046,9.7649559 53.5467647,9.7610622 53.5467889,9.7558931 53.5467925,9.7468122 53.5470687,9.7437523 53.5476646,9.7435887 53.5476758,9.7434245 53.5476682,9.7432397 53.5476444,9.7432211 53.5476131,9.7430086 53.5475646,9.7429965 53.5475478,9.7429663 53.5475539,9.7429393 53.5475601,9.7429514 53.5475774,9.7428767 53.547674,9.7428483 53.5477014,9.7428676 53.5477375,9.742063 53.5479149,9.7416538 53.5479724,9.7411616 53.5480759,9.7396262 53.5483463,9.7396622 53.5482911,9.7400464 53.5481972,9.7400427 53.5481459,9.7395598 53.5475886,9.7393387 53.5471803,9.7391586 53.5468887,9.7371099 53.5463764,9.7350782 53.5459909,9.7341222 53.5456004,9.7335417 53.5455294,9.7325722 53.5456153,9.7316466 53.5453873,9.7312198 53.5452047,9.7306991 53.5453316,9.7302723 53.5455852,9.7294528 53.5457069,9.7290765 53.5460145,9.7288041 53.5480755,9.728548 53.5486689,9.7278602 53.549062,9.7274452 53.5496677,9.72704431328 53.55,9.77365999931035 53.55,9.7738153 53.5499298,9.7740801 53.5498839,9.7743395 53.5497708,9.7745769 53.5496894,9.7749521 53.5497164,9.7751408 53.5496437,9.7754669 53.5494754,9.7759251 53.5494248,9.7762995 53.5494754,9.776926 53.5495621,9.7773294 53.5497304,9.7778702 53.5497049,9.7783852 53.5498834,9.778677 53.5497814,9.7789345 53.5499497,9.77912617162601 53.55,9.8419381995025 53.55,9.8455241 53.549422,9.8463024 53.5493064,9.8511442 53.5485905,9.8529313 53.5483313,9.8537694 53.5482091,9.8555575 53.5479542,9.8574573 53.5476684,9.8583438 53.5475311,9.858397 53.5475757,9.858505 53.5475692,9.8585019 53.5475199,9.8585608 53.5475064,9.8595052 53.5476204,9.8595209 53.5476585,9.8602344 53.5476073,9.8616402 53.5473167,9.8622988 53.5471841,9.8625521 53.5471052,9.8628431 53.5469317,9.862951 53.546916,9.8632135 53.5469748,9.8634726 53.5469715,9.8638791 53.547032,9.8641435 53.5471424,9.8643869 53.5472094,9.8645369 53.5471296,9.8646698 53.5471417,9.8648079 53.5470893,9.8652214 53.5471273,9.8664041 53.5471972,9.8665044 53.547228,9.866844 53.5471995,9.8672035 53.5471439,9.8677371 53.5469982,9.868283 53.5469133,9.8685758 53.5468587,9.8692973 53.5467545,9.8715021 53.5464333,9.8721886 53.5463282,9.8722989 53.5462645,9.872453 53.5462424,9.872535 53.546245,9.8727544 53.5461454,9.8736136 53.5459436,9.8744234 53.5458315,9.8748542 53.5457872,9.8751016 53.5457718,9.8755354 53.5457018,9.8757088 53.5456997,9.875827 53.5456821,9.8769193 53.5455638,9.8796605 53.5452419,9.8802933 53.5453537,9.8803834 53.5453569,9.8800267 53.5452943,9.8822449 53.5449543,9.8834261 53.5448464,9.8837727 53.5448257,9.8840367 53.5448355,9.8871929 53.5447422,9.8893141 53.5447012,9.891782 53.5446722,9.8925019 53.5446412,9.8929458 53.5446569,9.8930174 53.5446238,9.8931678 53.5446146,9.8932661 53.5446385,9.8936051 53.5445977,9.8943594 53.5445067,9.8948874 53.5444651,9.8954067 53.5444444,9.8959337 53.5444352,9.8964538 53.5444249,9.897247 53.544402,9.8981733 53.544342,9.8981605 53.5442532,9.8981681 53.5442412,9.8981993 53.5442292,9.8982379 53.5442366,9.8982936 53.544337,9.8983242 53.5443735,9.8985192 53.5444173,9.8986408 53.5444169,9.8988152 53.5443992,9.899253 53.5444138,9.8996045 53.5444282,9.9001734 53.5444089,9.900747 53.5443702,9.9017347 53.5443122,9.9025397 53.5442758,9.9032537 53.5442682,9.9038715 53.5442788,9.9052632 53.5442999,9.9057219 53.5442902,9.9107008 53.5441522,9.9107459 53.5441655,9.9107821 53.5442146,9.9108912 53.5442379,9.9113605 53.5442152,9.9118959 53.5441169,9.9121467 53.5440715,9.9124039 53.5439374,9.9124854 53.5439245,9.9125755 53.5439292,9.9126336 53.5439563,9.9127228 53.5440088,9.9128298 53.5441074,9.9129218 53.5441922,9.9131554 53.544413,9.9141517 53.5443042,9.9143946 53.5441594,9.9143931 53.5440181,9.9144269 53.5438918,9.9144544 53.5438819,9.9145525 53.5438798,9.9145788 53.5438962,9.9147071 53.5440774,9.9163045 53.5439776,9.9162895 53.5436582,9.9163024 53.5436544,9.917241 53.5436298,9.9180471 53.5436326,9.9180463 53.5435832,9.9187168 53.5435863,9.9187161 53.543613,9.9230596 53.543627,9.9230526 53.5436573,9.9249874 53.5438411,9.928411 53.5441404,9.9309498 53.5442356,9.9326043 53.5442789,9.9326144 53.5442537,9.93465 53.5442752,9.935931 53.5442644,9.9359049 53.5434802,9.9337488 53.5434648,9.9337512 53.5432875,9.9337584 53.5431257,9.9361844 53.543168,9.9361974 53.5431352,9.9423465 53.5432223,9.9423834 53.5435423,9.9425065 53.5436094,9.942537 53.5437722,9.9449598 53.5438294,9.9450523 53.543938,9.9451328 53.5439393,9.9451466 53.5441238,9.9466622 53.5441662,9.9466956 53.5438817,9.9483254 53.5439559,9.9483303 53.5439762,9.948771 53.5439907,9.9496279 53.5440786,9.949649 53.5440293,9.9506068 53.544152,9.950532 53.5443433,9.9536882 53.5450243,9.95439 53.5451245,9.954474 53.5450422,9.9545482 53.5450086,9.9546464 53.5450085,9.9557768 53.5451917,9.9608927 53.5458321,9.9609796 53.5458575,9.9610358 53.5458956,9.9610573 53.5459403,9.9621128 53.5459487,9.9621165 53.5457791,9.9624487 53.5457858,9.9632734 53.5457943,9.963264 53.5458964,9.96348 53.545882,9.9642809 53.5457735,9.9662921 53.5456671,9.96645 53.5456541,9.9665867 53.5456353,9.966801 53.545637,9.9668121 53.5456035,9.9668787 53.5455764,9.9681711 53.5455256,9.9700764 53.5454285,9.9701086 53.5454318,9.9701315 53.5454438,9.9701486 53.5454628,9.9704081 53.5454508,9.9722779 53.5449507,9.9729931 53.5449899,9.9734175 53.5448491,9.9742185 53.5445834,9.9745855 53.5444616,9.9747122 53.5444206,9.976479 53.5438491,9.976542 53.5438296,9.9767064 53.5437763,9.9767323 53.5437679,9.9767942 53.5437485,9.9768626 53.5437448,9.9770627 53.5437443,9.977658 53.5437637,9.9788482 53.5438,9.9792241 53.5438135,9.9803579 53.5438355,9.9806138 53.5438406,9.980724 53.5438359,9.9807699 53.5438329,9.9810448 53.5438152,9.9818594 53.5437718,9.9819522 53.5437666,9.9820018 53.5437635,9.9820193 53.5437649,9.9820307 53.5437683,9.9820834 53.543799,9.9821952 53.543859,9.982236 53.5438808,9.9822931 53.5439146,9.9823876 53.5439678,9.982512 53.5440269,9.9826749 53.5440896,9.9828615 53.5441562,9.9830806 53.5442313,9.9831784 53.544263,9.9831999 53.5442718,9.9832187 53.5442941,9.983221 53.5443248,9.983219 53.5443447,9.9831673 53.5448331,9.9832226 53.5452621,9.9832247 53.545262,9.9832271 53.545262,9.9832297 53.5452621,9.9832345 53.5452628,9.9832405 53.5452646,9.9832448 53.5452668,9.9832482 53.5452696,9.9832506 53.545273,9.9832516 53.5452768,9.9832526 53.5452901,9.9833355 53.5458053,9.9834524 53.5464735,9.9843156 53.5476198,9.984363 53.5476319,9.9845532 53.5477181,9.9846799 53.5478801,9.9849687 53.5482634,9.9850386 53.5483575,9.9850891 53.5483433,9.9852675 53.5484861,9.9854578 53.5486828,9.9856803 53.5489307,9.9857254 53.5489891,9.9857318 53.5489983,9.9858677 53.5491568,9.9859075 53.5492031,9.9861676 53.5493135,9.9861367 53.5493413,9.9863233 53.5494191,9.9866009 53.5495382,9.9867272 53.5495919,9.9887383 53.5504076,9.9893366 53.5506502,9.9894152 53.5506821,9.9894066 53.55073,9.9894317 53.5507403,9.9897353 53.5504728,9.9896828 53.5504566,9.9895766 53.550412,9.989271 53.5502797,9.9874938 53.5495865,9.987516 53.5495671,9.9874035 53.5495259,9.9874574 53.5494774,9.9873364 53.5494246,9.9876644 53.5491614,9.987695 53.5491744,9.9877066 53.5491666,9.9878626 53.5490686,9.9879513 53.5490142,9.9882677 53.5489364,9.9882968 53.5489561,9.9886024 53.5489208,9.98902 53.5489109,9.9894135 53.548972,9.9897942 53.5490709,9.9898818 53.5489742,9.9896325 53.5488877,9.9894036 53.5488399,9.9894011 53.5487889,9.989028 53.548754,9.9890253 53.5487454,9.9887908 53.5487462,9.9885696 53.5487476,9.9882982 53.5487753,9.9881635 53.5487925,9.9880693 53.5488146,9.9880047 53.5488341,9.9879075 53.5488665,9.9876684 53.5489875,9.9874961 53.5490737,9.9870885 53.5494027,9.9869335 53.5493468,9.9868079 53.5492923,9.9866199 53.5491862,9.9863744 53.5489479,9.9862024 53.5487716,9.9858167 53.5483993,9.9856921 53.5483051,9.985397 53.548197,9.9850364 53.5477482,9.9850859 53.5477379,9.9848611 53.547387,9.9843572 53.5467084,9.9839925 53.5462281,9.983941 53.5460431,9.9839207 53.5458857,9.9836723 53.545897,9.9836167 53.5457636,9.9835268 53.5452606,9.9835258 53.545258,9.9835255 53.5452554,9.9835261 53.5452527,9.9835273 53.5452502,9.9835304 53.5452469,9.9835349 53.5452441,9.9835386 53.5452426,9.9835427 53.5452415,9.9835476 53.5452407,9.9835525 53.5452406,9.9835573 53.545241,9.9835616 53.5452419,9.9835656 53.5452432,9.9835705 53.5452458,9.9835732 53.545248,9.9835751 53.5452504,9.9835764 53.5452531,9.9835768 53.5452559,9.9836189 53.545455,9.9839008 53.54538,9.9838617 53.5451813,9.9838965 53.545149,9.9839241 53.5451234,9.9839568 53.5449947,9.9840223 53.5449133,9.9840681 53.5448871,9.9841067 53.5449113,9.9841548 53.5448859,9.9842466 53.544835,9.984304 53.5448073,9.9843425 53.5447643,9.984576 53.5448133,9.9848541 53.5448694,9.9850904 53.5448903,9.9856886 53.5447724,9.9860426 53.544723,9.9863538 53.5447134,9.9867024 53.5447134,9.9868161 53.5447155,9.987104 53.5447149,9.9872603 53.5447103,9.9872869 53.5446172,9.9875606 53.5445928,9.9877832 53.5447329,9.9883918 53.5446897,9.988602 53.5446607,9.9891926 53.5446485,9.989192 53.544626,9.9894765 53.5446215,9.9894814 53.5445659,9.9895224 53.5445714,9.9895538 53.5445822,9.989578 53.5445984,9.9898544 53.5446154,9.989863 53.5446247,9.9899387 53.5446245,9.989992 53.5446271,9.9921292 53.5445615,9.992361 53.5445628,9.9925459 53.5445928,9.9926728 53.5446187,9.9927241 53.5446012,9.9927807 53.544606,9.9929452 53.544669,9.993022 53.5446984,9.9930886 53.5447239,9.993394 53.5448526,9.9934074 53.5448735,9.9933947 53.5448951,9.9941594 53.5451971,9.9945428 53.5453283,9.9945184 53.5453546,9.9956201 53.5457375,9.9956094 53.5457545,9.99686 53.5461871,9.9968833 53.5461616,9.9969358 53.5461756,9.9971868 53.5462425,9.9972394 53.5462562,9.9972371 53.5462958,9.9973048 53.546314,9.9973373 53.5463227,9.9973723 53.5463123,9.9974835 53.5463365,9.9974963 53.5463207,9.9977568 53.5463813,9.9977497 53.5463984,9.9980965 53.5464754,9.9984812 53.5465487,9.9987715 53.5465811,9.9992148 53.5466467,9.9994824 53.5467,9.9997963 53.5467774,10.0000915 53.5468544,10.0007035 53.547046,10.0007359 53.547038,10.0009687 53.5471151,10.000983 53.547102,10.0010003 53.5470978,10.0010278 53.5471019,10.0010434 53.547112,10.0010479 53.5471281,10.001367 53.5471271,10.0017737 53.5471277,10.0021772 53.5471308,10.0022672 53.5471359,10.0035816 53.5467534,10.0035802 53.5467437,10.0035787 53.5467326,10.0036247 53.5467217,10.0036103 53.5467035,10.004363 53.5464811,10.0045367 53.5464315,10.0045501 53.5464277,10.0057809 53.5460452,10.0060626 53.5459408,10.0063854 53.5458168,10.0064315 53.5457919,10.0066653 53.5457078,10.0066797 53.5457219,10.0067337 53.545699,10.0067897 53.545756,10.0069145 53.5457099,10.0071697 53.5455944,10.0072077 53.5455767,10.0071943 53.5455323,10.0072674 53.5454953,10.0072503 53.5454826,10.0074516 53.5453855,10.0093051 53.5446153,10.0093884 53.5445676,10.0094699 53.5445209,10.0095531 53.5444632,10.0096317 53.5443671,10.0097413 53.5442372,10.0097723 53.5442475,10.0099212 53.5440467,10.0098947 53.5440442,10.0101234 53.5437488,10.0102361 53.5437748,10.0102682 53.543735,10.0104812 53.5437971,10.0106659 53.5438432,10.0109655 53.5439236,10.010894 53.5440107,10.0115311 53.5441828,10.0115751 53.5441164,10.0117136 53.5441577,10.0118371 53.5441946,10.0117921 53.54427,10.0135756 53.544745,10.0150361 53.5451437,10.0150027 53.5451842,10.0152142 53.5452492,10.0152607 53.545205,10.0156755 53.5453254,10.0157325 53.5453185,10.0160319 53.5454028,10.0167686 53.5456165,10.0167999 53.5456749,10.0166797 53.5457994,10.0149157 53.5480164,10.0152598 53.5481148,10.0154636 53.5478584,10.0162563 53.5468613,10.0162939 53.546814,10.0165317 53.5465148,10.0171007 53.5457541,10.0171832 53.5457396,10.018206 53.5460034,10.0188373 53.5461662,10.0190504 53.5462216,10.01932 53.5462917,10.0193741 53.5462731,10.0198222 53.5463953,10.0203085 53.5465282,10.0203699 53.5466208,10.0229355 53.5473252,10.0234057 53.5474636,10.0276438 53.5486038,10.0282259 53.5487839,10.0287485 53.5489031,10.0287734 53.5489117,10.0288949 53.5488044,10.0289505 53.5487212,10.028836 53.5486946,10.0284024 53.5485617,10.0278028 53.5483914,10.0261749 53.5479429,10.0235548 53.5472211,10.0231031 53.5470963,10.0202742 53.5463289,10.0199547 53.5462133,10.0191902 53.5459964,10.0191685 53.54599,10.0188516 53.5458997,10.018817 53.5458902,10.0187115 53.5458598,10.0186707 53.5458479,10.0185204 53.5458046,10.0184883 53.545796,10.0184195 53.5457762,10.0182948 53.5458159,10.0181021 53.5457587,10.0180376 53.5456886,10.0173519 53.5455034,10.0172165 53.5454679,10.0170872 53.5454286,10.0167238 53.5453314,10.016386 53.5452448,10.0136012 53.5444416,10.0120096 53.5439977,10.0119425 53.544089,10.0117992 53.5440497,10.0116566 53.5440106,10.0117346 53.5439115,10.0111044 53.543751,10.0110352 53.543846,10.0105508 53.5437232,10.0101967 53.5436317,10.0102673 53.5435353,10.0105355 53.5433018,10.0108955 53.5430097,10.0110799 53.5428854,10.0112536 53.5427876,10.0114286 53.5427087,10.011585 53.5426461,10.0118372 53.5425659,10.0121267 53.5424939,10.0121398 53.542515,10.0124521 53.5424339,10.0124428 53.5424128,10.0162653 53.5414233,10.0162758 53.5414457,10.0165921 53.5413643,10.0165837 53.541348,10.0166891 53.5413154,10.0167081 53.5413367,10.0170076 53.5412553,10.0169947 53.5412337,10.0193214 53.5406348,10.0202918 53.5403816,10.0222763 53.5398693,10.0224481 53.5398262,10.0226078 53.5398079,10.0227825 53.5398048,10.0229491 53.5398143,10.0231226 53.539848,10.0233682 53.5399158,10.0233508 53.5399369,10.0236376 53.5400297,10.023661 53.5400052,10.0240711 53.5401286,10.0241985 53.5401357,10.0244571 53.5402613,10.0245508 53.5403093,10.0244253 53.54039,10.0249911 53.540686,10.0251256 53.5405967,10.0252227 53.540647,10.0252964 53.5406022,10.0253599 53.540562,10.0252575 53.5405091,10.0253947 53.5404224,10.0248268 53.5401235,10.0247583 53.5401721,10.0246802 53.5402256,10.0245824 53.5401697,10.0243267 53.5400486,10.0243785 53.5400106,10.0243069 53.5399741,10.0244097 53.5399031,10.0248492 53.5399018,10.0248819 53.5399243,10.0250068 53.5398531,10.0249732 53.5398387,10.0258743 53.5392846,10.0259054 53.5392222,10.0259129 53.5391737,10.0259536 53.5390907,10.0259818 53.5390445,10.0260154 53.5389896,10.0260938 53.5388612,10.0262001 53.5387344,10.0263559 53.5386471,10.0264154 53.5386225,10.0265291 53.5385754,10.0267512 53.5385189,10.026868 53.5384871,10.0275951 53.5377942,10.0276705 53.5372862,10.0281122 53.5369338,10.028945 53.5366089,10.0288944 53.536525,10.0287879 53.536342,10.0288284 53.5363346,10.0284143 53.53563,10.0283251 53.5356218,10.0270757 53.536464,10.0269417 53.5364251,10.027092 53.5362424,10.0277353 53.53546,10.0278458 53.5354928,10.0280322 53.5352797,10.0280554 53.5352527,10.0281418 53.5352789,10.028153 53.5352788,10.0282011 53.5352431,10.0283126 53.5351097,10.0284063 53.5349977,10.0284157 53.5349841,10.0284123 53.534973,10.0284043 53.5349646,10.0283882 53.5349586,10.0283098 53.5349367,10.0283218 53.5349232,10.0282372 53.5348308,10.0283187 53.5347254,10.0287127 53.534368,10.0287677 53.5343467,10.0288295 53.534344,10.0289317 53.5343421,10.0290787 53.5343222,10.0291998 53.5342895,10.029337 53.5342864,10.0294724 53.5342956,10.029646 53.5343285,10.0298144 53.5343769,10.0299307 53.5344149,10.0301859 53.5344554,10.031213 53.5339859,10.0336479 53.5331638,10.0336886 53.532692,10.0308914 53.5336041,10.0306777 53.5336036,10.0305023 53.533604,10.0303561 53.5335826,10.0301901 53.5335048,10.0300444 53.5334075,10.0299472 53.5332979,10.0298739 53.5331689,10.0298467 53.5329995,10.0299191 53.5328413,10.0301628 53.532566,10.0308554 53.5317835,10.0310804 53.5316453,10.0312935 53.531534,10.0316124 53.5313998,10.0323039 53.5311857,10.0329356 53.5309913,10.0343948 53.5304909,10.0353833 53.5301806,10.0358753 53.5300378,10.0363674 53.5298994,10.037012 53.5297835,10.0373521 53.5297073,10.037688 53.5296687,10.038012 53.5296043,10.0381454 53.5296183,10.038203 53.5296432,10.0382999 53.5298417,10.0384001 53.5299036,10.0384482 53.5298941,10.0385632 53.52989,10.0398724 53.5297384,10.0405027 53.5296884,10.0410591 53.5296708,10.0414651 53.5296634,10.0417275 53.5296756,10.0426195 53.5296983,10.042655 53.5297496,10.0429303 53.5297068,10.0437809 53.529572,10.0437729 53.5295205,10.0437704 53.5295008,10.0445971 53.5294736,10.0449199 53.5295018,10.0459304 53.529443,10.0468054 53.5296214,10.0481725 53.5301449,10.0489529 53.5303558,10.0489403 53.5304293,10.0496567 53.530607,10.0503477 53.5305736,10.0504151 53.5306,10.0505299 53.5307615,10.0506397 53.5308335,10.0518121 53.5306073,10.0542059 53.5302941,10.0580562 53.529782,10.0585789 53.529662,10.0589837 53.5295067,10.0600776 53.5289443,10.0602744 53.5288622,10.0604851 53.5288161,10.0619751 53.5287233,10.062668 53.5286706,10.0626744 53.5285217,10.062809 53.5285182,10.0628847 53.5285129,10.0647864 53.528381,10.0647771 53.5283372,10.0647679 53.5282934,10.0628676 53.5284248,10.0626787 53.5284344,10.0626522 53.5283081,10.0628338 53.5282924,10.0647394 53.5281578,10.0647309 53.5281146,10.0647222 53.5280693,10.062818 53.5282045,10.0626535 53.5282128,10.0626317 53.5280668,10.0621815 53.528098,10.0616974 53.5280598,10.0612703 53.527932,10.0610094 53.5277753,10.0608388 53.5275937,10.0607563 53.5273513,10.0607563 53.5270934,10.0609887 53.5267855,10.0618434 53.5260162,10.0634712 53.525295,10.0636886 53.5252776,10.0668009 53.5258395,10.0673262 53.5257141,10.0678828 53.525804,10.0693008 53.5254468,10.0703052 53.5251988,10.0710931 53.5252731,10.0729908 53.5262753,10.0732926 53.5263682,10.0740204 53.5267203,10.0744743 53.5269444,10.0744647 53.52703,10.075695 53.5276657,10.0796505 53.5297232,10.0801471 53.5299361,10.0801276 53.5299669,10.0823924 53.5304347,10.0827205 53.5304536,10.0832246 53.5304732,10.0838415 53.5304565,10.0844078 53.5303916,10.0850762 53.5302896,10.0909833 53.5291747,10.0930681 53.5287483,10.0931024 53.5286718,10.0938148 53.528529,10.0938663 53.5285953,10.0966919 53.5281452,10.1066696 53.5262711,10.1066677 53.5262122,10.1065911 53.5260408,10.1063271 53.5256262,10.1060639 53.5255427,10.1050233 53.5250668,10.1049386 53.5250395,10.1048619 53.5250364,10.1045924 53.5250861,10.1042078 53.5251586,10.1038347 53.5252436,10.1033137 53.5254136,10.1029758 53.5255665,10.1027046 53.5257358,10.1024351 53.5259339,10.1020777 53.5262188,10.1018201 53.5264763,10.0844308 53.5296958,10.084291 53.5297197,10.0841641 53.5297356,10.0841122 53.5297435,10.0840481 53.5297456,10.0839931 53.5297299,10.0839404 53.5297033,10.083906 53.5296577,10.0839048 53.5296311,10.0839162 53.5295855,10.0844224 53.5275838,10.084463 53.5273881,10.084578 53.5269763,10.0846213 53.5268691,10.0847179 53.5266919,10.0848774 53.5265072,10.0850603 53.5263362,10.0852843 53.526195,10.0855203 53.5260547,10.0857126 53.5259544,10.0860679 53.5258052,10.0896307 53.5243989,10.0899752 53.524249,10.0903794 53.5240913,10.0897839 53.52355,10.0837548 53.5259176,10.0837225 53.5259278,10.0836965 53.5259495,10.0836828 53.5259716,10.0833522 53.5272952,10.0833044 53.5274902,10.0830354 53.5285508,10.0832801 53.52857,10.083246 53.5287135,10.0830014 53.5286904,10.0829922 53.5287176,10.0829933 53.5287675,10.0828861 53.5289598,10.0828336 53.5290295,10.0825968 53.5294125,10.0825741 53.5294741,10.0825352 53.5295004,10.0823634 53.5295409,10.0820969 53.5295984,10.0819097 53.5296302,10.08173 53.5296498,10.0813447 53.5295597,10.0809428 53.5294391,10.0806134 53.5293186,10.0803492 53.5291953,10.0793267 53.5286588,10.076489 53.5271887,10.0753115 53.5265744,10.0752357 53.5266328,10.0746368 53.5263143,10.0741548 53.5260559,10.0740488 53.5259184,10.0723306 53.5250244,10.072318 53.5249563,10.0723202 53.5248841,10.0723518 53.5247854,10.0723975 53.5246854,10.0724509 53.5246264,10.0725346 53.5245983,10.0734894 53.5243961,10.0743236 53.5241935,10.0748111 53.5240475,10.0752929 53.5238759,10.0757116 53.5237163,10.0761375 53.5235187,10.0765265 53.5233094,10.077171 53.52292,10.0828457 53.519401,10.0867647 53.5169661,10.0867927 53.5169285,10.0867944 53.5168955,10.0868202 53.5168903,10.0860366 53.5164488,10.084031 53.5176942,10.0821335 53.5188877,10.0821388 53.5192717,10.0799751 53.5206124,10.0793106 53.5206282,10.0760085 53.5227103,10.0755965 53.5229246,10.0753292 53.5230722,10.0751595 53.5231509,10.0749168 53.5232403,10.0745378 53.523396,10.0740053 53.5235564,10.0738419 53.5235979,10.0717002 53.5241366,10.0715021 53.5241806,10.071378 53.5242221,10.0703621 53.5244673,10.0691194 53.5247705,10.068778 53.5248454,10.0682761 53.5249695,10.0678271 53.5249892,10.0676414 53.5249784,10.067282 53.5249154,10.0670659 53.5248681,10.0669381 53.5248324,10.066814 53.5247862,10.0666955 53.5247002,10.0666562 53.5246064,10.0666588 53.5245357,10.0666813 53.5244328,10.066668 53.5242006,10.0667235 53.524199,10.0667183 53.5241129,10.0667736 53.5236908,10.0668709 53.5228339,10.0668036 53.5227524,10.0667939 53.5227142,10.0668059 53.5226915,10.0669001 53.5225225,10.0669262 53.5224939,10.0670733 53.5224932,10.0671385 53.5224404,10.067176 53.5222889,10.067176 53.5222282,10.0671786 53.5219999,10.0671575 53.5217701,10.0671837 53.5211728,10.0671475 53.5211605,10.0671217 53.5209822,10.0671746 53.520753,10.068684 53.519363,10.0691744 53.5183597,10.069075 53.517823,10.0695778 53.5170889,10.0702902 53.5163234,10.0709168 53.5158641,10.071811 53.515493,10.0731357 53.5148295,10.0734229 53.5141068,10.0738844 53.5136076,10.0742158 53.5129071,10.0744307 53.5119362,10.0744706 53.5114427,10.0744203 53.5113681,10.0743443 53.5112917,10.074173 53.5112164,10.073986 53.5112025,10.0728468 53.5111407,10.0722258 53.511111,10.0713275 53.511064,10.0699939 53.5110079,10.0698693 53.5109627,10.069807 53.5108643,10.0697135 53.5108678,10.0696434 53.5109708,10.0695831 53.5109743,10.0695208 53.5109442,10.069511 53.5105956,10.0694422 53.5105521,10.0693553 53.5111028,10.0694916 53.511427,10.0692699 53.5127124,10.0692038 53.5129273,10.0690502 53.5130215,10.068959 53.51311,10.0688821 53.51313,10.0687477 53.5130957,10.0686853 53.5130129,10.0686565 53.512893,10.0684812 53.5127874,10.0684284 53.5126918,10.0683492 53.5126489,10.0683252 53.5125933,10.0683828 53.5124448,10.0684092 53.5122663,10.0684836 53.512135,10.0685437 53.511741,10.068514 53.5116261,10.0684596 53.5115369,10.0685725 53.5112171,10.0685268 53.5110501,10.0684236 53.5110386,10.0683756 53.5109901,10.0683804 53.5109102,10.0684332 53.5108531,10.0685356 53.5108471,10.0686037 53.5105761,10.0685893 53.5104576,10.0686181 53.5103163,10.0687261 53.5101121,10.0686757 53.5099365,10.0686808 53.5097627,10.0685461 53.5097124,10.0682892 53.5096781,10.0676938 53.5096896,10.0668824 53.5097652,10.0662654 53.5098566,10.0659366 53.5099408,10.065406 53.510025,10.0651323 53.5100365,10.064801 53.5101307,10.0646546 53.5102292,10.0646498 53.5103006,10.064597 53.5103948,10.0645202 53.5106118,10.0643737 53.5109401,10.0642705 53.5110858,10.0641001 53.51121,10.0638672 53.5112414,10.0633574 53.5113436,10.063271 53.5113921,10.0632139 53.5114899,10.0631885 53.5115302,10.0631341 53.5116548,10.0631894 53.5116976,10.063199 53.5117362,10.0631293 53.5117933,10.0631467 53.5121134,10.0631849 53.5121827,10.063187 53.5122601,10.0631918 53.5123129,10.0632062 53.5123957,10.0631758 53.5127657,10.0631513 53.5128688,10.0631309 53.5129924,10.0631472 53.5130615,10.0631982 53.5131112,10.0632838 53.5131536,10.0633205 53.5131912,10.0633215 53.5132142,10.0633083 53.5132233,10.0633103 53.5132366,10.0633287 53.5132488,10.0633214 53.5133021,10.0632782 53.5133535,10.0632902 53.5133864,10.0634031 53.5134263,10.0636335 53.5135077,10.0637127 53.5135691,10.0638808 53.513639,10.063972 53.5136333,10.0640968 53.5135834,10.0641569 53.5135819,10.0641833 53.5136119,10.0641016 53.5137218,10.0639984 53.5137675,10.0639168 53.5137846,10.0634629 53.513775,10.0629455 53.5137195,10.0628502 53.5136723,10.0626547 53.5145068,10.0624761 53.514765,10.0623746 53.5151222,10.0624195 53.5151956,10.0618551 53.5162951,10.0613559 53.5171445,10.0612397 53.5174096,10.061782 53.5186842,10.062265 53.519705,10.0623815 53.5199797,10.0625207 53.5211914,10.0625057 53.5216804,10.0622576 53.5225633,10.0621055 53.5230311,10.0620083 53.5232151,10.0618053 53.5234294,10.061294 53.5238449,10.0608272 53.5241031,10.060454 53.5243606,10.0591388 53.5251792,10.0589805 53.5253481,10.057304 53.5262995,10.0556732 53.5272893,10.0553316 53.5274761,10.0550354 53.5275678,10.0545477 53.527643,10.0537696 53.5277586,10.0534445 53.5278242,10.0531998 53.5278231,10.052626 53.5279032,10.0522103 53.527935,10.0511057 53.5279916,10.0511071 53.5279646,10.0507526 53.5279871,10.050726 53.5278819,10.0502407 53.5279147,10.0502947 53.5282419,10.0503432 53.5282813,10.0502442 53.528342,10.0500222 53.5283649,10.0499147 53.5283441,10.049902 53.5282755,10.0498754 53.5281319,10.0498437 53.52795,10.0484355 53.5279944,10.0473055 53.5280367,10.0463374 53.528073,10.0462984 53.528069,10.0462806 53.5280542,10.0462721 53.5280346,10.046118 53.5280591,10.0460958 53.5280602,10.0460621 53.5279304,10.0460051 53.5279366,10.0459686 53.5280099,10.0459342 53.5281071,10.0458546 53.5281058,10.0457507 53.5281152,10.0456761 53.528106,10.0456016 53.5279783,10.0454891 53.5279816,10.0454912 53.5280914,10.0454161 53.5281239,10.0453227 53.5281251,10.04527 53.5280769,10.0452273 53.5280142,10.0451238 53.5280166,10.0451178 53.5280974,10.0450589 53.5281191,10.0449818 53.5281046,10.0448636 53.5280101,10.04458 53.5282132,10.0444984 53.5282108,10.0436684 53.5282323,10.0436664 53.5282065,10.0436649 53.5281728,10.0434109 53.5281691,10.0432369 53.5280924,10.0426281 53.5281269,10.0419094 53.5281004,10.0417158 53.5280772,10.0415857 53.5280442,10.0415182 53.528001),(10.0027643 53.5464431,10.0021407 53.5466233,10.0021418 53.5466652,10.0016413 53.546721,9.9996958 53.5463476,9.9986141 53.5461814,9.9986913 53.5459778,9.9987137 53.5459294,9.9987516 53.5458476,9.9987864 53.5458538,9.9988094 53.5458082,9.9988861 53.5456442,9.9993363 53.5457186,9.9993702 53.5457241,9.9995277 53.5457486,10.000411 53.5458943,10.0019046 53.5461382,10.002063 53.5461646,10.002216 53.5461886,10.0025395 53.5462434,10.0026613 53.54626,10.0026652 53.5463348,10.0027643 53.5464431),(10.000411 53.5458943,10.0005189 53.5456715,10.0004721 53.5457682,10.000411 53.5458943),(10.0005189 53.5456715,10.000328 53.5456409,9.9999338 53.5455766,9.9996234 53.5455316,9.99956 53.5455218,9.9994714 53.5455104,9.9994283 53.5455028,9.999212 53.5454664,9.9989925 53.5454295,9.99881 53.5453988,9.9987661 53.5453914,9.9985108 53.5453418,9.9982769 53.5452964,9.9980327 53.545249,9.9978293 53.5452095,9.997787 53.5452003,9.9974891 53.5451353,9.9973362 53.5451085,9.9971693 53.5450827,9.9971742 53.5450465,9.9956257 53.5445979,9.995539 53.5445609,9.9954462 53.5445225,9.9955685 53.5444158,9.995554 53.5444031,9.9955965 53.5443348,9.9956373 53.5442764,9.995672 53.5442622,9.9957225 53.5442727,9.9958582 53.5442553,9.9959227 53.5442565,9.9959639 53.5442455,9.9960976 53.544285,9.996256 53.5443318,9.996304 53.544346,9.9973627 53.5446489,9.9974963 53.5446872,9.9976173 53.5447218,9.9976736 53.5447379,9.9976893 53.5447618,9.9995752 53.5453349,9.9996472 53.5453564,9.9997152 53.5453768,10.0000319 53.5454701,10.0004771 53.545603,10.0005555 53.5456277,10.0005189 53.5456715),(9.9550732 53.5047392,9.9558436 53.5031154,9.9564115 53.5029156,9.9572928 53.5018495,9.957646 53.5014392,9.9580216 53.5010834,9.9580724 53.5006899,9.9578091 53.5003109,9.957249 53.5000486,9.9566418 53.4999331,9.9553801 53.4998463,9.9549089 53.4997916,9.9545561 53.4998668,9.9522694 53.5021474,9.9521739 53.5022107,9.9520907 53.5022101,9.9519384 53.502142,9.9508225 53.5012809,9.9497513 53.5004839,9.9495297 53.5004483,9.9492996 53.5004722,9.9491191 53.5005292,9.9489472 53.5006216,9.9467432 53.5029956,9.9466933 53.5030909,9.9466981 53.5031931,9.946744 53.5032978,9.9501078 53.5057724,9.9501729 53.5058733,9.9501719 53.5059756,9.9479643 53.5105129,9.947885 53.510692,9.9479527 53.5108107,9.9478759 53.5108505,9.9435942 53.5142053,9.9423524 53.5150632,9.9421722 53.5150888,9.9420005 53.5149918,9.9418718 53.5148642,9.9417945 53.5146856,9.9417173 53.5144049,9.9424626 53.5090964,9.9428486 53.5066673,9.94323 53.5054975,9.9435511 53.5046801,9.9436198 53.5043149,9.9436981 53.5042344,9.9438166 53.5041683,9.9449641 53.5025223,9.9454227 53.5020978,9.9456494 53.5020164,9.9465281 53.5012171,9.9475366 53.50051,9.9484512 53.4998851,9.9484609 53.49978,9.9486744 53.4996105,9.9496714 53.4990279,9.9502588 53.4986832,9.9506337 53.4983546,9.9512761 53.4976642,9.9536222 53.4948512,9.9538444 53.4945665,9.9540823 53.4944671,9.9549769 53.4941954,9.9546967 53.4939764,9.9547081 53.4939403,9.9547665 53.4939387,9.9551149 53.4941956,9.9553656 53.4940689,9.9558735 53.4939815,9.9596299 53.4933599,9.9596995 53.493235,9.9596082 53.4930106,9.9594995 53.4928245,9.9593852 53.4926527,9.959069 53.4927232,9.9558752 53.4932607,9.9556837 53.4932308,9.9554137 53.4927753,9.9571651 53.4902248,9.958164 53.4890411,9.9595853 53.4875011,9.961166 53.4860409,9.9628102 53.4847557,9.9652719 53.4833483,9.9725836 53.4800016,9.9752052 53.4789594,9.9771546 53.4782209,9.9828846 53.4768445,9.9830906 53.4768157,9.9832751 53.4768373,9.983464 53.476915,9.9835372 53.4769955,9.9828615 53.4792181,9.9834327 53.479644,9.9834688 53.4796531,9.983479 53.4796745,9.9835223 53.4796822,9.9835108 53.4797302,9.9834989 53.479752,9.9833919 53.47983,9.9831463 53.4806096,9.9832085 53.4807233,9.9831854 53.4807762,9.983137 53.4807991,9.9829759 53.480858,9.9823338 53.4808818,9.9804588 53.4815166,9.9803451 53.4823058,9.9818763 53.4823966,9.9814214 53.4862032,9.9808026 53.4899128,9.9806358 53.4911177,9.9800317 53.4914812,9.9780318 53.4952189,9.9769798 53.4955154,9.9756897 53.4982701,9.9759871 53.4989639,9.9759146 53.4990877,9.9761748 53.499149,9.9753341 53.5004434,9.9751345 53.5003985,9.9741847 53.5011044,9.9721557 53.5018042,9.9709293 53.5023496,9.9677123 53.502853,9.9674914 53.5029686,9.9672655 53.5033618,9.9670647 53.5034119,9.9669692 53.5036699,9.9667775 53.5037209,9.9665631 53.5037029,9.9660618 53.5036593,9.9661248 53.5034142,9.9662869 53.5034338,9.9663972 53.5030685,9.966299 53.5030541,9.9663008 53.5029728,9.9660797 53.5028465,9.9657078 53.5027745,9.9656749 53.5027144,9.9654695 53.5027292,9.9654716 53.5028656,9.9652798 53.502907,9.9653545 53.5030765,9.9625996 53.5034805,9.9622736 53.5035936,9.9621488 53.503604,9.9588833 53.5041567,9.9565479 53.5045527,9.9552299 53.504786,9.9550732 53.5047392),(9.9666101 53.5042127,9.9667069 53.5042308,9.9667655 53.5042758,9.9666787 53.5045428,9.9669275 53.5045921,9.9671468 53.5047439,9.9672387 53.5049449,9.9674592 53.5060623,9.9675438 53.5065941,9.968566 53.5084813,9.9693412 53.5099951,9.969696 53.5108101,9.9699986 53.5113845,9.9701872 53.5113604,9.970395 53.511745,9.9702467 53.5117731,9.9707473 53.5125545,9.9710557 53.5125115,9.9711267 53.5126536,9.9709739 53.51268,9.9709655 53.5126513,9.970776 53.5126827,9.9707986 53.5127319,9.9707629 53.5128115,9.9700186 53.5129735,9.9698821 53.5133264,9.9702119 53.5133481,9.9702847 53.5133172,9.9708842 53.5133724,9.9710052 53.5134544,9.9709896 53.513488,9.9709536 53.5135468,9.9708223 53.5135665,9.9707773 53.5136084,9.9709292 53.5138209,9.9708307 53.5139732,9.9709475 53.514133,9.9710981 53.5141707,9.9711413 53.5141607,9.9712191 53.5141841,9.9713935 53.5144122,9.9710288 53.5145054,9.9710126 53.5145754,9.9711204 53.5147634,9.9712648 53.5148774,9.9714632 53.5148284,9.9714795 53.5148489,9.9715239 53.5149046,9.9716606 53.5149338,9.9717872 53.515142,9.9718121 53.5151828,9.9720121 53.5151929,9.9720901 53.5153159,9.9721545 53.5153864,9.9724963 53.5160875,9.9726583 53.516414,9.9727465 53.5165832,9.9727841 53.5167002,9.9728213 53.5167002,9.9739737 53.5189781,9.9745431 53.5201215,9.9744478 53.5201919,9.9747568 53.5210487,9.974838 53.5210288,9.9750061 53.5217234,9.9751122 53.5217118,9.9751377 53.5218002,9.974912 53.5218292,9.9750728 53.5224217,9.9749155 53.5224428,9.9749587 53.5225702,9.9752922 53.5225667,9.9754848 53.5234876,9.9752117 53.5235642,9.9749565 53.5237229,9.9749738 53.523832,9.9750543 53.5241237,9.9749343 53.5241728,9.9751165 53.5254197,9.9750713 53.5254462,9.9743447 53.5255062,9.974343 53.5255303,9.9743385 53.5255836,9.9743313 53.5256541,9.9743442 53.525658,9.9743442 53.5256694,9.9743184 53.5256704,9.9742806 53.5256681,9.9741805 53.5256651,9.9724518 53.5255769,9.9724669 53.525413,9.9724826 53.525404,9.9725144 53.525346,9.9725202 53.5253298,9.9725896 53.524664,9.9726129 53.5242412,9.9689965 53.5170852,9.9664921 53.5164196,9.9659062 53.5165397,9.965635 53.5166241,9.962799 53.5172595,9.9627127 53.5173235,9.9627087 53.5174275,9.9638841 53.5192952,9.9638581 53.5193773,9.963661 53.5194647,9.9637314 53.5196298,9.9637657 53.520089,9.9644743 53.520118,9.9647741 53.5201869,9.9672814 53.5216352,9.967323 53.5216128,9.9694274 53.5228107,9.9693447 53.5228678,9.969997 53.523263,9.9699541 53.523365,9.9703318 53.5242223,9.9705484 53.5247132,9.9707118 53.5246888,9.9708406 53.5249463,9.9707356 53.5250193,9.9680318 53.5248829,9.9607922 53.5207657,9.9605981 53.5208788,9.9603839 53.5207538,9.9603159 53.5207536,9.9601196 53.5207531,9.9600235 53.5260183,9.9582036 53.5266021,9.9578173 53.5265698,9.9574141 53.526421,9.9574427 53.5233296,9.9574481 53.5199985,9.9580609 53.5200144,9.9580431 53.5195943,9.9574227 53.5195804,9.956776 53.5192449,9.9530378 53.5200658,9.9527861 53.5202794,9.9521425 53.5204228,9.9518212 53.5204094,9.9488078 53.5210961,9.9486104 53.5212055,9.9483401 53.5212574,9.9482091 53.5211544,9.9429831 53.5223094,9.9427183 53.5225439,9.9423594 53.522578,9.9415876 53.5225253,9.9410505 53.5223044,9.9410586 53.5218042,9.9409299 53.5217799,9.940797 53.5217302,9.9406939 53.5216624,9.9406404 53.5215841,9.9406161 53.5214805,9.9406677 53.5213835,9.9407684 53.5213209,9.9409028 53.5212804,9.941077 53.519646,9.9411499 53.5195703,9.9424354 53.5193458,9.9438406 53.5191002,9.9442879 53.5190214,9.9445233 53.518661,9.9451906 53.5185235,9.9460737 53.5183504,9.9470951 53.5181095,9.9471982 53.5181057,9.9472776 53.5181184,9.9473339 53.5181655,9.9473971 53.5182782,9.949295 53.5178753,9.9493476 53.5179005,9.9514416 53.5174098,9.9516893 53.5173431,9.9543507 53.5167422,9.9544478 53.5167259,9.9545408 53.5167389,9.9546278 53.5167752,9.9558839 53.5164854,9.9558746 53.5164238,9.9612553 53.5152316,9.9613083 53.5152412,9.9658953 53.5142408,9.9662413 53.5141802,9.9663316 53.5141348,9.9665215 53.5140839,9.9665586 53.5140282,9.966628 53.5139723,9.9667113 53.5139338,9.9667901 53.5138787,9.9668063 53.5138484,9.9667299 53.5137025,9.9666326 53.513551,9.9665987 53.5135272,9.9665053 53.5135304,9.9663339 53.5135524,9.9661535 53.5135403,9.9633424 53.514183,9.9629025 53.5142615,9.9618239 53.5145116,9.9553286 53.5159749,9.9526593 53.5165669,9.9491488 53.5172661,9.9458272 53.517756,9.9435637 53.5181241,9.943366 53.5179027,9.9483726 53.5138178,9.9513566 53.5114781,9.9517455 53.5112944,9.9527189 53.5110296,9.9534769 53.5109278,9.9542274 53.5108785,9.9559504 53.5109049,9.9572505 53.5106055,9.957541 53.5104705,9.9577006 53.5102912,9.9577153 53.5100431,9.9575702 53.5098296,9.9565676 53.5093599,9.9550157 53.5087261,9.954969 53.5086084,9.9572464 53.5067984,9.959224 53.5064533,9.9625712 53.505927,9.9640842 53.5056632,9.9646236 53.5053904,9.9654683 53.5049588,9.9655375 53.5047905,9.9655939 53.5047734,9.9656898 53.5047305,9.9657776 53.5044374,9.966293 53.5044863,9.9663796 53.5042417,9.9664576 53.5042007,9.9666101 53.5042127),(9.9732203 53.5149757,9.9731416 53.5146967,9.9730577 53.5145227,9.9726647 53.5137082,9.9726298 53.513494,9.9723509 53.512946,9.9720986 53.5124914,9.9718273 53.5125369,9.9718043 53.5125054,9.9717004 53.5125233,9.9716402 53.5124131,9.97172 53.5123995,9.9716513 53.5122489,9.9714148 53.5117303,9.9716625 53.5114999,9.9718289 53.5113452,9.9720609 53.5109765,9.9718823 53.5105903,9.971605 53.5100121,9.9711327 53.5092327,9.9705847 53.5081546,9.9699953 53.5070257,9.9702943 53.5048093,9.9703351 53.5047426,9.9704401 53.5046415,9.9743849 53.5027705,9.9776794 53.5019345,9.9775641 53.5017694,9.9775266 53.5017049,9.9775144 53.5016653,9.9774127 53.501513,9.9773317 53.5015099,9.9772349 53.5014539,9.9772375 53.5013621,9.9787246 53.4989664,9.9788177 53.4988036,9.9808093 53.497308,9.9818244 53.4964601,9.981849 53.4962649,9.9816266 53.496099,9.9812811 53.4959282,9.9809902 53.4958396,9.9805987 53.495784,9.9805533 53.4957101,9.9807866 53.4952449,9.9808934 53.4950471,9.9811362 53.4949286,9.9813611 53.4949075,9.9816778 53.4948675,9.9820975 53.4948522,9.9822924 53.4948874,9.9823138 53.4949204,9.9823068 53.4949413,9.9826102 53.4947618,9.9827378 53.4948244,9.9829419 53.4949355,9.9828037 53.4950219,9.9844817 53.4958123,9.9846774 53.4960533,9.9850379 53.4965894,9.9850551 53.4969213,9.9848881 53.4991233,9.9873117 53.4989311,9.9875893 53.498903,9.98763 53.4987899,9.9874841 53.4986878,9.9860765 53.4987491,9.9858276 53.4986317,9.9859477 53.4969213,9.9852268 53.4959818,9.9850634 53.4957815,9.9848206 53.4955777,9.9843552 53.4953079,9.9840112 53.4951366,9.9839472 53.4950723,9.9837031 53.4949081,9.9835271 53.4948581,9.9832985 53.494746,9.9831833 53.4948141,9.9829483 53.4946971,9.9824781 53.4944385,9.982292 53.4933765,9.9826506 53.4926952,9.9827943 53.4927346,9.9828739 53.4926057,9.9829801 53.4925524,9.983054 53.4923667,9.982855 53.4921351,9.9827814 53.49216,9.9826689 53.4921331,9.9825297 53.4921394,9.9824963 53.4919967,9.983005 53.4885333,9.9831937 53.486926,9.983893 53.4828725,9.9844424 53.4809674,9.9841036 53.4809603,9.9838459 53.4809385,9.9838621 53.4808573,9.9838082 53.4808356,9.9837775 53.4808311,9.9837848 53.4808002,9.9838208 53.4807856,9.983878 53.4807897,9.9841731 53.4798231,9.9840972 53.4798157,9.9841064 53.4797851,9.9841189 53.4797472,9.9841504 53.47975,9.9842478 53.4797181,9.9852406 53.4792563,9.9860624 53.476678,9.9862119 53.4765405,9.9919315 53.4754957,9.9926963 53.4754761,9.9928927 53.4755695,9.9935078 53.4755036,9.9937512 53.4755247,9.9941097 53.4754009,9.994508 53.4753509,9.994766 53.4753178,9.9949594 53.4752929,9.995366 53.475191,9.9956047 53.4748526,9.9985487 53.4742702,9.9996005 53.4740533,10.0002795 53.474126,10.0005045 53.4740948,10.0007763 53.4740551,10.0011776 53.4739189,10.0016928 53.4738286,10.0023877 53.4737446,10.0029627 53.4736982,10.0035339 53.4735886,10.0047337 53.4734491,10.0059582 53.4734633,10.006953 53.4735117,10.0084084 53.4736514,10.0098411 53.4738072,10.0114423 53.4739657,10.0116905 53.4739185,10.0119736 53.47402,10.0122084 53.4741317,10.0121794 53.4743926,10.0118447 53.4745588,10.0119636 53.4746384,10.0121341 53.474746,10.0121964 53.4747089,10.0126144 53.4749878,10.0131492 53.4753342,10.0134668 53.475391,10.0149034 53.4756239,10.015269 53.4748836,10.0139351 53.4746236,10.0133368 53.4746443,10.0127588 53.4744255,10.0126398 53.4742635,10.0127634 53.4741875,10.0139733 53.4743855,10.0155921 53.4744954,10.0157881 53.4745299,10.0159114 53.4745495,10.0160704 53.4745331,10.0162435 53.4745328,10.0177622 53.4747929,10.0197585 53.4750785,10.021634 53.4753081,10.023312 53.4753765,10.023455 53.4753933,10.0235994 53.4753883,10.0237054 53.475405,10.0238582 53.4754141,10.0239936 53.4753975,10.0240168 53.4754198,10.0240594 53.4754374,10.0241246 53.4754409,10.0242292 53.4754334,10.024414 53.475487,10.0244696 53.4755632,10.0244141 53.4756104,10.0242831 53.4756358,10.0239867 53.4756312,10.0240042 53.4757118,10.0241867 53.4757433,10.0243243 53.4757474,10.0243369 53.4758198,10.0242891 53.4758555,10.0242883 53.4759181,10.024293 53.4759817,10.0242616 53.4760308,10.0243623 53.4760858,10.0245023 53.4761297,10.024613 53.4761622,10.0246905 53.4762103,10.02475 53.4762851,10.024868 53.4764521,10.0250825 53.4765379,10.0251695 53.4765992,10.0252101 53.476658,10.0252927 53.4767004,10.0253445 53.4767482,10.0254128 53.4768475,10.0256052 53.4770224,10.0259302 53.4772455,10.0258706 53.4772932,10.0259831 53.4773217,10.0260677 53.4773778,10.0261221 53.4774194,10.0262498 53.477465,10.0262515 53.4775356,10.026544 53.4776509,10.0265824 53.4774696,10.026616 53.47745,10.0268618 53.4774351,10.0272192 53.477426,10.0274323 53.4774655,10.0275726 53.4774103,10.0277837 53.4773705,10.0280005 53.4773465,10.0280886 53.4773395,10.028175 53.4773194,10.0282634 53.4773233,10.0283225 53.477312,10.0283485 53.4772754,10.0284076 53.4772558,10.0284501 53.4772023,10.0285919 53.4771882,10.028706 53.4771736,10.0288322 53.4771667,10.0289786 53.4771248,10.0291242 53.4770745,10.0292999 53.4770547,10.029423 53.4770128,10.029502 53.4769999,10.0296205 53.4769934,10.0297064 53.4769754,10.0297708 53.4769401,10.0298599 53.4769096,10.0299535 53.4768874,10.0301081 53.4768267,10.0300735 53.4768164,10.0300642 53.4767364,10.0300009 53.4766732,10.0298169 53.4767786,10.0296404 53.476778,10.0295933 53.476713,10.0296736 53.4766406,10.0299449 53.4765527,10.0301286 53.4765867,10.0302159 53.4766451,10.0302301 53.4768321,10.0302653 53.4769146,10.0303534 53.4769791,10.0303573 53.4770373,10.030306 53.4770851,10.0302818 53.4771346,10.0302382 53.477183,10.0302808 53.4772458,10.0303263 53.4772856,10.0303999 53.477285,10.0306138 53.4772706,10.0306254 53.4772412,10.0305751 53.4772418,10.030576 53.4771773,10.0305934 53.4771392,10.0306418 53.4771122,10.0306877 53.4770984,10.0308005 53.4770955,10.0308649 53.4771091,10.030959 53.4771482,10.0310557 53.4772064,10.031104 53.4772614,10.0311486 53.4773218,10.0311507 53.4773796,10.0311339 53.4774259,10.0311411 53.4774708,10.0311172 53.4774879,10.031122 53.4774993,10.0311734 53.4775053,10.0312477 53.4774886,10.0313494 53.4774537,10.0314548 53.4774013,10.0315697 53.4773974,10.0315948 53.477376,10.0317289 53.4773768,10.0320305 53.4773461,10.032161 53.4773233,10.0323872 53.477245,10.0327331 53.4771901,10.0328624 53.4771666,10.0329785 53.4771303,10.0330527 53.4771089,10.0330898 53.4770968,10.0332167 53.4770882,10.0333352 53.4770576,10.033431 53.4770519,10.0337219 53.4769764,10.0338823 53.4769173,10.0339433 53.4769009,10.0340606 53.476856,10.0341588 53.4768104,10.0342689 53.4767769,10.0344293 53.4767484,10.0345179 53.4767007,10.0346639 53.4766494,10.0347321 53.4766359,10.0347656 53.4766166,10.0348339 53.4766102,10.0349548 53.4765604,10.0350062 53.4765504,10.0350326 53.4765361,10.0351355 53.4765262,10.0351619 53.4765105,10.03512 53.4765126,10.0351164 53.4764856,10.035096 53.4764856,10.0350876 53.4765119,10.0350326 53.4765183,10.0349931 53.4765397,10.0348698 53.476581,10.0347788 53.4766003,10.0345777 53.4766665,10.0344233 53.4767328,10.0341552 53.476789,10.0341444 53.4768111,10.0340977 53.4768225,10.0339301 53.4768873,10.0338499 53.4769201,10.0337063 53.4769557,10.0335842 53.4769743,10.0334932 53.4770206,10.0333735 53.4770341,10.0332371 53.4770402,10.0331987 53.4770121,10.0331593 53.4770184,10.0331928 53.4770476,10.0331976 53.4770719,10.0331042 53.4770804,10.0330599 53.4771011,10.0329606 53.4771239,10.0328397 53.4771588,10.0324123 53.4772236,10.0321107 53.4773219,10.0320078 53.4773297,10.0318593 53.477349,10.0317384 53.4773568,10.0316738 53.4773511,10.0316223 53.4773604,10.0314823 53.4773775,10.0313757 53.4774195,10.0312357 53.4774544,10.0311914 53.4774594,10.0311579 53.4774451,10.0311507 53.477428,10.0311808 53.4773863,10.0311734 53.4773126,10.0311114 53.4772227,10.0310501 53.4771659,10.0309745 53.4771117,10.0308259 53.4770759,10.0306505 53.4770811,10.0305179 53.4771588,10.030515 53.4771963,10.0304473 53.4772124,10.0303979 53.4772141,10.0303553 53.477198,10.0303302 53.47716,10.0303853 53.4771081,10.030457 53.4770557,10.0304424 53.4769837,10.0303999 53.4769457,10.030335 53.4768587,10.0303077 53.4767957,10.0305396 53.4767266,10.0304758 53.476593,10.0305231 53.476548,10.0307121 53.4765508,10.0307169 53.4765944,10.0308303 53.4766043,10.0311257 53.4765663,10.0311469 53.4764707,10.0312414 53.4764636,10.0313312 53.4765044,10.0314305 53.4764679,10.0315581 53.4764651,10.0316833 53.4764299,10.0318298 53.4763019,10.0318026 53.4762462,10.0318397 53.4762205,10.0319414 53.4761878,10.0319893 53.4761878,10.0320851 53.4761479,10.0321509 53.4761365,10.0322694 53.4760681,10.0323089 53.4760645,10.0323352 53.4760517,10.0324011 53.4760538,10.0324526 53.4760816,10.0324992 53.4760823,10.0325507 53.4760831,10.032589 53.4760667,10.0326405 53.4760759,10.0327518 53.4761044,10.0326536 53.4760702,10.0325782 53.4760588,10.0325543 53.4760709,10.0324526 53.4760695,10.0324083 53.476041,10.0323185 53.4760417,10.0323077 53.476051,10.032261 53.4760553,10.0321413 53.4761237,10.0320623 53.4761465,10.0320192 53.4761611,10.0319869 53.4761628,10.0319857 53.476066,10.0319331 53.4760303,10.0319319 53.4759947,10.0319738 53.4760054,10.03203 53.4760154,10.0321114 53.4760033,10.0321365 53.4759669,10.0321533 53.4759327,10.0321701 53.4759156,10.0321904 53.4758494,10.0324538 53.4758301,10.0327279 53.4758287,10.0329948 53.4758451,10.0327051 53.4758159,10.0324202 53.4758202,10.0321856 53.4758366,10.0321689 53.4758586,10.0321495 53.4759207,10.0321294 53.475937,10.0320887 53.4760018,10.031951 53.4759869,10.0319055 53.4759684,10.031811 53.4759114,10.0318253 53.4758871,10.0318624 53.47588,10.0319498 53.4756969,10.031939 53.4755316,10.0318564 53.4755381,10.0318385 53.475558,10.0318134 53.4757368,10.0317751 53.4757468,10.0317619 53.4759997,10.0316051 53.4760047,10.0315368 53.4760524,10.0316853 53.4760888,10.0318062 53.4761308,10.0317722 53.4762055,10.0317774 53.476274,10.0317117 53.4762907,10.0316857 53.4763286,10.0316172 53.47633,10.031525 53.4762921,10.0314943 53.4762653,10.031388 53.4762569,10.0312485 53.4762583,10.0312178 53.4762428,10.0312178 53.4761769,10.0311727 53.4761781,10.0311868 53.4761277,10.0311019 53.4761177,10.0311127 53.4760846,10.0310846 53.4760801,10.031079 53.4760512,10.0312137 53.4760534,10.0312473 53.4760834,10.0312454 53.4760467,10.0312791 53.4760167,10.0313165 53.476,10.0313464 53.4759131,10.0314194 53.4758552,10.0314661 53.4758018,10.0314456 53.4757373,10.0314683 53.4757127,10.0313782 53.4756905,10.0313277 53.475705,10.0312511 53.4757651,10.0312473 53.4757963,10.0313782 53.4758441,10.0313689 53.4758744,10.0313334 53.4758898,10.0312829 53.475922,10.0312342 53.4759142,10.0312641 53.4759777,10.0312698 53.4760066,10.0312305 53.4760178,10.03118 53.4760111,10.031094 53.4760434,10.0311052 53.4760011,10.0311519 53.4759454,10.0311233 53.4759208,10.0310108 53.4759115,10.0308917 53.4759306,10.0308823 53.4759545,10.0309626 53.4760009,10.0309579 53.47606,10.030991 53.4761782,10.0309626 53.4762499,10.0306531 53.4762414,10.0303908 53.476399,10.0303213 53.4763802,10.0302377 53.4763724,10.0301844 53.4764089,10.0301057 53.4764196,10.0300419 53.47641,10.0300103 53.4764369,10.0299261 53.4764229,10.0298282 53.4764605,10.0297472 53.4764504,10.0296601 53.4764729,10.0296292 53.4765121,10.0293872 53.4765535,10.0293523 53.4765679,10.0293359 53.4766129,10.0292881 53.476623,10.0292236 53.4766019,10.0291278 53.4766083,10.0289536 53.4766826,10.0288936 53.4766526,10.0287387 53.4767062,10.0286026 53.4767406,10.0285709 53.4766741,10.028514 53.4765927,10.0284088 53.4765913,10.0283047 53.4765718,10.0282079 53.4765149,10.028128 53.476476,10.0280524 53.4763627,10.0279393 53.4763112,10.0277143 53.4762641,10.0276586 53.4763306,10.0276502 53.476412,10.0276102 53.4764854,10.0276115 53.4765553,10.0277119 53.4765812,10.0278075 53.4765682,10.0278727 53.4766053,10.0279805 53.4766496,10.0280059 53.4767079,10.0279923 53.4767562,10.0278325 53.4767946,10.0276712 53.4767848,10.0275851 53.476864,10.0275517 53.4769216,10.0273859 53.4769381,10.0273666 53.4769986,10.0273194 53.4770102,10.0272686 53.4769943,10.0271851 53.4770238,10.0271296 53.4770382,10.0270795 53.4770238,10.0270614 53.4770001,10.0270822 53.4769675,10.0270187 53.47697,10.0269567 53.4769499,10.0269198 53.4769192,10.0269272 53.4768752,10.0268467 53.476832,10.026716 53.4768134,10.0265816 53.4768039,10.0264656 53.4767917,10.0263285 53.4767591,10.0261824 53.4766948,10.0260719 53.4766519,10.0260411 53.4766074,10.0260762 53.4765879,10.0261593 53.4765846,10.0262455 53.4766328,10.0263085 53.4766686,10.0263642 53.476657,10.0264113 53.4766246,10.026415 53.4765656,10.0264573 53.4765469,10.0265045 53.4765382,10.0266678 53.4765231,10.0268154 53.4765253,10.0268905 53.4765238,10.0269183 53.4765152,10.0269667 53.4765253,10.0270235 53.4765145,10.027061 53.4765137,10.0270877 53.4764907,10.0271288 53.4765008,10.0271518 53.4765116,10.0271929 53.4764907,10.0272558 53.4764914,10.0272885 53.4764698,10.0273502 53.4764554,10.0273563 53.4764115,10.027297 53.4762444,10.0272328 53.4762567,10.0271881 53.4762315,10.0271409 53.4762452,10.0270731 53.4762365,10.026986 53.4762423,10.0268481 53.4762257,10.0267646 53.4762531,10.0266049 53.4762768,10.0264924 53.4763049,10.0264017 53.4763373,10.0262819 53.4763935,10.0262129 53.4764468,10.0261379 53.4764554,10.0260617 53.4764756,10.0260644 53.4764234,10.0261703 53.4763896,10.026233 53.4763434,10.0263881 53.4762797,10.0266502 53.4761874,10.0269899 53.4761479,10.0273072 53.476077,10.0274297 53.4760326,10.0276094 53.4759331,10.0276267 53.4758024,10.0277241 53.4756383,10.027751 53.4755151,10.0277977 53.4753721,10.0278268 53.475328,10.0278666 53.4753172,10.0279157 53.4753292,10.0279476 53.4753465,10.0279451 53.4753998,10.0279211 53.4754561,10.0278281 53.4756347,10.0278229 53.4756883,10.0278107 53.4757323,10.0278499 53.4757363,10.0279501 53.4757244,10.0280252 53.4756848,10.0281814 53.4756885,10.0283469 53.4756649,10.0285155 53.4756663,10.0286175 53.4756174,10.0287166 53.4755704,10.029071 53.4755028,10.0293224 53.4754116,10.0294728 53.4753807,10.0296346 53.4753222,10.029785 53.4752774,10.0298835 53.4752547,10.0299753 53.4751844,10.0301172 53.4751983,10.0302782 53.4751511,10.0307002 53.4750167,10.0309883 53.474946,10.0312349 53.4749146,10.0314127 53.4748774,10.0315884 53.4748549,10.0316896 53.4748024,10.0318581 53.4747512,10.0321111 53.4747556,10.0325367 53.4746805,10.0330494 53.4746211,10.0331991 53.4745805,10.033311 53.4745364,10.0333687 53.4745135,10.0334376 53.4745276,10.0335187 53.4744889,10.0335603 53.4744539,10.0337057 53.4744203,10.0337829 53.4744095,10.0339065 53.4743919,10.0342083 53.4743655,10.034435 53.4743436,10.0346052 53.4743059,10.0346849 53.474268,10.0347907 53.4742226,10.0350263 53.4741358,10.0350727 53.4741336,10.035099 53.4741518,10.0351283 53.4741745,10.0350999 53.4742105,10.0351023 53.4742506,10.0351295 53.4742964,10.0351674 53.47435,10.0352053 53.474381,10.035346 53.4743937,10.0354233 53.4744788,10.0353772 53.4745086,10.0353483 53.4745409,10.0353736 53.4745796,10.0354061 53.4746205,10.0354531 53.4746506,10.0354747 53.4746721,10.0354531 53.4747474,10.0354422 53.4748054,10.0354747 53.4749882,10.0354061 53.4750248,10.0352688 53.4750828,10.0352146 53.4750871,10.0351532 53.4750828,10.0351206 53.4750463,10.0350953 53.475014,10.0350267 53.4749968,10.034951 53.4749978,10.034901 53.4750077,10.0348659 53.4750287,10.0348363 53.4750474,10.0348252 53.4750826,10.0348178 53.4751311,10.0347993 53.4751531,10.034753 53.4751718,10.0346901 53.4751784,10.0346272 53.475185,10.0345921 53.4751674,10.0345606 53.4751487,10.034507 53.4751366,10.0344404 53.4751432,10.0343701 53.4751608,10.034346 53.4751861,10.0343183 53.475218,10.034309 53.4752423,10.0343201 53.4752632,10.0343386 53.4752885,10.0343368 53.4753281,10.0343146 53.4753535,10.0342887 53.47537,10.0342387 53.4753821,10.0341999 53.4753887,10.0341851 53.4753788,10.034174 53.4753678,10.0341203 53.4753623,10.0340759 53.4753755,10.0340389 53.4753975,10.0340149 53.4754372,10.0339945 53.475468,10.0339834 53.4754999,10.0339853 53.4755241,10.0339853 53.4755462,10.0339575 53.4755616,10.0339224 53.4755726,10.0338724 53.4755748,10.033828 53.4755693,10.033791 53.4755704,10.0337707 53.4755737,10.0337448 53.4755847,10.0337466 53.475599,10.0337614 53.4756056,10.0337947 53.47561,10.0338169 53.4756012,10.0338613 53.4755913,10.0339094 53.4755902,10.0339557 53.4755902,10.033976 53.475577,10.0340001 53.4755671,10.0340149 53.4755407,10.0340186 53.4755098,10.0340352 53.4754504,10.0340574 53.4754151,10.0340815 53.4753975,10.0341185 53.4753832,10.0341324 53.4753774,10.0341482 53.4753792,10.0341621 53.4753871,10.034176 53.4754023,10.0342112 53.4754073,10.03425 53.4754059,10.0342961 53.4753962,10.0343385 53.4753763,10.0343688 53.4753543,10.0343846 53.4753312,10.0343919 53.475306,10.0343931 53.4752774,10.034367 53.4752572,10.0343512 53.4752381,10.0343555 53.4752143,10.0343743 53.4751905,10.0344046 53.4751674,10.0344501 53.4751591,10.0344895 53.475158,10.034538 53.4751721,10.0345689 53.4751912,10.0345992 53.4751999,10.0346544 53.4752024,10.0347162 53.475202,10.0347629 53.4751916,10.0348223 53.475177,10.0348611 53.475155,10.0349018 53.4751264,10.0349203 53.4750955,10.034924 53.4750735,10.0349333 53.475035,10.0349758 53.4750559,10.0350535 53.4750911,10.0351127 53.475112,10.035183 53.4751308,10.0352644 53.4751396,10.0353347 53.4751396,10.0354013 53.4751231,10.0354846 53.4750999,10.0355456 53.4750757,10.0355789 53.4750427,10.0356196 53.4749898,10.0356381 53.4749469,10.0356307 53.4749161,10.0356196 53.4748632,10.0356141 53.4747993,10.0356307 53.4746474,10.0356252 53.4745769,10.0356108 53.4745344,10.0355911 53.4744941,10.0355655 53.4744557,10.0355617 53.4744177,10.0355607 53.4743475,10.0356162 53.4742994,10.0356816 53.4742268,10.0358164 53.4741627,10.0358979 53.4741105,10.0360417 53.4740704,10.0361739 53.4740366,10.0362746 53.4740066,10.0363052 53.4740535,10.0362945 53.4740981,10.0363856 53.4741177,10.0364688 53.4741324,10.0365021 53.474161,10.0365132 53.474205,10.0365539 53.4742117,10.0366223 53.4742271,10.0366254 53.4742587,10.0366686 53.4742821,10.0366723 53.4743185,10.0367 53.4743372,10.0367851 53.4743394,10.0369086 53.474389,10.0370555 53.4744432,10.0371792 53.4744638,10.0372162 53.4744539,10.0373031 53.4744528,10.0373198 53.4744693,10.0373771 53.474466,10.037514 53.4744671,10.037564 53.4744726,10.0376842 53.4744836,10.0378211 53.474455,10.0381893 53.4744484,10.0382245 53.4744286,10.0382744 53.4744065,10.0384557 53.4743999,10.0385348 53.4744319,10.0386925 53.4743999,10.0387573 53.4743713,10.0388239 53.4743504,10.0389275 53.4743449,10.0392235 53.4741885,10.0393696 53.4741159,10.0394325 53.474085,10.0395602 53.4740542,10.0396874 53.4740265,10.0398789 53.4739685,10.0400921 53.4739083,10.0402619 53.4738631,10.0403776 53.4738007,10.0405027 53.4737104,10.040647 53.4736543,10.0407179 53.4736807,10.0407375 53.4737244,10.040679 53.4738243,10.0406819 53.4738762,10.0406579 53.4739184,10.0406983 53.4740335,10.0406832 53.4739303,10.040718 53.4738747,10.0407187 53.4738097,10.0408264 53.4736715,10.0408184 53.4736193,10.0410221 53.4735907,10.0412973 53.4734986,10.0413845 53.4734975,10.0415501 53.4734235,10.0416974 53.4733866,10.0418191 53.4733478,10.041906 53.4733366,10.0420468 53.4733258,10.0422214 53.4733643,10.0424121 53.4734238,10.0428654 53.4734286,10.0430802 53.4733851,10.0431969 53.4733443,10.0433334 53.4732691,10.0436416 53.4731753,10.0438305 53.4731423,10.0439783 53.4730253,10.0441687 53.473006,10.0442204 53.4729304,10.0443519 53.4728631,10.0447221 53.4727943,10.0449304 53.4727147,10.0451544 53.4726351,10.0454434 53.4725189,10.0455193 53.4724759,10.0457181 53.4723985,10.0461313 53.4722247,10.0463406 53.4721345,10.0465321 53.4720195,10.0466966 53.4719569,10.0467381 53.4718654,10.0468818 53.4718274,10.0469612 53.471793,10.047062 53.4717607,10.0472249 53.4717207,10.0474039 53.4716359,10.0475399 53.4715399,10.0476143 53.4715117,10.0476395 53.4714709,10.04769 53.4714408,10.0477514 53.471443,10.0478813 53.4713807,10.047957 53.4713163,10.0479678 53.4712776,10.0482312 53.4710908,10.0482601 53.4710457,10.0483792 53.4709855,10.0485487 53.4709254,10.0486895 53.4708244,10.048823 53.4707579,10.0489565 53.4706526,10.0490214 53.4705925,10.0491838 53.4704959,10.0492487 53.4704765,10.0492451 53.4704443,10.0492956 53.4703885,10.0493533 53.4702704,10.0495265 53.4701393,10.0496708 53.4700534,10.049743 53.4699933,10.0498332 53.4699589,10.0499559 53.4699289,10.0499992 53.4698709,10.0500706 53.4697576,10.0500064 53.4698258,10.0499414 53.4698924,10.049844 53.4699504,10.0497971 53.4699525,10.0496853 53.470004,10.0496312 53.4700363,10.0495734 53.4700964,10.0494724 53.4701415,10.0494147 53.4702038,10.0492993 53.4702615,10.0492018 53.4704572,10.049108 53.4705173,10.0489312 53.470629,10.0488193 53.4707214,10.048657 53.4707965,10.0485091 53.4709082,10.0482854 53.4710006,10.0481988 53.4710714,10.0481194 53.4711616,10.0479282 53.4712819,10.0479137 53.4713227,10.0478452 53.4713742,10.047755 53.4714086,10.0476395 53.471443,10.047571 53.4714795,10.0475313 53.4714344,10.0474591 53.471415,10.0474014 53.4713871,10.0473328 53.4713442,10.0472643 53.4712797,10.0472174 53.471211,10.0471632 53.4711852,10.0472138 53.4712389,10.0472498 53.4713098,10.0473509 53.4713742,10.0474014 53.4714107,10.0474627 53.4714408,10.0475176 53.471478,10.0474566 53.4715275,10.0473531 53.471611,10.0472918 53.4716382,10.0471098 53.4716971,10.0468472 53.4717702,10.0465994 53.4718978,10.0465391 53.4719782,10.0462841 53.4721105,10.0458879 53.4722867,10.0455374 53.4724157,10.0454065 53.4724899,10.0452881 53.472547,10.0450998 53.4726302,10.0448677 53.4727049,10.0447926 53.472734,10.0447358 53.4727594,10.0446279 53.472789,10.044441 53.4727918,10.044286 53.4728433,10.0441326 53.4728771,10.0439968 53.4729446,10.0438725 53.4730201,10.0437777 53.4730345,10.0437204 53.4729981,10.0437216 53.4730619,10.0436483 53.4731185,10.0434576 53.4731204,10.0433356 53.4731672,10.0431908 53.4732536,10.0430248 53.4733286,10.0429679 53.4732577,10.0428849 53.4732807,10.0428916 53.4733292,10.0427659 53.473396,10.0425784 53.473396,10.042437 53.4733414,10.0423353 53.4732462,10.0421185 53.4732129,10.0418806 53.4732398,10.0417146 53.4732525,10.0415693 53.473285,10.0414487 53.4733099,10.0412941 53.4733667,10.0411559 53.4734267,10.0410314 53.47347,10.0409357 53.4734935,10.0408025 53.4735166,10.0406294 53.4735447,10.0404297 53.4735615,10.0403594 53.4735879,10.0402706 53.4736166,10.0401781 53.4736595,10.0400875 53.4736959,10.0399228 53.4737828,10.0398229 53.4738357,10.0397471 53.4738621,10.0395787 53.4739018,10.0394196 53.4739293,10.0393086 53.473959,10.0389756 53.4740702,10.0388591 53.4741242,10.0387351 53.4741914,10.0386889 53.4742321,10.0385668 53.4742772,10.0384872 53.4742894,10.0383226 53.4743037,10.0381616 53.4743158,10.0380525 53.4743389,10.0378527 53.4743378,10.0377731 53.4742883,10.0377297 53.4740846,10.0376285 53.4740219,10.0375891 53.4739872,10.0375682 53.4739657,10.0375187 53.4739797,10.0374738 53.4739976,10.0374688 53.4740679,10.0375179 53.4741337,10.0376569 53.4741649,10.0376806 53.4742618,10.037396 53.4742229,10.0372678 53.4741908,10.0371102 53.4741749,10.0370445 53.4741402,10.0370174 53.474086,10.0369635 53.4740351,10.0368833 53.4739889,10.0369203 53.4739283,10.0370232 53.4738825,10.0371902 53.4738852,10.0372072 53.4738416,10.0370261 53.473774,10.0370322 53.4736736,10.0368445 53.4736592,10.0366631 53.4737091,10.0365701 53.473661,10.0365392 53.4735777,10.0375858 53.4732449,10.0385261 53.4728986,10.0399768 53.4723227,10.0415448 53.4716881,10.0418629 53.4715826,10.042004 53.4716617,10.0422624 53.4715901,10.0422417 53.4715122,10.0423332 53.4714319,10.0423004 53.4713719,10.0424153 53.4713384,10.0425454 53.4712605,10.0432038 53.4710027,10.0433064 53.4711207,10.043438 53.4711304,10.0436306 53.4710395,10.0436426 53.4709773,10.0436161 53.4708119,10.0441998 53.4705244,10.0447761 53.4702996,10.0452813 53.4699777,10.0456579 53.4697445,10.04571 53.4697536,10.0457526 53.469779,10.0457574 53.4698163,10.0457574 53.4698622,10.0457941 53.4698925,10.0458687 53.4698988,10.0459551 53.4698812,10.0460368 53.4698452,10.0460711 53.4698107,10.0460842 53.4697741,10.0460628 53.4697339,10.0460036 53.4697163,10.0459042 53.4697198,10.0458391 53.4697057,10.0457799 53.4696712,10.0464486 53.4692573,10.0480351 53.4682859,10.0481462 53.4683146,10.0482342 53.4683014,10.0483234 53.4682704,10.0483886 53.4682307,10.0484219 53.4682004,10.0484447 53.4681671,10.0484068 53.4681238,10.048292 53.4680786,10.0498218 53.466944,10.0504489 53.4662899,10.0505402 53.4661293,10.050682 53.4660094,10.0507886 53.4659642,10.0510778 53.4657433,10.0511555 53.4657727,10.051184 53.4658045,10.0511907 53.4658483,10.0511321 53.465958,10.0511131 53.466092,10.0510002 53.466239,10.0508699 53.46642,10.050238 53.4673649,10.0509148 53.466488,10.0510072 53.4663944,10.0510566 53.4663356,10.051192 53.4661869,10.0512352 53.4660833,10.0512428 53.4660019,10.0512624 53.4659326,10.051311 53.4658654,10.0513724 53.4658225,10.0514199 53.4657665,10.0514331 53.4657104,10.0514239 53.4656906,10.0513812 53.4656671,10.0513195 53.4656337,10.0513015 53.4656151,10.0513151 53.4655859,10.0527782 53.4644849,10.0535124 53.463992,10.0536417 53.4639709,10.0545226 53.4643108,10.0550053 53.4645893,10.0554773 53.464748,10.0555236 53.4648294,10.0555993 53.4648767,10.0556679 53.4649089,10.0558267 53.4649003,10.0557433 53.4648761,10.0556692 53.4648669,10.0556011 53.4648302,10.0555633 53.4647585,10.0556787 53.4647371,10.055805 53.4647048,10.0558396 53.4646765,10.0560268 53.4645956,10.0559278 53.4645692,10.0557956 53.4646039,10.0556605 53.4646209,10.0554538 53.4646205,10.0551874 53.4644533,10.0547733 53.464206,10.055217 53.4640033,10.055341 53.4640401,10.0555688 53.4639514,10.0556948 53.4638408,10.0558277 53.463671,10.0554484 53.4633357,10.0555572 53.4633666,10.0558151 53.4635273,10.0559837 53.4635387,10.056329 53.4633961,10.0567018 53.4632134,10.0585935 53.4618537,10.0590406 53.4614802,10.0590007 53.4613842,10.0588641 53.4612975,10.0587065 53.4612784,10.0603655 53.4604489,10.0620466 53.4595882,10.063994 53.4585829,10.0640758 53.4585707,10.0641289 53.4586414,10.0641818 53.4586686,10.0644458 53.4586773,10.0646201 53.4586535,10.0647086 53.4585704,10.0647207 53.4584674,10.0646696 53.4583961,10.064551 53.4583847,10.0644796 53.4583417,10.0644742 53.4582928,10.0663857 53.4573639,10.0677157 53.4567988,10.0685899 53.456456,10.0698148 53.4560866,10.071368 53.4557294,10.0716926 53.4556793,10.0718288 53.4556723,10.0719483 53.4557307,10.0720164 53.4558954,10.0719957 53.4559838,10.07168 53.456079,10.0706243 53.4565389,10.0696115 53.4571113,10.0681187 53.4580532,10.0669422 53.4588897,10.0664965 53.4592848,10.0655914 53.4601431,10.065522 53.4601446,10.0652754 53.4600007,10.0649681 53.4602797,10.0650196 53.4603053,10.0647349 53.4605272,10.0649675 53.4606992,10.0646939 53.4609785,10.0644493 53.4608926,10.064383 53.4611636,10.0644801 53.4611898,10.0645233 53.4612588,10.0638597 53.4621105,10.063761 53.4621177,10.0637023 53.4620794,10.0639324 53.4615907,10.0632037 53.461397,10.0631485 53.4614602,10.0632343 53.4615419,10.0629854 53.4618894,10.0628996 53.4618843,10.0627966 53.4620529,10.0632061 53.4626495,10.0632847 53.4626873,10.0633608 53.4628389,10.0633842 53.4630024,10.0633496 53.4631732,10.0628573 53.4643437,10.062724 53.4646281,10.0622513 53.4646095,10.0621837 53.4646364,10.0622333 53.4647062,10.0625039 53.4647518,10.0626167 53.4647522,10.0626569 53.4647844,10.0619814 53.4664679,10.0617391 53.4672315,10.0613845 53.4684704,10.0611983 53.4693161,10.061144 53.4701525,10.0610783 53.4710618,10.0610725 53.4725531,10.0610441 53.4726385,10.0609592 53.4726791,10.0609322 53.4727905,10.0609066 53.4728812,10.0609879 53.4729162,10.061002 53.4730293,10.0608461 53.4736892,10.0607112 53.4743144,10.0602688 53.4752308,10.0600322 53.475837,10.0593479 53.4771813,10.0587904 53.4780769,10.0578228 53.4790055,10.0567118 53.4798692,10.0551411 53.4809729,10.0549063 53.480874,10.0548009 53.4809057,10.0543137 53.4806974,10.0541784 53.4808048,10.0542592 53.4808579,10.0548888 53.481157,10.053423 53.482539,10.053164 53.482866,10.0523318 53.4836871,10.0517259 53.4844046,10.0511844 53.4851485,10.0509465 53.4858166,10.0507214 53.4864556,10.050508 53.4871394,10.0504002 53.4879134,10.0502689 53.488687,10.0499851 53.4900374,10.0489986 53.4899902,10.0489649 53.4900855,10.0492175 53.4901223,10.0493846 53.4901666,10.0496325 53.4902345,10.0499721 53.4903198,10.0499678 53.4903929,10.0499086 53.4907896,10.0492189 53.4913346,10.0484341 53.4919154,10.0476766 53.4923969,10.0458555 53.4928817,10.0452347 53.4935552,10.0448027 53.4942303,10.0445566 53.4948387,10.0444773 53.4952389,10.0444253 53.4954374,10.0445183 53.4955529,10.0449996 53.4955464,10.0461016 53.4955447,10.0470286 53.4955334,10.0472282 53.4954732,10.0471461 53.4952975,10.0467961 53.4949103,10.0464707 53.4944776,10.0465391 53.4941913,10.0470077 53.4942123,10.0472881 53.4938085,10.0491553 53.4925607,10.0493071 53.4924485,10.0494137 53.4924127,10.0495723 53.4924412,10.0496446 53.4925598,10.0495974 53.4928764,10.0495947 53.4949103,10.0497143 53.4957238,10.0499546 53.496756,10.0505628 53.4980311,10.0512782 53.4993624,10.0516366 53.4999428,10.0515616 53.5000249,10.0513786 53.500022,10.0512591 53.5000957,10.0512474 53.5001625,10.0512892 53.5002373,10.0514116 53.5002871,10.0516196 53.5002744,10.0516671 53.50019,10.0517543 53.5001674,10.0518478 53.5002682,10.0533842 53.5024291,10.0537371 53.5029082,10.0537082 53.5030179,10.0536223 53.5030587,10.0534163 53.5030536,10.0533058 53.5031326,10.053374 53.5033368,10.0531377 53.5034258,10.0527313 53.5034575,10.0524558 53.5034653,10.0520635 53.5034572,10.0520461 53.5035731,10.0525441 53.5035848,10.0528271 53.503595,10.0531489 53.503572,10.0541012 53.5034482,10.0541973 53.5034994,10.054992 53.5046367,10.0557258 53.5057198,10.0555847 53.5057534,10.0554833 53.5058097,10.0552781 53.5058964,10.055286 53.5060701,10.0551433 53.5061281,10.0552243 53.5062174,10.0555992 53.5061553,10.0557645 53.5061227,10.0559046 53.5061927,10.0566269 53.5074024,10.0573801 53.5089595,10.0576001 53.5095756,10.0577622 53.5103589,10.0578893 53.5112503,10.0559157 53.5121023,10.0564796 53.5126501,10.0566733 53.5125955,10.0571399 53.5131596,10.0573985 53.5131625,10.0576824 53.513371,10.0573608 53.5142967,10.0567925 53.5153583,10.0561032 53.5162846,10.0545832 53.5179249,10.0528494 53.5192415,10.0510469 53.5203897,10.0490986 53.5215633,10.0464807 53.5230023,10.0466095 53.5230737,10.0460258 53.5233442,10.0459314 53.5232625,10.0453049 53.5235942,10.0446526 53.5237983,10.0435024 53.5243749,10.0427299 53.5247218,10.0422235 53.5248137,10.0415111 53.5250382,10.0416227 53.52513,10.0408417 53.5254311,10.0406614 53.5255076,10.0400864 53.5258137,10.0401618 53.5259048,10.0393121 53.5262364,10.0384452 53.5264711,10.0373895 53.5268334,10.0367972 53.5269201,10.0363852 53.5270069,10.0356643 53.5271956,10.0351235 53.5274201,10.0348145 53.5273589,10.0346104 53.5272067,10.0346189 53.5266097,10.0356661 53.5262985,10.0381466 53.5254872,10.0382629 53.5250122,10.0384996 53.5241651,10.0382845 53.5239657,10.0379799 53.5236582,10.0355754 53.5244267,10.0334191 53.5251325,10.03172 53.5256847,10.0315817 53.5257451,10.0315036 53.5258086,10.0314995 53.5258917,10.031555 53.5259813,10.0327661 53.5272946,10.0329657 53.527428,10.0331561 53.5274972,10.0332248 53.5275668,10.0332135 53.5276375,10.0331432 53.5277389,10.0330524 53.5277907,10.0328848 53.5278515,10.0327099 53.5278895,10.0322083 53.5273751,10.0308827 53.5259521,10.030741 53.5259952,10.0308178 53.5260754,10.030732 53.5261002,10.0306128 53.5259734,10.0307999 53.5259174,10.0308196 53.525893,10.030673 53.5257146,10.0306982 53.5257086,10.0303258 53.5252857,10.0301129 53.5250657,10.0300558 53.5249406,10.030209 53.5248012,10.0375629 53.5223645,10.0380863 53.5222074,10.0403218 53.5214914,10.042009 53.5209153,10.04909 53.5185883,10.0498539 53.5183076,10.0507465 53.5178687,10.0515376 53.5175048,10.0525208 53.5167925,10.053492 53.5156118,10.0531778 53.5155075,10.0531436 53.515444,10.0535286 53.5153512,10.0537373 53.5149908,10.055144 53.5136744,10.0546357 53.5131627,10.0541456 53.5126022,10.0490295 53.5143199,10.045005 53.5156886,10.0416319 53.5168012,10.0405759 53.5171299,10.0403854 53.5171282,10.0379871 53.517921,10.0301002 53.5204811,10.0286987 53.5209412,10.0284 53.5211308,10.0281765 53.5211121,10.0280965 53.5210004,10.0273416 53.5208789,10.0251442 53.5207913,10.021865 53.5207346,10.0202975 53.5207054,10.0196272 53.5206938,10.019018 53.5206796,10.0190154 53.5207238,10.0162022 53.5217773,10.0160587 53.5219055,10.0150939 53.521883,10.0148506 53.5218751,10.014551 53.5218673,10.0145505 53.5216882,10.0135461 53.5216727,10.0128431 53.5218226,10.0126054 53.5216579,10.0116576 53.5216429,10.0114665 53.5218094,10.0114293 53.5218197,10.0113855 53.5218252,10.0112967 53.5218214,10.0109548 53.5218066,10.0108808 53.5218018,10.0108466 53.5217771,10.0108559 53.5216926,10.0029142 53.5208687,9.9944942 53.5200165,9.993025 53.519888,9.990487 53.5196355,9.9885519 53.5199318,9.98828 53.5200267,9.9850266 53.522,9.9840796 53.5225783,9.9841007 53.5226798,9.9839511 53.5227735,9.983894 53.522708,9.9838619 53.5226711,9.9838025 53.5226756,9.9837472 53.5226822,9.9837468 53.5227112,9.9837453 53.5228245,9.9835812 53.5229176,9.983414 53.5229203,9.9832944 53.5232419,9.9832244 53.5233501,9.983106 53.5234681,9.9830717 53.5236036,9.982798 53.5237624,9.9826962 53.5238283,9.9821863 53.5239061,9.9817664 53.5236611,9.978954 53.5254362,9.9787422 53.5255002,9.9785897 53.5254662,9.9784624 53.5254437,9.978513 53.5253589,9.9786611 53.525211,9.979094 53.5249581,9.979237 53.5248274,9.9808743 53.5238589,9.9819219 53.5231876,9.9818412 53.5230636,9.9817244 53.5229908,9.9815603 53.52288,9.9813934 53.5227676,9.9777647 53.525029,9.9773638 53.5251376,9.9770581 53.5251102,9.976711 53.5234981,9.9764186 53.5221118,9.9762665 53.5214909,9.9763267 53.521337,9.9772444 53.5208722,9.9772497 53.5207773,9.9778327 53.5204913,9.9780001 53.5204975,9.9793867 53.5198328,9.9791515 53.5196263,9.9790349 53.5195258,9.9788253 53.5195318,9.9768237 53.5205701,9.9767863 53.5206105,9.976655 53.5206813,9.976458 53.520698,9.9762993 53.5206656,9.976162 53.5205817,9.9756593 53.5197369,9.9755905 53.5197622,9.9753787 53.5193478,9.9754468 53.5193192,9.9754772 53.5191102,9.9753048 53.5187463,9.9752013 53.5179222,9.9749257 53.5178737,9.975046 53.5177587,9.9752342 53.5178317,9.9756761 53.5175255,9.9755192 53.5174042,9.9758199 53.5172021,9.9756552 53.5171026,9.9751297 53.5174415,9.9748943 53.5173622,9.9747976 53.5173793,9.9743034 53.5168383,9.9742276 53.5168647,9.9736775 53.5158236,9.9733665 53.5152105,9.9733012 53.5151227,9.9732203 53.5149757),(9.98366618865225 53.4797484305812,9.9836872 53.4797509,9.9837038 53.4797018,9.9839413 53.4797292,9.9839322 53.4797665,9.9840256 53.4797765,9.9839447 53.4797679,9.9839368 53.4798038,9.9839184 53.4798024,9.9836264 53.4807728,9.9836411 53.480775,9.9836268 53.4808149,9.9833525 53.4807919,9.9833794 53.4807362,9.9836715 53.4797819,9.9836544 53.4797802,9.98366618865225 53.4797484305812),(9.9962623 53.4713923,9.9967986 53.4709797,9.9968564 53.4708219,9.9969083 53.4708105,9.9977041 53.470668,9.9978465 53.470714,9.9980124 53.4707387,9.9982293 53.4707643,9.998547 53.4707183,9.9991017 53.4707315,9.9996178 53.4709332,9.9999258 53.471106,10.0000014 53.4712502,10.0000116 53.4713404,9.9999381 53.4714058,9.9998209 53.4714322,9.9987322 53.4715777,9.9982312 53.4716744,9.9974204 53.4718308,9.9963584 53.4720357,9.9962309 53.4719332,9.9961264 53.4718383,9.9961579 53.4715923,9.9962623 53.4713923),(9.9346829 53.5396609,9.9347256 53.5395435,9.9348824 53.5393985,9.9368761 53.5381585,9.9383018 53.5369516,9.9391276 53.5359941,9.9391504 53.5359062,9.9396968 53.5351519,9.9397864 53.5351259,9.9403284 53.5343268,9.9409802 53.5328796,9.9412566 53.532017,9.9413512 53.5317068,9.9414246 53.5312618,9.9414763 53.5309984,9.9415622 53.5305601,9.941516 53.528944,9.9414542 53.5275914,9.9414419 53.5273226,9.941959 53.526559,9.9419374 53.5264377,9.9418356 53.5260321,9.941757 53.5259428,9.9416749 53.525896,9.9414118 53.5257921,9.9413517 53.5257636,9.9413166 53.5257243,9.9412381 53.5249282,9.9411935 53.5247945,9.94129 53.5246752,9.9412182 53.5245169,9.9412012 53.5241378,9.9411937 53.5238454,9.9421004 53.5232599,9.9421475 53.5231999,9.942275 53.5230374,9.9425043 53.5230224,9.9426678 53.5230117,9.9427598 53.5230057,9.9428061 53.5230027,9.9430129 53.5231259,9.9431143 53.523242,9.9448175 53.5228463,9.945595 53.522696,9.9460489 53.5226076,9.9466553 53.5223878,9.9474965 53.5219766,9.9480289 53.5217217,9.9483443 53.5216675,9.9494303 53.521669,9.9523682 53.5209766,9.9523533 53.5209424,9.9525176 53.5207563,9.9532495 53.5205953,9.9537924 53.5206133,9.9542988 53.5208194,9.954281 53.5254652,9.9540423 53.5254665,9.9536035 53.5283155,9.9533831 53.5283614,9.9535639 53.5284007,9.95209 53.5313673,9.950376 53.5349274,9.9472235 53.5388461,9.944293 53.5389539,9.9442476 53.5390257,9.9441878 53.5390605,9.9441248 53.5390828,9.9440544 53.5391079,9.9439405 53.5391288,9.9422966 53.5391855,9.9387409 53.5392989,9.9379408 53.5398383,9.9376565 53.5399966,9.937585 53.540002,9.935183 53.5399754,9.9349812 53.5399377,9.934832 53.5398659,9.9347421 53.5397774,9.9346829 53.5396609),(9.977757 53.5268388,9.9781055 53.5266212,9.9782879 53.526523,9.9784277 53.5264619,9.9785615 53.5264124,9.978759 53.5263607,9.9790037 53.5263234,9.9796263 53.5262405,9.9798748 53.5261909,9.9805213 53.5260767,9.9806719 53.5260617,9.9808431 53.5260282,9.9809989 53.5259856,9.9812092 53.5259174,9.9813554 53.5258624,9.9814648 53.5258148,9.9814506 53.525805,9.981589 53.5257328,9.9817037 53.5256795,9.98233 53.525583,9.9823292 53.5255685,9.9831477 53.5254154,9.9877758 53.5246831,9.9889851 53.5246352,9.989678 53.5246071,10.0035379 53.5224686,10.0045368 53.5223597,10.0051105 53.522356,10.0056327 53.5224095,10.006131 53.5225038,10.0072531 53.5228784,10.0077946 53.522941,10.007844 53.5229244,10.0081599 53.5229589,10.0091296 53.5233353,10.009182 53.5233525,10.0091574 53.5235085,10.0094916 53.5238802,10.0094983 53.5241147,10.0093186 53.5241828,10.00937 53.5242384,10.009166 53.524303,10.0089658 53.5243664,10.0084956 53.524194,10.0070635 53.5244337,10.0070976 53.5245247,10.0052854 53.5248101,10.0041284 53.5249923,10.0040407 53.524807,10.0024044 53.5250749,10.002407 53.5251954,10.0012017 53.5253768,10.0009274 53.5257903,9.9981505 53.5300096,9.9974125 53.5303788,9.9974622 53.5304144,9.9961734 53.531064,9.9944505 53.5319288,9.9940872 53.5318381,9.9941059 53.5318108,9.9940349 53.5317936,9.9942539 53.5314603,9.9909479 53.5335462,9.9904717 53.533999,9.9904565 53.5341744,9.9900993 53.5340447,9.9900944 53.5339578,9.9899989 53.5339589,9.9897233 53.5317786,9.9895088 53.5317281,9.9895526 53.5315783,9.989286 53.5313259,9.9888993 53.5312252,9.9880001 53.5312677,9.9886255 53.5361521,9.9876544 53.5368152,9.9871772 53.5369328,9.9859942 53.5353233,9.985547 53.5354307,9.9854476 53.5352927,9.9849381 53.5313133,9.9848261 53.5312811,9.9848087 53.5311511,9.9836111 53.531207,9.9837293 53.5320513,9.9836187 53.5321238,9.9830831 53.5321404,9.9820385 53.5321949,9.9820472 53.5322321,9.9819997 53.5322808,9.9817285 53.5322905,9.9816589 53.5322604,9.981653 53.5322106,9.9788952 53.5323475,9.9788873 53.5323679,9.9787897 53.5323801,9.9787623 53.532433,9.9785278 53.5324351,9.9784594 53.5323801,9.9784131 53.5323801,9.97842 53.53241,9.9767541 53.5324703,9.9765945 53.5323821,9.9772176 53.5283231,9.977275 53.5280018,9.9773078 53.5277564,9.9773559 53.5274582,9.9774001 53.5274642,9.9774222 53.5273219,9.9774216 53.5271978,9.9774796 53.527181,9.9775282 53.5269965,9.977757 53.5268388),(10.0103271 53.5243519,10.0115724 53.5257127,10.0128424 53.5271096,10.0129308 53.5272028,10.0137272 53.5280477,10.013852 53.5280775,10.0142513 53.5281552,10.0142731 53.5281807,10.0144014 53.5283117,10.0142775 53.5284297,10.0142518 53.5286287,10.013505 53.5290317,10.0132938 53.5294714,10.0137565 53.5298665,10.014154 53.530349,10.0077462 53.5329566,10.0076671 53.5328984,10.0043815 53.5342491,10.0044583 53.5343053,10.0021854 53.5352546,10.0021116 53.5351819,10.0018573 53.5352994,10.000642 53.5354622,9.9971608 53.5356657,9.9970921 53.5355025,9.9998661 53.5348027,9.9999912 53.5342315,9.9985362 53.5333055,9.9980047 53.5331731,9.9979301 53.5331095,9.9975404 53.533308,9.9975361 53.5334304,9.9943988 53.5355719,9.9928745 53.5347883,10.000758 53.5303138,10.0005102 53.5301623,10.0008658 53.5300308,10.0006926 53.5299251,10.000694 53.5299084,10.0067511 53.5268716,10.0068259 53.5269243,10.0087021 53.5259812,10.0087983 53.5259081,10.0088811 53.5258097,10.0089159 53.5256683,10.0089208 53.5249601,10.0089794 53.5249601,10.0090644 53.5249613,10.0090832 53.5245143,10.0094915 53.5243758,10.0095476 53.5244322,10.0099879 53.5244615,10.0103271 53.5243519),(10.0150133 53.5279295,10.0149795 53.5278714,10.0150037 53.5278188,10.0150764 53.5277691,10.0158348 53.5275072,10.0145338 53.5259125,10.014417 53.5259461,10.0129446 53.52434,10.0130797 53.5243092,10.0126006 53.5238893,10.0120131 53.5235479,10.0115495 53.5233741,10.0108172 53.5232535,10.0102873 53.5232038,10.010041 53.5231991,10.0094712 53.5230096,10.0093765 53.5229709,10.0090529 53.5228181,10.0092849 53.5227026,10.0093784 53.5225651,10.0098274 53.5223953,10.0101195 53.522295,10.0101533 53.5222574,10.0104925 53.5221283,10.0105896 53.5221157,10.0106234 53.522116,10.0107557 53.5221171,10.0109206 53.522122,10.0110307 53.5221477,10.0110712 53.5221698,10.0110917 53.5222179,10.0110808 53.52229,10.0131502 53.5223309,10.0131413 53.5222011,10.0139438 53.5222039,10.0148346 53.5222165,10.015092 53.5222203,10.015658 53.522229,10.0158258 53.5223009,10.0160251 53.5223885,10.017144 53.5226038,10.0186757 53.5226364,10.0189347 53.5226424,10.018936 53.5226094,10.0189633 53.5225954,10.019006 53.5225897,10.0190436 53.5225892,10.019092 53.5225956,10.0191281 53.5226075,10.0191295 53.5226424,10.0193019 53.5226455,10.019573 53.522651,10.0211898 53.5226805,10.0214214 53.522685,10.0214221 53.5226505,10.0214487 53.5226365,10.0214898 53.5226326,10.0215271 53.5226333,10.0215777 53.5226398,10.0216075 53.5226536,10.0216065 53.5226877,10.0221569 53.5227016,10.0227838 53.5227108,10.0228116 53.5227136,10.0233272 53.5227664,10.0239849 53.5227853,10.024464 53.5228519,10.0245836 53.5228504,10.0262355 53.5228714,10.0266718 53.5227569,10.0267023 53.5227276,10.0268552 53.5227822,10.0269422 53.5228392,10.026956 53.5229165,10.0269239 53.5230281,10.0274467 53.5235999,10.0283546 53.5245991,10.0284254 53.5246415,10.0291869 53.5254864,10.0291637 53.525502,10.0298213 53.5262138,10.0298624 53.5262152,10.0300357 53.5261608,10.0301654 53.5262964,10.0300819 53.526329,10.0299926 53.5262433,10.029884 53.5262786,10.0309116 53.5274996,10.0310391 53.5275167,10.0314493 53.5279498,10.0317831 53.5284385,10.0290464 53.5293181,10.0289688 53.5293034,10.0285782 53.5294674,10.0286593 53.5295481,10.0282088 53.5297196,10.028093 53.5296149,10.0276878 53.5297663,10.0276237 53.5297443,10.0266982 53.5300406,10.0265683 53.5300178,10.0263172 53.5290005,10.0261016 53.5283939,10.0259974 53.5280945,10.0258364 53.5279069,10.0256714 53.5278875,10.0253598 53.5279991,10.0252105 53.5280545,10.0252425 53.5282452,10.0252907 53.5284731,10.0253472 53.5286502,10.0253599 53.5287347,10.0253901 53.5288681,10.0254443 53.5289871,10.0254811 53.5290459,10.0255732 53.5293302,10.0255923 53.5293744,10.0254888 53.5293885,10.0255266 53.5295124,10.0256449 53.5295488,10.0257655 53.5299595,10.0258741 53.5303405,10.0258413 53.5303874,10.0257637 53.5304685,10.0256603 53.5305324,10.0253314 53.5306346,10.0247171 53.530807,10.0245312 53.5308668,10.0243055 53.5307408,10.024072 53.53078,10.0240335 53.5308929,10.0238482 53.5310566,10.0226506 53.5312483,10.0224355 53.5311777,10.0219442 53.5312918,10.0222012 53.5315726,10.0220472 53.531618,10.0208915 53.5319855,10.01893 53.5325557,10.0167002 53.5332601,10.0151195 53.5337717,10.0150537 53.5337493,10.0128968 53.5341804,10.0116822 53.5343993,10.0105358 53.5345897,10.0097542 53.5346989,10.0089851 53.5348145,10.0085764 53.5348664,10.0083584 53.5348955,10.0068426 53.5350659,10.0051839 53.5352376,10.0038005 53.5353348,10.00378 53.5352406,10.0038333 53.53522,10.0053166 53.5349351,10.0082391 53.5343069,10.0081793 53.5339513,10.0084127 53.5337135,10.0085698 53.5337466,10.0088828 53.5333668,10.0100002 53.5329082,10.0097877 53.5327474,10.0097944 53.5327222,10.0153249 53.5304291,10.0154377 53.5300869,10.0155039 53.5299849,10.0157236 53.5298604,10.0174885 53.5292466,10.0162592 53.527838,10.0152612 53.5281504,10.0151962 53.528152,10.0151378 53.5281201,10.0150788 53.5280675,10.0150133 53.5279295),(10.0277792 53.5218414,10.027928 53.521867,10.0279967 53.5219235,10.0283423 53.5218154,10.0309551 53.5208821,10.0337633 53.5199907,10.0361878 53.5192079,10.0387246 53.5183769,10.0413261 53.5175149,10.0428163 53.5170819,10.0458376 53.5160867,10.0493871 53.5149699,10.0518457 53.514132,10.052292 53.5142443,10.0525298 53.5145187,10.0527664 53.5148278,10.0525888 53.5150867,10.0519011 53.5158744,10.0514066 53.5163539,10.0506772 53.5170241,10.0487215 53.5179545,10.0474083 53.5184291,10.0449621 53.5192303,10.0433399 53.5197253,10.041134 53.5204857,10.0365678 53.5219553,10.0342418 53.5227411,10.0313236 53.52368,10.02979 53.5242003,10.0291746 53.5240915,10.0284014 53.5232919,10.0276951 53.5225058,10.0273679 53.5223702,10.0273317 53.5223388,10.0277137 53.5218697,10.0277792 53.5218414),(9.9829275 53.5240637,9.9832015 53.5239077,9.9833644 53.5239039,9.9848815 53.5229892,9.9933855 53.5216854,9.9935251 53.5216831,9.9936245 53.5217371,9.9941947 53.5230293,9.9941661 53.5230983,9.9940673 53.523144,9.9926495 53.5233675,9.9865074 53.5243328,9.985311 53.5245208,9.9842164 53.5246928,9.9840388 53.5247207,9.9836724 53.5248078,9.9830431 53.5249104,9.9826539 53.5251334,9.9821501 53.5251997,9.9820564 53.5250922,9.9829075 53.5242777,9.9829275 53.5240637),(10.0194168 53.5224277,10.0194323 53.522208,10.0200838 53.5222242,10.0200684 53.5224439,10.0195491 53.522431,10.0194168 53.5224277),(10.0643142 53.5111885,10.0643989 53.5110723,10.0644543 53.5109309,10.0645162 53.510838,10.0645813 53.5108534,10.0646041 53.5109329,10.0647897 53.510991,10.065021 53.5109735,10.0652001 53.5109503,10.0653694 53.5109154,10.0654215 53.5109561,10.0653597 53.5110897,10.0653499 53.5111924,10.0652261 53.5114461,10.0651349 53.5114887,10.0648516 53.5114596,10.0647669 53.511479,10.0647083 53.5116572,10.0645976 53.5116959,10.0644869 53.5116668,10.0643077 53.5115332,10.0643142 53.5114015,10.0643403 53.511264,10.0643142 53.5111885),(10.06561 53.5115241,10.0654716 53.5114794,10.0654795 53.5113712,10.0655586 53.5112583,10.0658235 53.5111149,10.0663494 53.5110561,10.0667369 53.5110655,10.066994 53.511089,10.0670968 53.5111643,10.0671126 53.5113195,10.0670651 53.5114182,10.0668753 53.5114841,10.06561 53.5115241),(9.9724329 53.5257433,9.9732725 53.5257835,9.9741545 53.5258368,9.9742974 53.5258411,9.9743252 53.5258413,9.9743331 53.5258508,9.9743238 53.5258651,9.9742946 53.5258643,9.9742858 53.525932,9.9742812 53.5259671,9.9742543 53.5261608,9.9742462 53.5262319,9.9742655 53.5262377,9.9742641 53.5262483,9.9742407 53.5262566,9.9741074 53.5262498,9.9723764 53.5261607,9.9724329 53.5257433),(9.9723437 53.52632,9.9742451 53.5264139,9.9742086 53.526446,9.9742019 53.5265026,9.9742086 53.5265058,9.9742006 53.5265767,9.974212 53.5265949,9.9742556 53.5266198,9.9748201 53.5267648,9.9756095 53.5269619,9.9757817 53.527,9.975834 53.5270343,9.9759623 53.5271941,9.9760164 53.5272129,9.9760714 53.5272488,9.9761113 53.5272528,9.9760319 53.5279432,9.9760063 53.5279421,9.976003 53.5279758,9.9758635 53.5279686,9.9758094 53.5280098,9.9757722 53.5282508,9.9757632 53.5283146,9.9758801 53.5283189,9.9758763 53.5283546,9.9759611 53.5283579,9.9759193 53.5289018,9.9754699 53.5318919,9.9753078 53.5318833,9.9752146 53.53266,9.9748002 53.5329836,9.9731762 53.5329049,9.9731286 53.5329714,9.9728072 53.5329524,9.972809 53.5329304,9.9727126 53.5329257,9.9726515 53.5328045,9.9714683 53.5327455,9.9714211 53.5327186,9.9713999 53.5326808,9.9717095 53.5306266,9.9714673 53.5304026,9.9711328 53.5303854,9.9710901 53.5306974,9.9707699 53.5326886,9.9702166 53.532665,9.9701817 53.5328735,9.9694696 53.5328263,9.9690954 53.5328137,9.9689247 53.5328121,9.9687664 53.5328033,9.9683002 53.5327768,9.9683577 53.532282,9.9677596 53.5322452,9.9679535 53.5310812,9.9679369 53.5310461,9.9675319 53.5309218,9.967279 53.5310909,9.9669453 53.5312597,9.9566573 53.53546,9.9565524 53.5353668,9.9569459 53.5352107,9.9556854 53.5344157,9.9555642 53.5345689,9.9555092 53.5345524,9.9554631 53.5346114,9.9553811 53.5345872,9.955896 53.5339433,9.9638199 53.5307141,9.9643099 53.5303749,9.9648799 53.5299508,9.9671398 53.5280603,9.9677172 53.5276118,9.9674082 53.5274482,9.9674516 53.527409,9.9669656 53.5271861,9.9669066 53.5272381,9.966635 53.5271223,9.9656745 53.5275187,9.9648365 53.5278946,9.9631124 53.5286501,9.9635231 53.5289922,9.9585898 53.5311547,9.956079 53.5322636,9.9555424 53.5318276,9.9556256 53.5317957,9.9554403 53.5316583,9.955456 53.5315606,9.9553026 53.5315525,9.9553861 53.5309602,9.9553377 53.5309563,9.9558185 53.5297473,9.9564902 53.5294741,9.9599784 53.5279047,9.9642491 53.5260473,9.9649244 53.5260674,9.9665308 53.526108,9.9668792 53.526137,9.9672335 53.5261462,9.9680063 53.5264775,9.9679773 53.5264894,9.9683181 53.5266352,9.9683824 53.5266263,9.968518 53.5266527,9.9692333 53.5268114,9.9695476 53.5268677,9.9698276 53.5269056,9.970175 53.5269292,9.9708413 53.5269572,9.9721997 53.5270072,9.9722824 53.5266013,9.972316 53.5264417,9.9723437 53.52632),(9.9566235 53.5374056,9.9599827 53.5361766,9.9676869 53.5333312,9.9677307 53.533374,9.9682415 53.5331705,9.9682729 53.5329316,9.9685949 53.5329563,9.9687316 53.5329675,9.9687436 53.5330002,9.9689475 53.5330154,9.9689542 53.5329851,9.9694048 53.5329978,9.9694517 53.5330202,9.9701567 53.533035,9.970134 53.5332253,9.9707147 53.5332534,9.9703736 53.5355942,9.9701448 53.5369349,9.9685761 53.5373802,9.9685606 53.5375198,9.9681187 53.5376544,9.9680422 53.5375686,9.9670686 53.5378607,9.9665907 53.5380467,9.9656724 53.5383233,9.9655703 53.5384487,9.965524 53.5385788,9.9655234 53.5387219,9.9655533 53.5388597,9.9655977 53.539025,9.965669 53.5391864,9.9662519 53.5399504,9.9666231 53.540382,9.9667829 53.5412004,9.9675939 53.5421675,9.9677584 53.5422453,9.9677936 53.5422881,9.9670608 53.5424668,9.966157 53.5425975,9.9653546 53.5425887,9.9647652 53.5429589,9.9644923 53.5428963,9.9643777 53.5429722,9.9638814 53.5426921,9.9610171 53.5425075,9.9605936 53.5423292,9.9605204 53.5420363,9.9592966 53.5420998,9.9593184 53.5422574,9.9591274 53.5422896,9.958511 53.542172,9.9553686 53.541596,9.9541162 53.5413722,9.9511924 53.5408449,9.9512583 53.540579,9.9527102 53.5405237,9.9569825 53.5401552,9.9572815 53.5401324,9.9571201 53.5396766,9.9572151 53.5396639,9.9572064 53.5396404,9.9571366 53.539567,9.9570784 53.5394896,9.9570328 53.5394057,9.9570084 53.5393167,9.957002 53.5392241,9.9570136 53.539116,9.9570051 53.539093,9.9569085 53.5391066,9.9566751 53.5384648,9.9527352 53.5389801,9.9526771 53.538976,9.952634 53.5389621,9.9526001 53.5389379,9.952588 53.5389085,9.9525952 53.5388812,9.9526207 53.5388552,9.9566235 53.5374056),(9.9707503 53.5373588,9.9709314 53.5362572,9.9713819 53.5332887,9.9725862 53.5333727,9.9726909 53.5332234,9.973071 53.5332435,9.9731274 53.5333971,9.974645 53.5334784,9.9746289 53.5336433,9.9745298 53.5336743,9.9743591 53.533666,9.9743174 53.5339774,9.9749782 53.5340119,9.9750006 53.5340331,9.9731375 53.5359758,9.9731562 53.5360185,9.9740941 53.5363295,9.974197 53.5363103,9.9742553 53.5362539,9.9750512 53.5354111,9.9751087 53.5353933,9.9760633 53.5365414,9.9770901 53.5377349,9.9780398 53.5388564,9.9781228 53.5389569,9.9781151 53.5389755,9.9776632 53.5391124,9.977686 53.5391802,9.977636 53.5392525,9.9762613 53.5396761,9.9760184 53.5395789,9.9757265 53.5392306,9.9754858 53.5392812,9.9756715 53.5395432,9.9756862 53.5396946,9.9757911 53.539887,9.9758148 53.5399505,9.975795 53.5400045,9.9757278 53.5400515,9.9749567 53.5402748,9.9742235 53.5404601,9.9741943 53.5404899,9.9741246 53.540516,9.9732711 53.5407678,9.9731031 53.5407653,9.9727544 53.5408717,9.9727911 53.5409269,9.9725691 53.5409941,9.9727219 53.5411734,9.9717356 53.5414694,9.9717085 53.5414412,9.9706719 53.5417446,9.9704454 53.5416972,9.9703101 53.5416892,9.9701919 53.5417009,9.9689022 53.5420539,9.9663875 53.5390039,9.966298 53.5387332,9.9663427 53.5386449,9.9671263 53.5384153,9.9671426 53.5384234,9.9680498 53.5381587,9.9680732 53.5379728,9.9685144 53.5378437,9.9686272 53.5379682,9.9699448 53.5375309,9.9700629 53.5375424,9.9701225 53.537603,9.9703098 53.5378893,9.9708791 53.5386096,9.97097 53.5386646,9.971189 53.5386155,9.9713032 53.5385684,9.9713382 53.5385142,9.970758 53.5377272,9.9707256 53.5375441,9.9707503 53.5373588),(9.979119 53.5385333,9.9790534 53.538446,9.9791562 53.538419,9.9788951 53.5380201,9.9787618 53.5380542,9.9767553 53.5354776,9.9764183 53.5349174,9.9763779 53.5342933,9.9764374 53.533533,9.9765337 53.5335146,9.9765958 53.5330808,9.9767223 53.5329877,9.9767539 53.5329527,9.9767702 53.5328891,9.9767678 53.5328134,9.9768387 53.5328023,9.9784679 53.5327543,9.9785004 53.5326935,9.9785569 53.5326528,9.9787743 53.5326507,9.9788274 53.5326874,9.9788704 53.5327449,9.9816798 53.5326424,9.9817347 53.5325699,9.9820309 53.5325601,9.9820847 53.532625,9.9827806 53.5326056,9.9831325 53.5325402,9.9833111 53.5326213,9.9835198 53.5342772,9.9836099 53.5342966,9.9836563 53.5344789,9.9839672 53.5345261,9.9840499 53.535188,9.9841048 53.5353437,9.9843627 53.535684,9.9844818 53.5356538,9.9850739 53.5365062,9.9847985 53.5365873,9.9847517 53.536641,9.9849571 53.5369339,9.9846392 53.5370189,9.9847529 53.5371874,9.9846451 53.5372162,9.9848527 53.5375109,9.9849524 53.5374885,9.9849926 53.5375708,9.9825886 53.5381924,9.9825708 53.538172,9.9819351 53.5383382,9.9810891 53.5385859,9.9795121 53.5390499,9.979119 53.5385333),(9.9766487 53.5279768,9.9766247 53.5279573,9.9766425 53.5278347,9.9766726 53.5278189,9.9767041 53.5278429,9.9766843 53.5279613,9.9766487 53.5279768),(10.0288621 53.5325901,10.0287958 53.5326956,10.0285008 53.5330457,10.0284878 53.5331287,10.0284554 53.5331854,10.0281248 53.533596,10.0281128 53.5336271,10.0281141 53.5336574,10.028127 53.5337113,10.0281354 53.5337223,10.02811 53.533741,10.0280615 53.5338042,10.0279611 53.5339348,10.0278627 53.5340595,10.027797 53.5341484,10.027785 53.534164,10.0274397 53.534381,10.0261329 53.5359548,10.0260008 53.5361139,10.0257033 53.536472,10.025462 53.5367642,10.0252213 53.5370997,10.0248627 53.5375674,10.0244618 53.5380633,10.0242071 53.5383209,10.0240071 53.5385095,10.0237312 53.5387255,10.0237619 53.5388784,10.0231543 53.5390505,10.022127 53.5393493,10.0210416 53.5395997,10.0207351 53.5396572,10.0180511 53.5400603,10.0180813 53.5401228,10.0180346 53.5401537,10.0161906 53.5404281,10.0138669 53.5407663,10.0129935 53.5408918,10.0126157 53.5409921,10.0123304 53.541088,10.012033 53.54122,10.0117975 53.541355,10.0116144 53.5414688,10.0114208 53.5416072,10.011196 53.541762,10.0111088 53.5418423,10.0100525 53.5428152,10.0091641 53.5435773,10.0081544 53.544338,10.0079066 53.5445187,10.0069648 53.5446701,10.0068366 53.5446816,10.0065581 53.544708,10.0063706 53.5447255,10.0062821 53.5447298,10.0043714 53.5444757,10.0033426 53.5443342,10.0033122 53.5444348,10.0033012 53.5444342,10.0032673 53.5444242,10.0032423 53.5444185,10.0032049 53.5444126,10.0031545 53.5444113,10.0031291 53.5444119,10.0030804 53.5444156,10.0030731 53.5444152,10.0030864 53.5443668,10.0029566 53.5443586,10.0029394 53.5443779,10.0019006 53.5441096,10.0018691 53.5440981,10.0018293 53.5440795,10.0017914 53.5440701,10.0014194 53.543978,9.9993234 53.5434157,9.9995632 53.5431115,9.9996454 53.5429956,9.9997107 53.543,9.9998236 53.5428714,9.9999805 53.5426999,10.0000047 53.5427087,9.9999966 53.5427358,10.0000208 53.5427374,10.0000516 53.5427414,10.0000637 53.5427326,10.0001669 53.5427685,10.0011235 53.5414031,10.0011608 53.5413472,10.0015102 53.540854,10.0014834 53.540761,10.0018955 53.5406535,10.0019564 53.5405666,10.0019739 53.5405708,10.0024444 53.5398943,10.0027456 53.5397771,10.0059061 53.5399177,10.0068044 53.5399172,10.007313 53.5398987,10.007808 53.5398656,10.0085134 53.5398088,10.0085137 53.5398185,10.0092274 53.5397533,10.0099079 53.5396837,10.0099036 53.53967,10.0105942 53.539587,10.011264 53.5394979,10.0116828 53.5394384,10.0123291 53.5393321,10.0133908 53.5391616,10.0138436 53.539089,10.0149263 53.5389096,10.0159541 53.5387109,10.0171374 53.5384922,10.0185379 53.5376123,10.0185064 53.5375316,10.0187264 53.5374168,10.0204752 53.5369992,10.0202582 53.5366336,10.0195194 53.5368107,10.0178648 53.5371967,10.0172878 53.5373239,10.0167093 53.5374455,10.0166804 53.537598,10.0167549 53.5376261,10.0165849 53.5379073,10.0164417 53.5379245,10.0164088 53.5380507,10.01597 53.5380522,10.0159862 53.5379831,10.0156129 53.5381031,10.0156934 53.5382034,10.0150566 53.5383931,10.0149743 53.5382951,10.0147592 53.5383681,10.0146064 53.5383007,10.014396 53.5385354,10.0134152 53.5386399,10.0128858 53.5383357,10.0128717 53.5382838,10.0126436 53.5381873,10.0125132 53.538207,10.011922 53.5382964,10.0108845 53.5384456,10.0098023 53.5385901,10.008781 53.5387092,10.0086124 53.5387274,10.0086161 53.5387495,10.0084948 53.5387623,10.0080044 53.5388138,10.007998 53.5387914,10.007281 53.5388593,10.0069658 53.5388781,10.0067195 53.5388803,10.0067157 53.538854,10.0062658 53.53886,10.0057272 53.5388364,10.0051577 53.5387921,10.0047971 53.5387393,10.0044578 53.538679,10.0040301 53.5385862,10.0040389 53.5385637,10.0036414 53.5384921,10.003761 53.538155,10.0041529 53.5381438,10.0041479 53.5380996,10.0045217 53.5380807,10.0044916 53.5379598,10.0044513 53.5377572,10.0059345 53.5376027,10.007524 53.5374249,10.0092544 53.5372118,10.0098179 53.537145,10.0110504 53.5369753,10.0116465 53.5368866,10.012009 53.5368292,10.0133268 53.5365977,10.0144649 53.5363935,10.0155561 53.5361922,10.016548 53.536001,10.0171569 53.5358762,10.0181794 53.5356581,10.0192985 53.5354165,10.0203701 53.5351927,10.0211277 53.5350399,10.0214738 53.5349689,10.0225019 53.5347512,10.0225408 53.5344084,10.0230079 53.5342591,10.0230636 53.5343259,10.0231337 53.534304,10.0242024 53.5339745,10.0241669 53.5339326,10.0267605 53.5330332,10.0273293 53.5328432,10.0282136 53.532557,10.0286731 53.5323976,10.0287447 53.5323856,10.0287896 53.5323888,10.0288326 53.5324027,10.0288615 53.5324231,10.0288858 53.5324505,10.0289015 53.5324883,10.0288959 53.5325232,10.0288621 53.5325901),(9.9834397 53.5457875,9.9833608 53.5452774,9.9833599 53.5452731,9.9833608 53.5452687,9.9833648 53.545263,9.9833686 53.5452601,9.9833733 53.5452576,9.9833789 53.5452557,9.9833851 53.5452545,9.9833915 53.5452541,9.983398 53.5452544,9.9834041 53.5452555,9.9834098 53.5452573,9.9834168 53.5452611,9.983422 53.5452665,9.9834236 53.5452703,9.9834239 53.5452742,9.983519 53.5457778,9.9835427 53.5459475,9.9835278 53.5459746,9.9834932 53.5459548,9.9834397 53.5457875),(9.9827894 53.5433379,9.9829247 53.543253,9.98228 53.5428787,9.9823139 53.5428458,9.9831559 53.543011,9.9833221 53.5430766,9.9835575 53.5432072,9.9835437 53.5432125,9.983594 53.5432577,9.9869458 53.5434891,9.9870626 53.5434958,9.987122 53.5434993,9.9872086 53.5435058,9.9874974 53.5435253,9.9878885 53.5435518,9.9882762 53.5435781,9.9885196 53.5435946,9.9888456 53.5436167,9.9893227 53.543649,9.9896538 53.5436714,9.9898469 53.543683,9.9899474 53.5436912,9.9900653 53.5437009,9.9903244 53.5437126,9.9906709 53.5437349,9.990998 53.543751,9.9914563 53.5437876,9.9919186 53.543818,9.9923932 53.543848,9.9925923 53.5438645,9.9930782 53.5439047,9.993174 53.5438933,9.9935801 53.5439243,9.9939511 53.5439561,9.9941582 53.5439743,9.9946924 53.5440209,9.9948699 53.5440368,9.9951979 53.5440618,9.9952337 53.5441115,9.9952964 53.5441138,9.995288 53.544168,9.9952869 53.5442426,9.9952567 53.5443036,9.9952154 53.5443712,9.9951027 53.5443607,9.9948677 53.5445634,9.9946734 53.5447295,9.994716 53.5447478,9.9946549 53.5448014,9.9945928 53.5448558,9.9938493 53.5445497,9.9937917 53.5446009,9.9929232 53.544287,9.9925684 53.5442216,9.992385 53.5442019,9.9921849 53.5441949,9.9899859 53.5442495,9.9897905 53.5442442,9.9896223 53.5442392,9.988078 53.544169,9.9868652 53.5440403,9.9867485 53.5440331,9.9867026 53.5439327,9.9861197 53.5438888,9.9861113 53.543938,9.9854459 53.5438976,9.9854541 53.5438499,9.9849078 53.5438185,9.9839412 53.5437554,9.9839536 53.5437325,9.9835005 53.54371,9.9834007 53.5437026,9.9833726 53.5436949,9.9833506 53.5436867,9.9832781 53.5436597,9.9832196 53.5436357,9.9831889 53.5436232,9.9829804 53.5435016,9.98292 53.5435271,9.9828605 53.5434931,9.9827887 53.5434521,9.9827574 53.5434342,9.9828273 53.5433898,9.9827894 53.5433379),(9.9824118 53.5425929,9.9822434 53.5423672,9.9820449 53.5423011,9.9820865 53.5422597,9.98212 53.5422692,9.9820776 53.5422234,9.9820993 53.5422043,9.9820467 53.5421628,9.9820548 53.5421562,9.9820792 53.542156,9.9820876 53.5421588,9.9821531 53.5421604,9.9821798 53.5421416,9.9823318 53.5421519,9.9823491 53.5421153,9.9825746 53.5421277,9.9848711 53.5422886,9.9849634 53.5422944,9.9850123 53.5422973,9.985067 53.5423006,9.9851759 53.5423072,9.9852247 53.5423101,9.9853237 53.5423206,9.9854376 53.5423283,9.9872737 53.5424537,9.9872641 53.5424961,9.9873097 53.5424997,9.987348 53.5425026,9.9873883 53.5425058,9.9901883 53.5426873,9.9902585 53.5426931,9.9924111 53.5428412,9.992544 53.5428502,9.9925992 53.5428004,9.9923588 53.5427138,9.9923847 53.5426921,9.9924088 53.5427014,9.9924511 53.5426631,9.9924891 53.5426279,9.9924626 53.5426178,9.9925303 53.5425538,9.9923919 53.5425503,9.9924053 53.5422982,9.9923523 53.5423075,9.992322 53.5423131,9.9922838 53.5423201,9.9921348 53.5420338,9.9921652 53.5420282,9.9921683 53.5420002,9.9920468 53.5419955,9.9918524 53.5419854,9.9904993 53.5419092,9.9894226 53.5418539,9.9884932 53.5418062,9.9857449 53.541665,9.9854788 53.541649,9.9853633 53.5416448,9.9853183 53.5416425,9.9851923 53.5416355,9.9851367 53.5416324,9.9850728 53.5416287,9.9848669 53.5416212,9.9832434 53.5415472,9.9827273 53.5415194,9.9827575 53.5413769,9.9849795 53.5407348,9.9851585 53.5406829,9.9852125 53.5406656,9.9852988 53.5406378,9.9853808 53.540612,9.9854743 53.5405826,9.9859339 53.5406078,9.9859288 53.5406325,9.9859749 53.5406543,9.9860119 53.5406714,9.9860302 53.5406739,9.986634 53.5406055,9.9867019 53.5405984,9.9867722 53.540595,9.9868201 53.5405949,9.9868688 53.5405974,9.9869421 53.540606,9.9872209 53.5406612,9.9875159 53.5406337,9.9875868 53.5406276,9.9876555 53.5406242,9.9877247 53.5406256,9.9877951 53.5406311,9.9878643 53.5406439,9.9879605 53.5406681,9.9883259 53.540767,9.9896607 53.5408263,9.9896561 53.5408509,9.9928878 53.5409989,9.9929501 53.5409168,9.992991 53.5408634,9.9929573 53.5408489,9.992844 53.5406814,9.9928325 53.5406553,9.9928325 53.5406155,9.9928435 53.5405755,9.9928725 53.5404699,9.9925952 53.5403345,9.9925698 53.5403148,9.9925506 53.5402978,9.9925336 53.5402722,9.9925205 53.5402443,9.9925074 53.5400516,9.9924983 53.5400303,9.9924872 53.5400058,9.9924939 53.5399106,9.9911786 53.5399648,9.989521 53.540019,9.9883094 53.5400442,9.9879219 53.5400461,9.9878796 53.5398545,9.990143 53.5393068,9.992452 53.5387475,9.9925618 53.5387219,9.9931812 53.5385773,9.9933366 53.5385422,9.9934958 53.538515,9.9936504 53.5385035,9.9941781 53.5384903,9.9950158 53.538466,9.9950226 53.5383178,9.9985197 53.538367,10.0016896 53.5391553,10.0015857 53.5393,10.0014454 53.5394996,10.0013914 53.539486,10.0003178 53.5409786,10.0003433 53.5409749,10.0002926 53.5410617,10.0001637 53.5412825,10.0001196 53.5412864,10.0000422 53.5414099,9.9998964 53.541635,9.9997307 53.5418871,9.9997509 53.5419571,9.9996936 53.5419436,9.9996224 53.5420519,9.9990425 53.5428315,9.9989706 53.5429612,9.9988687 53.5431126,9.9988124 53.5431955,9.9986827 53.543396,9.9986579 53.5433885,9.9985159 53.5435726,9.9984308 53.5436935,9.9984394 53.5436957,9.9984176 53.5437304,9.9984074 53.5437286,9.9983445 53.5438088,9.9985508 53.5438693,9.998978 53.5438939,9.9990353 53.543897,9.9994155 53.5439187,9.9994066 53.5439233,9.9993476 53.5439539,9.9989146 53.5441782,9.9988416 53.5442161,9.9987867 53.5442445,9.9986771 53.5443007,9.9986275 53.544327,9.9985201 53.5443826,9.9984706 53.5444082,9.9983976 53.5444461,9.9982529 53.5445211,9.9981902 53.5445242,9.9981165 53.5445278,9.9980064 53.5444988,9.9978513 53.5445027,9.9977663 53.5445243,9.9977262 53.5445149,9.9976834 53.5445048,9.9976007 53.5444853,9.9975115 53.5444643,9.9974626 53.5444621,9.9966683 53.5442491,9.9965892 53.5442266,9.996434 53.5441825,9.9963662 53.5441628,9.9962961 53.5441424,9.9960476 53.5440731,9.995734 53.5439857,9.9954085 53.543895,9.995309 53.5438745,9.9951983 53.543859,9.9949085 53.543831,9.9933481 53.5436976,9.9932622 53.5436912,9.9930991 53.5436801,9.9930719 53.5436732,9.9909319 53.5435295,9.9909371 53.543484,9.9908283 53.5434746,9.9908304 53.5434457,9.9906801 53.5434334,9.9906738 53.5434568,9.9905493 53.5434448,9.9905343 53.5435084,9.9901169 53.5434779,9.9900065 53.5434689,9.9898985 53.5434601,9.9894672 53.543434,9.9890218 53.5434038,9.9885711 53.5433733,9.9881347 53.5433437,9.9876906 53.5433137,9.9872447 53.5432835,9.9871583 53.5432776,9.9871051 53.5432745,9.9869822 53.5432674,9.9841069 53.5430686,9.9840422 53.5430613,9.9839445 53.5430459,9.9838533 53.5430225,9.9836093 53.5429043,9.9836335 53.5428898,9.9835611 53.5428543,9.9834812 53.5428255,9.9833914 53.5427992,9.9833069 53.5427753,9.9832103 53.542749,9.9824118 53.5425929),(9.8984253 53.5365827,9.8984113 53.5363965,9.8983845 53.5363973,9.8983818 53.5363702,9.8982933 53.536371,9.8982906 53.536336,9.8985374 53.5363304,9.8985401 53.5363941,9.898469 53.5363957,9.8984803 53.5365819,9.8984253 53.5365827),(9.9998883 53.5438871,10.0012644 53.5442697,10.0016507 53.5443775,10.0029667 53.5447625,10.0029924 53.5447321,10.002997 53.5447098,10.003016 53.5447117,10.0030505 53.5447224,10.0030715 53.5447269,10.0031149 53.5447328,10.0031596 53.5447346,10.0031847 53.5447325,10.0032192 53.5447311,10.0032387 53.544733,10.0032111 53.5448016,10.0034724 53.5448348,10.0034294 53.5449543,10.0052586 53.5451957,10.0052728 53.5452025,10.005355 53.5455826,10.0053502 53.5455875,10.0036184 53.5461411,10.0035728 53.5461557,10.0034461 53.5462189,10.0033129 53.5462733,10.0032943 53.5462807,10.0031209 53.5463339,10.0030629 53.5462689,10.0028511 53.5460337,10.0024317 53.5459548,10.0024123 53.5459534,10.0022605 53.5459143,10.0021103 53.5458762,10.0006785 53.5454626,9.9998524 53.5452148,9.9997822 53.5451939,9.9997124 53.545173,9.9984516 53.5447988,9.9984318 53.5447693,9.9986008 53.5446072,9.9987865 53.5444962,9.9988348 53.5444674,9.9989208 53.544416,9.9990693 53.5443273,9.9990926 53.5443134,9.9991144 53.5443017,9.9997297 53.5439723,9.999786 53.5439421,9.9998883 53.5438871),(9.9948565 53.544962,9.9949199 53.5449063,9.9949797 53.5448539,9.9950305 53.5448808,9.9952587 53.5446802,9.9953514 53.5447185,9.9954382 53.5447556,9.997102 53.5453282,9.9972659 53.5453465,9.9974245 53.5453681,9.9975764 53.5453943,9.998013 53.5454773,9.9983613 53.5455436,9.9985213 53.5455794,9.9985762 53.5456464,9.9985014 53.5457994,9.9985307 53.5458114,9.9984937 53.5458914,9.9984704 53.5459416,9.9983827 53.5461449,9.9978874 53.5460518,9.9974951 53.5459434,9.9973288 53.5458899,9.9972758 53.5458997,9.9970169 53.5458431,9.9969925 53.5458161,9.9969301 53.5457857,9.9968734 53.545764,9.9952107 53.5452096,9.9949724 53.5451252,9.9949978 53.5450977,9.9949181 53.5450694,9.9949832 53.5450087,9.9948565 53.544962),(9.9979856 53.5462158,9.9980064 53.5461808,9.9983439 53.5462381,9.9983874 53.5462523,9.9983973 53.5462727,9.9983602 53.5462808,9.9983225 53.546276,9.9979856 53.5462158))'
hamburg_polygon = loads(DEHAM_POLYGON)

def number_of_land_intersections(G: nx.Graph, polygon: BaseGeometry) -> int:
    count = 0
    positions = {node_id: G.node[node_id]['position'] for node_id in G.nodes()}
    for edge in G.edges:
        source = (positions[edge[0]][0], positions[edge[1]][0])
        target = (positions[edge[0]][1], positions[edge[1]][1])
        line = LineString([source, target])
        if polygon.contains(line):
            continue
        count += 1
    return count
                          
n_land_intersect = number_of_land_intersections(G, hamburg_polygon)

In [117]:
# Grading / Report
print('--- Results ---')
print(f'--- The total number of nodes in your graph is {len(G)}')
print(f'--- The number of land intersections of your graph is {n_land_intersect}')
print(nx.shortest_path(G,source=0,target=4))

--- Results ---
--- The total number of nodes in your graph is 10001
--- The number of land intersection of your graph is 51
